In [35]:
import numpy as np
import os
import pandas as pd
from arcpy import env
from arcpy import TableToTable_conversion
import matplotlib.ticker as mtick

In [53]:
dirWork = os.getcwd()
dirData = os.path.join(dirWork, r'data')
dirTDM  = os.path.join(dirData, r'tdm\1_WF')
dirFactors   = os.path.join(dirWork, r'data-processed\factors')
dirResults   = os.path.join(dirWork, r'results')

csv_FacGeoFunGroupAreaTypeVolume_ToStationGroup = os.path.join(dirFactors, r'facgeo_fungroup_areatype_volume_to_station_group.csv')
csv_SeasonFactors = os.path.join(dirFactors, r'SeasonFactors.csv')
csv_SeasonFactors = os.path.join(dirFactors, r'SeasonFactors.csv')
csv_DOWFactors = os.path.join(dirFactors, r'DOWFactors.csv')
csv_TODFactors = os.path.join(dirFactors, r'TODFactors.csv')


shp_Forecasts = r"H:\Shared drives\Analytics Group\1.0 Travel Forecasting\1.1 TDM Application\2019 RTP\Forecasts\WFRC-MAG Forecasts 2021-10-23\results_webapp_data\layers\Utah Statewide Traffic Volume Historic and Forecast.shp"

# Run Settings

In [54]:
#set to true for first time to create CSVs, false to speed up for subsequent times
#process_TDMdbftocsv = True

#export results
export_results = True

# Define Scenarios

In [55]:
df_Scenarios = pd.DataFrame()


#define AdjCode for pivot when model is not related to forecast
#AADT will be adjusted based on difference between scenario and forecast scenario TDMs
#if ForecastScenarioID is note defined, then the associated forecast volume will be used for that year
#DBF don't include filename extension

#df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'Base15',
#                                    'ScenarioName'   : '2015 Base',
#                                    'ScenarioYear'   :  2015,
#                                    'ScenarioGroup'  : 'Base',
#                                    'SegSummaryCSV'  : 'v831_SE15_Net15_Summary_SEGID_Detailed.csv',
#                                    'Process'        : 'Yes'
#                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'Base19',
                                    'ScenarioName'   : '2019 Base',
                                    'ScenarioYear'   :  2019,
                                    'ScenarioGroup'  : 'Base',
                                    'SegSummaryCSV'  : 'v832_SE19_Net19_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
#                                    'PrvScenarioCode': 'Base15'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'TIP24',
                                    'ScenarioName'   : '2024 TIP Projects Scenario',
                                    'ScenarioYear'   :  2024,
                                    'ScenarioGroup'  : 'TIP',
                                    'SegSummaryCSV'  : 'v832_SE24_Net24_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'Base19'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'RTP30',
                                    'ScenarioName'   : '2030 Fiscally-Constrained Scenario',
                                    'ScenarioYear'   :  2030,
                                    'ScenarioGroup'  : 'RTP',
                                    'SegSummaryCSV'  : 'v832_SE30_Net30_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'TIP24'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'RTP40',
                                    'ScenarioName'   : '2040 Fiscally-Constrained Scenario',
                                    'ScenarioYear'   :  2040,
                                    'ScenarioGroup'  : 'RTP',
                                    'SegSummaryCSV'  : 'v832_SE40_Net40_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'RTP30'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode' : 'RTP50',
                                    'ScenarioName' : '2050 Fiscally-Constrained Scenario',
                                    'ScenarioYear' :  2050,
                                    'ScenarioGroup': 'RTP',
                                    'SegSummaryCSV': 'v832_SE50_Net50_Summary_SEGID_Detailed.csv',
                                    'Process'      : 'Yes',
                                    'PrvScenarioCode': 'RTP40'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'NB30',
                                    'ScenarioName'   : '2030 No Build Scenario',
                                    'ScenarioYear'   :  2030,
                                    'ScenarioGroup'  : 'No Build',
                                    'AdjScenarioCode': 'RTP30',
                                    'SegSummaryCSV'  : 'v832_NoBuild_SE30_Net24_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'TIP24'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'NB40',
                                    'ScenarioName'   : '2040 No Build Scenario',
                                    'ScenarioYear'   :  2040,
                                    'ScenarioGroup'  : 'No Build',
                                    'AdjScenarioCode': 'RTP40',
                                    'SegSummaryCSV'  : 'v832_NoBuild_SE40_Net24_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'NB30'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'NB50',
                                    'ScenarioName'   : '2050 No Build Scenario',
                                    'ScenarioYear'   :  2050,
                                    'ScenarioGroup'  : 'No Build',
                                    'AdjScenarioCode': 'RTP50',
                                    'SegSummaryCSV'  : 'v832_NoBuild_SE50_Net24_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'NB40'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF30',
                                    'ScenarioName'   : '2030 Needs-Based Scenario',
                                    'ScenarioYear'   :  2030,
                                    'ScenarioGroup'  : 'Needs',
                                    'AdjScenarioCode': 'RTP30',
                                    'SegSummaryCSV'  : 'v832_Needs_SE30_Net30_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'TIP24'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF40',
                                    'ScenarioName'   : '2040 Needs-Based Scenario',
                                    'ScenarioYear'   :  2040,
                                    'ScenarioGroup'  : 'Needs',
                                    'AdjScenarioCode': 'RTP40',
                                    'SegSummaryCSV'  : 'v832_Needs_SE40_Net40_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'UF30'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF50',
                                    'ScenarioName'   : '2050 Needs-Based Scenario',
                                    'ScenarioYear'   :  2050,
                                    'ScenarioGroup'  : 'Needs',
                                    'AdjScenarioCode': 'RTP50',
                                    'SegSummaryCSV'  : 'v832_Needs_SE50_Net50_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'UF40'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'S50_WithOp',
                                    'ScenarioName'   : '2050 Test Scenario With Operations',
                                    'ScenarioYear'   :  2050,
                                    'ScenarioGroup'  : 'Scenario',
                                    'AdjScenarioCode': 'RTP50',
                                    'SegSummaryCSV'  : 'v832_ExtForcesWithOpProj_SE50_Net50_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'RTP50'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'S50_WithoutOp',
                                    'ScenarioName'   : '2050 Test Scenario Without Operations',
                                    'ScenarioYear'   :  2050,
                                    'ScenarioGroup'  : 'Scenario',
                                    'AdjScenarioCode': 'RTP50',
                                    'SegSummaryCSV'  : 'v832_ExtForcesWithoutOpProj_SE50_Net50_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'RTP50'
                                   },ignore_index=True)

#all scenarios will be read in to allow for pivoting if needed
df_Scenarios.index.name = 'ScenarioID'
df_Scenarios['ScenarioYear'] = pd.to_numeric(df_Scenarios['ScenarioYear'], downcast='integer')
df_Scenarios

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryCSV,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Yes,Base19,Base,2019 Base,2019,v832_SE19_Net19_Summary_SEGID_Detailed.csv,NaN,NaN
1,Yes,TIP24,TIP,2024 TIP Projects Scenario,2024,v832_SE24_Net24_Summary_SEGID_Detailed.csv,Base19,NaN
...,...,...,...,...,...,...,...,...
11,Yes,S50_WithOp,Scenario,2050 Test Scenario With Operations,2050,v832_ExtForcesWithOpProj_SE50_Net50_Summary_SE...,RTP50,RTP50
12,Yes,S50_WithoutOp,Scenario,2050 Test Scenario Without Operations,2050,v832_ExtForcesWithoutOpProj_SE50_Net50_Summary...,RTP50,RTP50


In [56]:
df_Scenarios_toProcess = df_Scenarios[df_Scenarios['Process']=='Yes']
df_Scenarios_toProcess

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryCSV,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Yes,Base19,Base,2019 Base,2019,v832_SE19_Net19_Summary_SEGID_Detailed.csv,NaN,NaN
1,Yes,TIP24,TIP,2024 TIP Projects Scenario,2024,v832_SE24_Net24_Summary_SEGID_Detailed.csv,Base19,NaN
...,...,...,...,...,...,...,...,...
11,Yes,S50_WithOp,Scenario,2050 Test Scenario With Operations,2050,v832_ExtForcesWithOpProj_SE50_Net50_Summary_SE...,RTP50,RTP50
12,Yes,S50_WithoutOp,Scenario,2050 Test Scenario Without Operations,2050,v832_ExtForcesWithoutOpProj_SE50_Net50_Summary...,RTP50,RTP50


In [57]:
df_Scenarios_toAdjust = df_Scenarios_toProcess[df_Scenarios_toProcess['AdjScenarioCode'].notnull()]
df_Scenarios_toAdjust

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryCSV,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
5,Yes,NB30,No Build,2030 No Build Scenario,2030,v832_NoBuild_SE30_Net24_Summary_SEGID_Detailed...,TIP24,RTP30
6,Yes,NB40,No Build,2040 No Build Scenario,2040,v832_NoBuild_SE40_Net24_Summary_SEGID_Detailed...,NB30,RTP40
...,...,...,...,...,...,...,...,...
11,Yes,S50_WithOp,Scenario,2050 Test Scenario With Operations,2050,v832_ExtForcesWithOpProj_SE50_Net50_Summary_SE...,RTP50,RTP50
12,Yes,S50_WithoutOp,Scenario,2050 Test Scenario Without Operations,2050,v832_ExtForcesWithoutOpProj_SE50_Net50_Summary...,RTP50,RTP50


In [58]:
#define functions to get scenario data
def idFromCode (sCode):
    return df_Scenarios.index[df_Scenarios['ScenarioCode'] == sCode].tolist()[0]
def yrFromCode (sCode):
    return df_Scenarios[df_Scenarios['ScenarioCode'] == sCode].iloc[0]['ScenarioYear']
def codeFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioCode']
def yearFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioYear']

#quick tests of functions
#display(idFromCode('50RTP'))
#display(yrFromCode('50RTP'))
#display(codeFromID(4))
#display(yearFromID(4))
#display(df_Scenarios.loc[7]['ScenarioCode'])

# Define Desired V/C Groupings

In [59]:
#season groups

df_VCGroup_Seasons = pd.DataFrame()

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Ann',
                                                'SeasonName'    : 'Annual',
                                                'SEASONGROUP'   : 'S00-Ann',
                                                'SeasonType'    : 'Season'
                                               },ignore_index=True)

#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Win',
#                                                'SeasonName'    : 'Winter',
#                                                'SEASONGROUP'   : 'S01-Win',
#                                                'SeasonType'    : 'Season'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Spr',
#                                                'SeasonName'    : 'Spring',
#                                                'SEASONGROUP'   : 'S02-Spr',
#                                                'SeasonType'    : 'Season'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Sum',
#                                                'SeasonName'    : 'Summer',
#                                                'SEASONGROUP'   : 'S03-Sum',
#                                                'SeasonType'    : 'Season'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Fal',
#                                                'SeasonName'    : 'Fall',
#                                                'SEASONGROUP'   : 'S04-Fal',
#                                                'SeasonType'    : 'Season'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Jan',
#                                                'SeasonName'    : 'January',
#                                                'SEASONGROUP'   : 'M01-Jan',
#                                                'SeasonType'    : 'Month',
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Feb',
#                                                'SeasonName'    : 'February',
#                                                'SEASONGROUP'   : 'M02-Feb',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Mar',
#                                                'SeasonName'    : 'March',
#                                                'SEASONGROUP'   : 'M03-Mar',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Apr',
#                                                'SeasonName'    : 'April',
#                                                'SeasonGroup'   : 'M04-Apr',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'May',
#                                                'SeasonName'    : 'May',
#                                                'SEASONGROUP'   : 'M05-May',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Jun',
#                                                'SeasonName'    : 'June',
#                                                'SEASONGROUP'   : 'M06-Jun',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Jul',
#                                                'SeasonName'    : 'July',
#                                                'SEASONGROUP'   : 'M07-Jul',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Aug',
#                                                'SeasonName'    : 'August',
#                                                'SEASONGROUP'   : 'M08-Aug',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Sep',
#                                                'SeasonName'    : 'September',
#                                                'SEASONGROUP'   : 'M09-Sep',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Oct',
#                                                'SeasonName'    : 'October',
#                                                'SEASONGROUP'   : 'M10-Oct',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Nov',
#                                                'SeasonName'    : 'November',
#                                                'SEASONGROUP'   : 'M11-Nov',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Dec',
#                                                'SeasonName'    : 'December',
#                                                'SEASONGROUP'   : 'M12-Dec',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)

display(df_VCGroup_Seasons)

#day-of-week groups

df_VCGroup_DOWPkHr = pd.DataFrame()

df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'WkAM',
                                                'DOWPkName' : 'AM - Weekday',
                                                'DOWGROUP'  : '1-Weekday (Tu-Th)',
                                                'PERIOD'    : 'AM',
                                                'HRPCTOF'   : 'Prd' #calculate peak Hr using Prd volume
                                               },ignore_index=True)

df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'WkMD',
                                                'DOWPkName' : 'Midday - Weekday',
                                                'DOWGROUP'  : '1-Weekday (Tu-Th)',
                                                'PERIOD'    : 'MD',
                                                'HRPCTOF'   : 'Prd' #calculate peak Hr using Prd volume
                                               },ignore_index=True)

df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'WkPM',
                                                'DOWPkName' : 'PM - Weekday',
                                                'DOWGROUP'  : '1-Weekday (Tu-Th)',
                                                'PERIOD'    : 'PM',
                                                'HRPCTOF'   : 'Prd' #calculate peak Hr using Prd volume
                                               },ignore_index=True)

#df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'FrPM',
#                                                'DOWPkName' : 'PM - Friday',
#                                                'DOWGROUP'  : '5-Friday',
#                                                'PERIOD'    : 'PM',
#                                                'HRPCTOF'   : 'DY' #calculate peak Hr using DY volume
#                                               },ignore_index=True)
#
#df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'SaDY',
#                                                'DOWPkName' : 'Saturday',
#                                                'DOWGROUP'  : '6-Saturday',
#                                                'PERIOD'    : 'DY',
#                                                'HRPCTOF'   : 'DY' #calculate peak Hr using DY volume
#                                               },ignore_index=True)
#
#df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'SuDY',
#                                                'DOWPkName' : 'Sunday',
#                                                'DOWGROUP'  : '7-Sunday',
#                                                'PERIOD'    : 'DY',
#                                                'HRPCTOF'   : 'DY' #calculate peak Hr using DY volume
#                                               },ignore_index=True)

display(df_VCGroup_DOWPkHr)

#directional factor when using DY volumes for peak Hr calcs
#taken from HCM 2000 Exhibit 9-2
dDYDirectionalFactor = 0.6

#directional factor not used in Prd volume calcs, since those include directionality

,SEASONGROUP,SeasonCode,SeasonName,SeasonType
0,S00-Ann,Ann,Annual,Season


,DOWGROUP,DOWPkCode,DOWPkName,HRPCTOF,PERIOD
0,1-Weekday (Tu-Th),WkAM,AM - Weekday,Prd,AM
1,1-Weekday (Tu-Th),WkMD,Midday - Weekday,Prd,MD
2,1-Weekday (Tu-Th),WkPM,PM - Weekday,Prd,PM


In [60]:
import itertools

#create df with all possible season and day-of-weak/peak groups
df_VCGroups = pd.DataFrame(list(itertools.product(df_VCGroup_Seasons['SeasonCode'],df_VCGroup_DOWPkHr['DOWPkCode'])),columns=['SeasonCode','DOWPkCode'])

df_VCGroups = pd.DataFrame(df_VCGroups)
pd.set_option('display.max_rows', df_VCGroups.shape[0]+1)

#display(df_VCGroups)

#add season data
df_VCGroups = pd.DataFrame.merge(df_VCGroups, df_VCGroup_Seasons, on='SeasonCode')

#add dow-peak data
df_VCGroups = pd.DataFrame.merge(df_VCGroups, df_VCGroup_DOWPkHr, on='DOWPkCode')

#set id name
df_VCGroups.index.name = 'VCGroupID'

df_VCGroups['VCGroupCode'] = df_VCGroups['SeasonCode'] + df_VCGroups['DOWPkCode']

df_VCGroups = df_VCGroups.drop(columns=['SeasonCode','DOWPkCode','SeasonName','DOWPkName'])

display(df_VCGroups)

,SEASONGROUP,SeasonType,DOWGROUP,HRPCTOF,PERIOD,VCGroupCode
VCGroupID,,,,,,
0,S00-Ann,Season,1-Weekday (Tu-Th),Prd,AM,AnnWkAM
1,S00-Ann,Season,1-Weekday (Tu-Th),Prd,MD,AnnWkMD
2,S00-Ann,Season,1-Weekday (Tu-Th),Prd,PM,AnnWkPM


# TDM SEGID Detail

In [61]:
#column names from TDM
nameSegID      = 'SEGID'
nameFG         = 'FUNCGROUP'
nameLinks      = 'LINKS'
nameLanes      = 'LANES'
nameVol        = 'VOL'
nameCap1HL     = 'CAP1HL'
nameAT         = 'AREATYPE'
nameFT         = 'FT'
nameFTClass    = 'FTCLASS'
nameFGFac      = 'FGFAC'
namePrdFac     = 'PRDFAC'
nameDYVOL      = 'DY_VOL'
nameDirections = 'DIRECTIONS'
nameDFac       = 'DFAC'
nameMDPct      = 'MDPCT'
nameHVPct      = 'HVPCT'
nameSegTotal   = 'SegDYVol_TDM'

#column for scenario year
nameSID = 'ScenarioID'
nameS =   'ScenarioName'
nameSYr = 'ScenarioYear'

#additional column name variables
nameLT = 'LT'
nameMD = 'MD'
nameHV = 'HV'

nameVolType = 'VolType'
namePrd     = 'Prd'
nameCapPer  = 'CapPer'
nameCap1Hr  = 'Cap1Hr'
nameDir     = 'Dir'

#value for functional group (FG) total row, so that it can be removed
nameFGTotalValue = 'Total'

#arrays for values by direction

aNameDir1Columns   = ['D1_'+nameFT,'D1_'+nameFTClass,'D1_'+nameLanes,'D1_'+nameCap1HL]
aNameDir2Columns   = ['D2_'+nameFT,'D2_'+nameFTClass,'D2_'+nameLanes,'D2_'+nameCap1HL]
aNameDirColumns    = [      nameFT,      nameFTClass,      nameLanes,      nameCap1HL]    
aNameDirIndexNoDir = [nameSegID,nameFG]
aNameDirIndexWiDir = [nameSegID,nameFG,nameDir]

#arrays for values by Prd
aNamePrdFac = ['DY_PRDFAC','AM_PRDFAC','MD_PRDFAC','PM_PRDFAC','EV_PRDFAC'] #add DY_PRDFAC = 1 later in script for calc ease

#arrays for values by direction, Prd
aNameDirPrdDFac      = ['D1_DY_'+nameDFac     ,'D1_AM_'+nameDFac     ,'D1_MD_'+nameDFac     ,'D1_PM_'+nameDFac     ,'D1_EV_'+nameDFac     ,'D2_DY_'+nameDFac     ,'D2_AM_'+nameDFac     ,'D2_MD_'+nameDFac     ,'D2_PM_'+nameDFac     ,'D2_EV_'+nameDFac     ]
aNameDirPrdMDPct = ['D1_DY_'+nameMDPct,'D1_AM_'+nameMDPct,'D1_MD_'+nameMDPct,'D1_PM_'+nameMDPct,'D1_EV_'+nameMDPct,'D2_DY_'+nameMDPct,'D2_AM_'+nameMDPct,'D2_MD_'+nameMDPct,'D2_PM_'+nameMDPct,'D2_EV_'+nameMDPct]
aNameDirPrdHVPct = ['D1_DY_'+nameHVPct,'D1_AM_'+nameHVPct,'D1_MD_'+nameHVPct,'D1_PM_'+nameHVPct,'D1_EV_'+nameHVPct,'D2_DY_'+nameHVPct,'D2_AM_'+nameHVPct,'D2_MD_'+nameHVPct,'D2_PM_'+nameHVPct,'D2_EV_'+nameHVPct]

indexTotal             = [nameSegID]
indexMelt              = [nameSegID,nameFG]
#indexDY             = [nameSegID,nameFG,nameCap1HL,nameLinks,nameLanes,nameAT,nameFT]

#Hrs within Prd, for capacity
dPrdDuration = {
     'PERIOD': ['AM','MD','PM','EV','DY'],
     'PRDHRS': [   3,   6,   3,  12,     24]
     
}
df_PrdDuration = pd.DataFrame(dPrdDuration)

In [62]:
dfs_TDM =[]
dfs_TDM_Total = []

#all scenarios will be read in to allow for pivoting

for (idx, row) in df_Scenarios.iterrows():

    print(row.loc[nameS] + ' CSV Processing')

    csvFileName = os.path.join(dirTDM, row.SegSummaryCSV)

    df_TDM_import = pd.read_csv(csvFileName)
    df_TDM_import[nameFG] = df_TDM_import[nameFG].str.strip() #strip leading and trailing spaces
    
    #totals for DY segment volume
    df_TDM_SegDYVol = df_TDM_import[df_TDM_import[nameFG] == nameFGTotalValue].copy()
    df_TDM_SegDYVol = df_TDM_SegDYVol[[nameSegID,nameDYVOL]]
    df_TDM_SegDYVol = df_TDM_SegDYVol.rename(columns={nameDYVOL: nameSegTotal})
    
    dfs_TDM_Total.append(df_TDM_SegDYVol[[nameSegID,nameSegTotal]])
    
    #display(df_TDM_SegDYVol)
    
    #Remove rows with no links and total
    df_TDM_import         = df_TDM_import[df_TDM_import[nameLinks] > 0                ]
    df_TDM_import         = df_TDM_import[df_TDM_import[nameFG]    != nameFGTotalValue]

    #data with segment level values
    df_TDM_Seg = pd.DataFrame.merge(df_TDM_import[[nameSegID,nameFG,nameAT,nameFGFac]],df_TDM_SegDYVol,on=nameSegID)    
    
    #normalize data with direction fields
    df_Dir1Columns          = df_TDM_import[aNameDirIndexNoDir + aNameDir1Columns].copy()
    df_Dir1Columns.columns  = (aNameDirIndexNoDir + aNameDirColumns)
    df_Dir1Columns[nameDir] = 'D1'
    
    df_Dir2Columns          = df_TDM_import[aNameDirIndexNoDir + aNameDir2Columns].copy()
    df_Dir2Columns.columns  = (aNameDirIndexNoDir + aNameDirColumns)
    df_Dir2Columns[nameDir] = 'D2'
    
    #display(df_Dir1Columns)
    #display(df_Dir2Columns)
    
    df_TDM_Dir = pd.concat([df_Dir1Columns,df_Dir2Columns])
    #display(df_Dir)
    
    #normalize data with Prd fields
    df_TDM_import['DY_PRDFAC'] = 1
    df_TDM_PrdFac = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=aNamePrdFac)

    #directional factors
    df_TDM_PrdFac[namePrd] = df_TDM_PrdFac['variable'].str.get(0) + df_TDM_PrdFac['variable'].str.get(1)
    df_TDM_PrdFac          = df_TDM_PrdFac.rename(columns={'value': namePrdFac})
    df_TDM_PrdFac          = df_TDM_PrdFac.drop(columns=['variable'])
    
    df_TDM_Prd = df_TDM_PrdFac
    
    #display(df_TDM_Prd)
    
    #normalize data with dir/Prd fields
    df_TDM_DFac      = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=aNameDirPrdDFac     )
    df_TDM_MDPct = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=aNameDirPrdMDPct)
    df_TDM_HVPct = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=aNameDirPrdHVPct)

    #directional factors
    df_TDM_DFac[nameDir] = df_TDM_DFac['variable'].str.get(0) + df_TDM_DFac['variable'].str.get(1)
    df_TDM_DFac[namePrd] = df_TDM_DFac['variable'].str.get(3) + df_TDM_DFac['variable'].str.get(4)
    df_TDM_DFac          = df_TDM_DFac.rename(columns={'value': nameDFac})
    df_TDM_DFac          = df_TDM_DFac.drop(columns=['variable'])
    
    #display(df_TDM_DFac)
    
    #medium truck Pcts
    df_TDM_MDPct[nameDir] = df_TDM_MDPct['variable'].str.get(0) + df_TDM_MDPct['variable'].str.get(1)
    df_TDM_MDPct[namePrd] = df_TDM_MDPct['variable'].str.get(3) + df_TDM_MDPct['variable'].str.get(4)
    df_TDM_MDPct          = df_TDM_MDPct.rename(columns={'value': nameMDPct})
    df_TDM_MDPct          = df_TDM_MDPct.drop(columns=['variable'])
    
    #display(df_TDM_MDPct)    
    
    #medium truck Pcts
    df_TDM_HVPct[nameDir] = df_TDM_HVPct['variable'].str.get(0) + df_TDM_HVPct['variable'].str.get(1)
    df_TDM_HVPct[namePrd] = df_TDM_HVPct['variable'].str.get(3) + df_TDM_HVPct['variable'].str.get(4)
    df_TDM_HVPct          = df_TDM_HVPct.rename(columns={'value': nameHVPct})
    df_TDM_HVPct          = df_TDM_HVPct.drop(columns=['variable'])
    
    #display(df_TDM_HVPct)
    
    #merge dir/Prd dataframes
    df_TDM_DirPrd = pd.DataFrame.merge(df_TDM_DFac  ,df_TDM_MDPct,on=(nameSegID,nameFG,nameDir,namePrd))
    df_TDM_DirPrd = pd.DataFrame.merge(df_TDM_DirPrd,df_TDM_HVPct,on=(nameSegID,nameFG,nameDir,namePrd))
    
    #display(df_DirPrd)
    
    df_TDM_byFGDirPrd = pd.DataFrame.merge(df_TDM_Seg          ,df_TDM_Dir   ,on=(nameSegID,nameFG)) #merge direction
    df_TDM_byFGDirPrd = pd.DataFrame.merge(df_TDM_byFGDirPrd,df_TDM_Prd   ,on=(nameSegID,nameFG)) #merge Prd
    df_TDM_byFGDirPrd = pd.DataFrame.merge(df_TDM_byFGDirPrd,df_TDM_DirPrd,on=(nameSegID,nameFG,namePrd,nameDir)) #merge Prd/direction

    #calculate DY directional for peak-Hr-Pct-of-DY calcs
    #assumes when DY volumes is used, TDM directional factor not valid, eg. weekends    
    #both directions use same directional factor
    df_TDM_byFGDirPrd.loc[(df_TDM_byFGDirPrd[namePrd] == 'DY'), nameDFac] = dDYDirectionalFactor
    df_TDM_byFGDirPrd.loc[(df_TDM_byFGDirPrd[namePrd] == 'DY'), namePrd] = 'DY'
        
    df_TDM_byFGDirPrd = df_TDM_byFGDirPrd[[nameSegID,nameAT,nameSegTotal,nameFG,nameFGFac,nameDir,nameLanes,nameFT,nameFTClass,nameCap1HL,nameDFac,namePrd,namePrdFac,nameMDPct,nameHVPct]]
    display(df_TDM_byFGDirPrd)

    dfs_TDM.append(df_TDM_byFGDirPrd)

    df_Check = df_TDM_byFGDirPrd.copy()
    df_Check['FactorsCombined'] = df_Check[nameFGFac] * df_Check[nameDFac] * df_Check[namePrdFac]
    
    #The sum of all factors should 2.2 (1.0 for non-total and 2xDFac(0.6) for totals, add little extra on each side for rounding
    df_Check = df_Check.groupby(indexTotal).agg({'FactorsCombined':[np.sum]})
    df_Check.columns = df_Check.columns.droplevel(1)
    df_Check = df_Check[((df_Check['FactorsCombined'] > 2.2003) | (df_Check['FactorsCombined'] < 2.1997) & (df_Check['FactorsCombined'] != 0))]
    display(print(row.loc[nameS] + ' Checks:'))
    display(df_Check)

print('Done')

2019 Base CSV Processing


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT
0,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.2,6.0
1,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5332,AM,0.1842,2.9,4.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37448,WFRC_8223,4.0,4343.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.5576,PM,0.3914,10.6,5.6
37449,WFRC_8223,4.0,4343.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4736,EV,0.1490,11.0,7.3


2019 Base Checks:


None

,FactorsCombined
SEGID,


...

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT
0,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.3,6.0
1,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6020,AM,0.1912,2.4,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41558,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4294,PM,0.3706,13.1,6.9
41559,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3916,EV,0.1254,12.0,8.7


2050 Test Scenario Without Operations Checks:


None

,FactorsCombined
SEGID,


Done


In [63]:
df_TDM_byFGDirPrd[df_TDM_byFGDirPrd[nameSegID]=='2878_005.4']

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT
29190,2878_005.4,4.0,46570.0,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.6000,DY,1.0000,12.2,4.2
29191,2878_005.4,4.0,46570.0,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.5565,AM,0.1734,7.6,2.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29198,2878_005.4,4.0,46570.0,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6234,PM,0.2125,9.1,3.5
29199,2878_005.4,4.0,46570.0,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6590,EV,0.2912,13.1,4.4


In [64]:
#df_TDM_Seg[df_TDM_Seg[nameSegID]=='0015_282.2']

In [65]:
df_TDM_byFGDirPrd

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT
0,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.3,6.0
1,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6020,AM,0.1912,2.4,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41558,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4294,PM,0.3706,13.1,6.9
41559,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3916,EV,0.1254,12.0,8.7


In [66]:
df_Scenarios_toProcess

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryCSV,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Yes,Base19,Base,2019 Base,2019,v832_SE19_Net19_Summary_SEGID_Detailed.csv,NaN,NaN
1,Yes,TIP24,TIP,2024 TIP Projects Scenario,2024,v832_SE24_Net24_Summary_SEGID_Detailed.csv,Base19,NaN
...,...,...,...,...,...,...,...,...
11,Yes,S50_WithOp,Scenario,2050 Test Scenario With Operations,2050,v832_ExtForcesWithOpProj_SE50_Net50_Summary_SE...,RTP50,RTP50
12,Yes,S50_WithoutOp,Scenario,2050 Test Scenario Without Operations,2050,v832_ExtForcesWithoutOpProj_SE50_Net50_Summary...,RTP50,RTP50


In [67]:
df_Scenarios_toAdjust

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryCSV,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
5,Yes,NB30,No Build,2030 No Build Scenario,2030,v832_NoBuild_SE30_Net24_Summary_SEGID_Detailed...,TIP24,RTP30
6,Yes,NB40,No Build,2040 No Build Scenario,2040,v832_NoBuild_SE40_Net24_Summary_SEGID_Detailed...,NB30,RTP40
...,...,...,...,...,...,...,...,...
11,Yes,S50_WithOp,Scenario,2050 Test Scenario With Operations,2050,v832_ExtForcesWithOpProj_SE50_Net50_Summary_SE...,RTP50,RTP50
12,Yes,S50_WithoutOp,Scenario,2050 Test Scenario Without Operations,2050,v832_ExtForcesWithoutOpProj_SE50_Net50_Summary...,RTP50,RTP50


In [68]:
#calculate adjustment for forecast for those non-RTP scenarios

dfs_ForecastAdj = []

for (idx, row) in df_Scenarios_toAdjust.iterrows():

    #print(row.loc[nameS] + ' CSV Importing')
    
    df_base = dfs_TDM_Total[idFromCode(row.loc['AdjScenarioCode'])]
    df_base.columns = (['SEGID','DYVolBase'])
    df_scenario  = dfs_TDM_Total[idFromCode(row.loc['ScenarioCode'])]
    df_scenario.columns = (['SEGID','DYVolScenario'])
    
    display(df_base)
    display(df_scenario)
    
    df_ForecastAdj = pd.DataFrame.merge(df_scenario,df_base,on=nameSegID,how='left')
    df_ForecastAdj['DYVolAdj'] = df_ForecastAdj['DYVolScenario'] - df_ForecastAdj['DYVolBase']
    
    df_ForecastAdj.to_csv(os.path.join(dirResults,'ForecastAdj_' + row.loc['ScenarioCode'] + '.csv'))
    
    display(df_ForecastAdj)
    
    dfs_ForecastAdj.append(df_ForecastAdj)

,SEGID,DYVolBase
0,0006_146.9,1639.0
5,0006_149.9,2333.1
...,...,...
18990,WFRC_8222,3278.4
18995,WFRC_8223,5883.0


,SEGID,DYVolScenario
0,0006_146.9,1546.9
5,0006_149.9,2471.3
...,...,...
18490,WFRC_8222,3541.9
18495,WFRC_8223,5970.1


,SEGID,DYVolScenario,DYVolBase,DYVolAdj
0,0006_146.9,1546.9,1639.0,-92.1
1,0006_149.9,2471.3,2333.1,138.2
...,...,...,...,...
3698,WFRC_8222,3541.9,3278.4,263.5
3699,WFRC_8223,5970.1,5883.0,87.1


,SEGID,DYVolBase
0,0006_146.9,1715.0
5,0006_149.9,2992.9
...,...,...
19320,WFRC_8222,3716.5
19325,WFRC_8223,6678.8


,SEGID,DYVolScenario
0,0006_146.9,1663.0
5,0006_149.9,3531.6
...,...,...
18490,WFRC_8222,4641.6
18495,WFRC_8223,7278.8


...

,SEGID,DYVolScenario
0,0006_146.9,1856.0
5,0006_149.9,3845.2
...,...,...
19740,WFRC_8222,9016.8
19745,WFRC_8223,8649.0


,SEGID,DYVolScenario,DYVolBase,DYVolAdj
0,0006_146.9,1856.0,1849.0,7.0
1,0006_149.9,3845.2,3869.7,-24.5
...,...,...,...,...
3948,WFRC_8222,9016.8,9166.7,-149.9
3949,WFRC_8223,8649.0,8743.2,-94.2


,SEGID,DYVolBase
0,0006_146.9,1849.0
5,0006_149.9,3869.7
...,...,...
19740,WFRC_8222,9166.7
19745,WFRC_8223,8743.2


,SEGID,DYVolScenario
0,0006_146.9,1836.0
5,0006_149.9,3829.5
...,...,...
19740,WFRC_8222,9351.1
19745,WFRC_8223,9261.5


,SEGID,DYVolScenario,DYVolBase,DYVolAdj
0,0006_146.9,1836.0,1849.0,-13.0
1,0006_149.9,3829.5,3869.7,-40.2
...,...,...,...,...
3948,WFRC_8222,9351.1,9166.7,184.4
3949,WFRC_8223,9261.5,8743.2,518.3


# AADT and AvgAnnualPrdVol Forecasts (Segments, Forecasts, TDM)

In [69]:
#import arcgis libraries
from arcgis.gis import *
gis = GIS()
                        
#import into spatially-enabled DataFrame
sdf_Forecasts = pd.DataFrame.spatial.from_featureclass(shp_Forecasts)
sdf_Forecasts

,FID,Id,RELEASE,SEGID,ROUTE_ID,BMP,EMP,FULLNAME,CO_FIPS,CO_NAME,X,Y,DISTANCE,F2050,F2040,F2030,F2024,CH40TO50,CH30TO40,CH24TO30,CH19TO24,CH19TO50,FNOTES,MOREINFO,WFRC_FLG,AADT2019,AADT2018,AADT2017,AADT2016,AADT2015,AADT2014,AADT2013,AADT2012,AADT2011,AADT2010,AADT2009,AADT2008,AADT2007,AADT2006,AADT2005,AADT2004,AADT2003,AADT2002,AADT2001,AADT2000,AADT1999,AADT1998,AADT1997,AADT1996,AADT1994,AADT1993,AADT1992,AADT1991,AADT1990,AADT1989,AADT1988,AADT1987,AADT1986,AADT1985,AADT1984,AADT1983,AADT1982,AADT1981,FACGEO,CCSGROUP19,FAC_MON,FAC_TUE,FAC_WED,FAC_THU,FAC_FRI,FAC_SAT,FAC_SUN,FAC_WDAVG,FAC_WEAVG,FAC_WEMAX,FAC_JAN,FAC_FEB,FAC_MAR,FAC_APR,FAC_MAY,FAC_JUN,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_WIN,FAC_SPR,FAC_SUM,FAC_FAL,FAC_MAXMO,FAC_MAX,FACMANADJ,SUTRUCKS,CUTRUCKS,PLANAREA,Shape_Leng,SHAPE
0,0,0,,2599_000.5,2599,0.457,0.845,SIDEWINDER DR,43,SUMMIT,458687.3525,4.501838e+06,0.385730,850.0,0.0,0.0,0.0,850.0,0.0,0.0,-681.0,169.0,0,,0.0,681.0,669.0,660.0,645.0,615.0,590.0,580.0,600.0,610.0,940.0,950.0,950.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ParkCity,COK,1.0500,1.0720,1.0800,1.0920,1.148,0.9390,0.6220,1.0740,0.9030,1.1480,1.1420,1.0760,1.0780,0.8200,0.8410,1.0060,1.111,1.0920,0.9830,0.9250,0.8850,1.0740,1.0970,0.9130,1.0700,0.9310,1,1.1420,0,0.0230,0.0050,UDOT,620.772768,"{""paths"": [[[-12411162.427099999, 4962989.7434..."
1,1,0,,1242_002.3,1242,2.305,2.711,1200 E,5,CACHE,433138.4591,4.626175e+06,0.410286,3100.0,0.0,0.0,0.0,3100.0,0.0,0.0,-2847.0,253.0,0,,0.0,2847.0,2797.0,2800.0,2696.0,2560.0,2500.0,2400.0,2500.0,2500.0,2600.0,1900.0,1900.0,2000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Statewide,COS,0.9850,0.9950,1.0200,1.0440,1.142,1.0360,0.7730,1.0120,0.9850,1.1420,0.8510,0.8960,0.9800,1.0150,1.0570,1.0670,1.039,1.0670,1.0550,1.0430,0.9730,0.9270,0.8920,1.0170,1.0570,1.0220,6,1.0670,0,0.0360,0.0260,Cache,660.291812,"{""paths"": [[[-12446035.5269, 5128326.032300003..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8165,8165,0,2019 RTP v8.3.2 20211023,WFRC_8222,WFRC,0.000,0.000,300 W,35,SALT LAKE,424246.0003,4.492593e+06,0.442365,8400.0,3400.0,3000.0,2300.0,5000.0,400.0,700.0,2300.0,8400.0,,0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Statewide,COU,1.0545,1.0851,1.0925,1.1063,1.145,0.8873,0.6293,1.0946,0.7583,0.8873,0.9346,0.9705,0.9965,1.0142,1.0245,1.0222,0.996,1.0223,1.0238,1.0103,0.9944,0.9907,0.9653,1.0117,1.0135,1.0095,5,1.0245,0,0.1097,0.0482,WFRC,0.000000,"{""paths"": [[[-12457180.699468773, 4951449.3762..."
8166,8166,0,2019 RTP v8.3.2 20211023,WFRC_8223,WFRC,0.000,0.000,300 W,35,SALT LAKE,424246.0003,4.492593e+06,0.442365,8000.0,6100.0,5400.0,4300.0,1900.0,700.0,1100.0,4300.0,8000.0,,0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Statewide,COU,1.0545,1.0851,1.0925,1.1063,1.145,0.8873,0.6293,1.0946,0.7583,0.8873,0.9346,0.9705,0.9965,1.0142,1.0245,1.0222,0.996,1.0223,1.0238,1.0103,0.9944,0.9907,0.9653,1.0117,1.0135,1.0095,5,1.0245,0,0.1097,0.0482,WFRC,0.000000,"{""paths"": [[[-12456731.252167525, 4950684.1332..."


In [70]:
for col in sdf_Forecasts.columns: 
    print(col) 

FID
Id
RELEASE
SEGID
ROUTE_ID
BMP
EMP
FULLNAME
CO_FIPS
CO_NAME
X
Y
DISTANCE
F2050
F2040
F2030
F2024
CH40TO50
CH30TO40
CH24TO30
CH19TO24
CH19TO50
FNOTES
MOREINFO
WFRC_FLG
AADT2019
AADT2018
AADT2017
AADT2016
AADT2015
AADT2014
AADT2013
AADT2012
AADT2011
AADT2010
AADT2009
AADT2008
AADT2007
AADT2006
AADT2005
AADT2004
AADT2003
AADT2002
AADT2001
AADT2000
AADT1999
AADT1998
AADT1997
AADT1996
AADT1994
AADT1993
AADT1992
AADT1991
AADT1990
AADT1989
AADT1988
AADT1987
AADT1986
AADT1985
AADT1984
AADT1983
AADT1982
AADT1981
FACGEO
CCSGROUP19
FAC_MON
FAC_TUE
FAC_WED
FAC_THU
FAC_FRI
FAC_SAT
FAC_SUN
FAC_WDAVG
FAC_WEAVG
FAC_WEMAX
FAC_JAN
FAC_FEB
FAC_MAR
FAC_APR
FAC_MAY
FAC_JUN
FAC_JUL
FAC_AUG
FAC_SEP
FAC_OCT
FAC_NOV
FAC_DEC
FAC_WIN
FAC_SPR
FAC_SUM
FAC_FAL
FAC_MAXMO
FAC_MAX
FACMANADJ
SUTRUCKS
CUTRUCKS
PLANAREA
Shape_Leng
SHAPE


In [71]:
df = sdf_Forecasts

df = df[(df[nameSegID]=='2878_005.4')]

df

,FID,Id,RELEASE,SEGID,ROUTE_ID,BMP,EMP,FULLNAME,CO_FIPS,CO_NAME,X,Y,DISTANCE,F2050,F2040,F2030,F2024,CH40TO50,CH30TO40,CH24TO30,CH19TO24,CH19TO50,FNOTES,MOREINFO,WFRC_FLG,AADT2019,AADT2018,AADT2017,AADT2016,AADT2015,AADT2014,AADT2013,AADT2012,AADT2011,AADT2010,AADT2009,AADT2008,AADT2007,AADT2006,AADT2005,AADT2004,AADT2003,AADT2002,AADT2001,AADT2000,AADT1999,AADT1998,AADT1997,AADT1996,AADT1994,AADT1993,AADT1992,AADT1991,AADT1990,AADT1989,AADT1988,AADT1987,AADT1986,AADT1985,AADT1984,AADT1983,AADT1982,AADT1981,FACGEO,CCSGROUP19,FAC_MON,FAC_TUE,FAC_WED,FAC_THU,FAC_FRI,FAC_SAT,FAC_SUN,FAC_WDAVG,FAC_WEAVG,FAC_WEMAX,FAC_JAN,FAC_FEB,FAC_MAR,FAC_APR,FAC_MAY,FAC_JUN,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_WIN,FAC_SPR,FAC_SUM,FAC_FAL,FAC_MAXMO,FAC_MAX,FACMANADJ,SUTRUCKS,CUTRUCKS,PLANAREA,Shape_Leng,SHAPE
6840,6840,0,2019 RTP v8.3.2 20211023,2878_005.4,2878,5.38,5.875,TRIUMPH BLVD,49,UTAH,424851.2939,4.474386e+06,0.503143,32500.0,28500.0,21500.0,38000.0,4000.0,7000.0,-16500.0,36894.0,31394.0,Clubhouse Dr extension in 2030,0,0.0,1106.0,1086.0,1075.0,1047.0,995.0,955.0,945.0,965.0,985.0,990.0,995.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Statewide,COT,1.0506,1.0813,1.0887,1.1071,1.1418,0.9245,0.6061,1.0924,0.7653,0.9245,0.9024,0.9471,0.9888,1.0146,1.0278,1.0352,1.01,1.0396,1.0434,1.0358,0.9936,0.9616,0.937,1.0104,1.0283,1.0243,9,1.0434,0,0.1143,0.0577,MAG,0.0,"{""paths"": [[[-12455184.644384455, 4926180.6876..."


In [72]:
#normalize data with forecasts

meltIndex    = ['SEGID','FACGEO']
valueColumns = ['AADT2019','F2024','F2030','F2040','F2050']
df_Forecasts = pd.melt(sdf_Forecasts, id_vars=meltIndex, value_vars=valueColumns)
df_Forecasts.columns = ('SEGID','FACGEO','FCOL','ForecastAADT')
df_Forecasts['FYEAR'] = (df_Forecasts['FCOL'].str[-4:]).astype('int64')
df_Forecasts['ForecastAADTAdj'] = 0 #placeholder column to preserve location
df_Forecasts['AvgAnnualPrdVol'] = 0 #placeholder column to preserve location
df_Forecasts = df_Forecasts[['SEGID','FYEAR','ForecastAADT','ForecastAADTAdj','AvgAnnualPrdVol','FACGEO']]
df_Forecasts

,SEGID,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO
0,2599_000.5,2019,681.0,0,0,ParkCity
1,1242_002.3,2019,2847.0,0,0,Statewide
...,...,...,...,...,...,...
40833,WFRC_8222,2050,8400.0,0,0,Statewide
40834,WFRC_8223,2050,8000.0,0,0,Statewide


In [73]:
df_Forecasts[(df_Forecasts[nameSegID]=='2878_005.4')]

,SEGID,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO
6840,2878_005.4,2019,1106.0,0,0,Statewide
15007,2878_005.4,2024,38000.0,0,0,Statewide
...,...,...,...,...,...,...
31341,2878_005.4,2040,28500.0,0,0,Statewide
39508,2878_005.4,2050,32500.0,0,0,Statewide


In [74]:
df_Scenarios_toProcess

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryCSV,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Yes,Base19,Base,2019 Base,2019,v832_SE19_Net19_Summary_SEGID_Detailed.csv,NaN,NaN
1,Yes,TIP24,TIP,2024 TIP Projects Scenario,2024,v832_SE24_Net24_Summary_SEGID_Detailed.csv,Base19,NaN
...,...,...,...,...,...,...,...,...
11,Yes,S50_WithOp,Scenario,2050 Test Scenario With Operations,2050,v832_ExtForcesWithOpProj_SE50_Net50_Summary_SE...,RTP50,RTP50
12,Yes,S50_WithoutOp,Scenario,2050 Test Scenario Without Operations,2050,v832_ExtForcesWithoutOpProj_SE50_Net50_Summary...,RTP50,RTP50


In [75]:
#Hrs within Prd, for capacity
dRoundForecast = {
     'RoundIndex'  : [ 1,  2,   3,    4,     5],
     'ForecastBin' : [ 0,100,1000,10000,100000],
     'RoundToValue': [10, 50, 100,  500,  1000]
}
df_RoundForecast = pd.DataFrame(dRoundForecast)
display(df_RoundForecast)

lRoundBins = df_RoundForecast['ForecastBin'].to_list()
lRoundBase = df_RoundForecast['RoundToValue'].to_list()

def forecast_round(x):
    base = lRoundBase[np.digitize([x],lRoundBins)[0]-1]
    return int(base * round(float(x)/base))

#df = pd.Series([11,16,21]).apply(lambda x: custom_round(x, base=5))

,RoundIndex,ForecastBin,RoundToValue
0,1,0,10
1,2,100,50
...,...,...,...
3,4,10000,500
4,5,100000,1000


In [76]:
display(lRoundBase[np.digitize([1],lRoundBins)[0]-1])
display(lRoundBase[np.digitize([54],lRoundBins)[0]-1])
display(lRoundBase[np.digitize([123],lRoundBins)[0]-1])
display(lRoundBase[np.digitize([1254],lRoundBins)[0]-1])
display(lRoundBase[np.digitize([15456],lRoundBins)[0]-1])
display(lRoundBase[np.digitize([2897651],lRoundBins)[0]-1])
display(lRoundBase[np.digitize([1564654654654],lRoundBins)[0]-1])

display(forecast_round(1))
display(forecast_round(54))
display(forecast_round(123))
display(forecast_round(1254))
display(forecast_round(15456))
display(forecast_round(2897651))
display(forecast_round(1564654654654))

10

10

50

100

500

...

100

1300

15500

2898000

1564654655000

In [77]:
#calculate forecast DY volume for each scenario to process

dfs_PrdForecasts = []
df_Forecasts_SYEAR_tocsv = pd.DataFrame()

for (idx, row) in df_Scenarios_toProcess.iterrows():

    _scenarioID = idx
    
    print(row.loc[nameS] + ' Processing')
    print(idx)
    #df_base = dfs_TDM_Total[idFromCode(row.loc['AdjScenarioCode'])]

    #display(dfs_TDM[idx])
    
    df_TDM = dfs_TDM[idx]
    
    #get TDM year
    _scenarioyear = yearFromID(_scenarioID)
    
    #filter only for year of forecast
    
    df_Forecasts['ForecastAADTAdj'] = df_Forecasts['ForecastAADT'] # set adjusted volume so value is set for unadjusted scenarios
    
    df_Forecasts_SYEAR = df_Forecasts[df_Forecasts.FYEAR == _scenarioyear]
    
    #adjust forecast
    
    for (idx, row) in df_Scenarios_toAdjust.iterrows():
        if (_scenarioID == idx):
            #display("adjust this")
            
            #get adjust id
            _adjustId = df_Scenarios_toAdjust.index.get_loc(idx)

            #display(_adjustId)
            #display(dfs_ForecastAdj[_adjustId])
            
            df_Forecasts_Adjust = pd.DataFrame.merge(df_Forecasts_SYEAR,dfs_ForecastAdj[_adjustId],on='SEGID')
            
            #adjust volume. if scenario vol = 0 then zero out forecast volume, otherwise forecast volume is adjusted
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['DYVolScenario'] == 0), 'ForecastAADTAdj'] = 0
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['DYVolScenario']  > 0), 'ForecastAADTAdj'] = df_Forecasts_Adjust['ForecastAADT'] + df_Forecasts_Adjust['DYVolAdj'] 
            
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['DYVolScenario'] == 0), 'ForecastAADTAdj'] = 0
            
            #if adjustment < 0 then bring to 0
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['ForecastAADTAdj']   < 0), 'ForecastAADTAdj'] = 0  
            
            #display(df_Forecasts_Adjust) 
            
            df_Forecasts_Adjust = df_Forecasts_Adjust[['SEGID','FYEAR','ForecastAADT','ForecastAADTAdj','AvgAnnualPrdVol','FACGEO']]
            #df_Forecasts_Adjust = df_Forecasts_Adjust.rename(columns={"DYVolBase": "ForecastAADT"})
            
            #display(df_Forecasts_Adjust['ForecastAADTAdj'])
            df_Forecasts_Adjust['RoundBin'] = np.digitize(df_Forecasts_Adjust['ForecastAADTAdj'],lRoundBins)
            df_Forecasts_Adjust_forRounding = pd.DataFrame.merge(df_Forecasts_Adjust,df_RoundForecast,left_on='RoundBin',right_on='RoundIndex')
            df_Forecasts_Adjust_forRounding['ForecastAADTAdjRounded'] = round((df_Forecasts_Adjust_forRounding['ForecastAADTAdj']/df_Forecasts_Adjust_forRounding['RoundToValue']),0)*df_Forecasts_Adjust_forRounding['RoundToValue']
            df_Forecasts_Adjust_forRounding = df_Forecasts_Adjust_forRounding.fillna(0)
            df_Forecasts_Adjust_forRounding['ForecastAADTAdjRounded'] = df_Forecasts_Adjust_forRounding['ForecastAADTAdjRounded'].astype(int)
            #display(df_Forecasts_Adjust_forRounding[['ForecastAADTAdj','RoundBin','RoundToValue','ForecastAADTAdjRounded']])
            #display(df_Forecasts_Adjust_forRounding[df_Forecasts_Adjust_forRounding['SEGID']=='0091_002.0'])
                    
            df_Forecasts_Adjust_forRounding['ForecastAADTAdj'] = df_Forecasts_Adjust_forRounding['ForecastAADTAdjRounded']
            
            df_Forecasts_Adjust_forRounding = df_Forecasts_Adjust_forRounding.drop(columns=['RoundIndex','ForecastBin','RoundBin','RoundToValue','ForecastAADTAdjRounded'])
            
            df_Forecasts_SYEAR = df_Forecasts_Adjust_forRounding.copy()      
            #display(df_Forecasts_SYEAR[df_Forecasts_SYEAR['SEGID']=='0091_002.0'])
            
    df_Forecasts_SYEAR = df_Forecasts_SYEAR.fillna(0)
    #df_Forecasts_SYEAR['ForecastAADT'] = df_Forecasts_SYEAR['ForecastAADT'].astype(int)
    #df_Forecasts_SYEAR['ForecastAADTAdj'] = df_Forecasts_SYEAR['ForecastAADTAdj'].astype(int)
    
    #display(df_Forecasts_SYEAR)

    df_Forecasts_SYEAR_export = df_Forecasts_SYEAR.copy()
    df_Forecasts_SYEAR_export['ScenarioCode'] = codeFromID(_scenarioID)
    df_Forecasts_SYEAR_export['ForecastAADT'] = df_Forecasts_SYEAR_export['ForecastAADTAdj'] #override for export
    df_Forecasts_SYEAR_export = df_Forecasts_SYEAR_export[['ScenarioCode','SEGID','ForecastAADT']]

    #display(df_Forecasts_SYEAR_export[df_Forecasts_SYEAR_export['SEGID']=='0091_002.0'])
    
    df_Forecasts_SYEAR_tocsv = pd.concat([df_Forecasts_SYEAR_tocsv,df_Forecasts_SYEAR_export])
    
    #display(os.path.join(dirResults + "0_Forecast_" + codeFromID(_scenarioID) + ".csv"))
    #df_Forecasts_SYEAR_export.to_csv(os.path.join(dirResults + r"\\0_Forecast_" + codeFromID(_scenarioID) + ".csv"),index=False)
    
    #join with TDM data
    df_Forecasts_withPrd = pd.DataFrame.merge(df_TDM, df_Forecasts_SYEAR, on=('SEGID'))
    
    df_Forecasts_withPrd['AvgAnnualPrdVol'] = df_Forecasts_withPrd['ForecastAADTAdj'] * df_Forecasts_withPrd[nameFGFac] * df_Forecasts_withPrd[nameDFac] * df_Forecasts_withPrd[namePrdFac]
    
    #display(df_Forecasts_withPrd.head(20))
    
    dfs_PrdForecasts.append(df_Forecasts_withPrd)

display(os.path.join(dirResults + r"\\0_Forecasts.csv"))
df_Forecasts_SYEAR_tocsv.to_csv(os.path.join(dirResults + r"\\0_Forecasts.csv"),index=False)
display('CSV Exported')

2019 Base Processing
0
2024 TIP Projects Scenario Processing
1
2030 Fiscally-Constrained Scenario Processing
2
2040 Fiscally-Constrained Scenario Processing
3
2050 Fiscally-Constrained Scenario Processing
4
2030 No Build Scenario Processing
5
2040 No Build Scenario Processing
6
2050 No Build Scenario Processing
7
2030 Needs-Based Scenario Processing
8
2040 Needs-Based Scenario Processing
9
2050 Needs-Based Scenario Processing
10
2050 Test Scenario With Operations Processing
11
2050 Test Scenario Without Operations Processing
12


'E:\\GitHub\\V-over-C-Calculations\\results\\\\0_Forecasts.csv'

'CSV Exported'

In [78]:
df_Forecasts_SYEAR[df_Forecasts_SYEAR['SEGID']=='2878_005.4']

,SEGID,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO
3209,2878_005.4,2050,32500.0,34500,0,Statewide


In [79]:
#format forecasts as table for joining with other analysis
table = pd.pivot_table(df_Forecasts_SYEAR_tocsv, values='ForecastAADT', index=['SEGID'],
                    columns=['ScenarioCode'], aggfunc=np.sum, fill_value=0)
display(table)

table.to_csv(os.path.join(dirResults + r"\\0_Forecasts_Table.csv"))

ScenarioCode,Base19,NB30,NB40,NB50,RTP30,RTP40,RTP50,S50_WithOp,S50_WithoutOp,TIP24,UF30,UF40,UF50
SEGID,,,,,,,,,,,,,
0006_000.0,415,0,0,0,500,550,600,0,0,450,0,0,0
0006_000.7,415,0,0,0,450,500,550,0,0,450,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
WFRC_8222,0,3300,4300,5200,3000,3400,8400,8300,8600,2300,2600,5900,6600
WFRC_8223,0,5500,6700,8300,5400,6100,8000,7900,8500,4300,4900,8100,6500


In [80]:
segid ='2878_005.4'
pd.set_option("max_rows", 13)
display(df_Forecasts_SYEAR_tocsv[df_Forecasts_SYEAR_tocsv[('SEGID')]==segid])

,ScenarioCode,SEGID,ForecastAADT
6840,Base19,2878_005.4,1106.0
15007,TIP24,2878_005.4,38000.0
23174,RTP30,2878_005.4,21500.0
31341,RTP40,2878_005.4,28500.0
39508,RTP50,2878_005.4,32500.0
3218,NB30,2878_005.4,47500.0
3196,NB40,2878_005.4,57500.0
3119,NB50,2878_005.4,56000.0
3247,UF30,2878_005.4,22000.0
3236,UF40,2878_005.4,27000.0


In [81]:
dfs_PrdForecasts[0]

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO
0,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.2,6.0,2019,1517.0,1517.0,910.200000,Statewide
1,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5332,AM,0.1842,2.9,4.7,2019,1517.0,1517.0,148.992822,Statewide
2,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5058,MD,0.3225,3.5,6.9,2019,1517.0,1517.0,247.453799,Statewide
3,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4941,PM,0.2710,2.4,5.1,2019,1517.0,1517.0,203.127969,Statewide
4,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5466,EV,0.2223,3.8,6.9,2019,1517.0,1517.0,184.329426,Statewide
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37415,WFRC_8223,4.0,4343.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,11.0,6.3,2019,0.0,0.0,0.000000,Statewide
37416,WFRC_8223,4.0,4343.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3618,AM,0.1549,12.6,6.5,2019,0.0,0.0,0.000000,Statewide
37417,WFRC_8223,4.0,4343.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4524,MD,0.3047,11.1,7.0,2019,0.0,0.0,0.000000,Statewide
37418,WFRC_8223,4.0,4343.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.5576,PM,0.3914,10.6,5.6,2019,0.0,0.0,0.000000,Statewide


In [82]:
df = dfs_PrdForecasts[0]

pd.set_option('display.max_rows', 10)

df[(df[nameSegID]=='2878_005.4')].head(10)

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO
28090,2878_005.4,3.0,28507.7,Arterial,1.0,D1,2.0,3.0,Minor Arterial,829,0.6000,DY,1.0000,8.6,3.3,2019,1106.0,1106.0,663.600000,Statewide
28091,2878_005.4,3.0,28507.7,Arterial,1.0,D1,2.0,3.0,Minor Arterial,829,0.7946,AM,0.1755,5.1,1.8,2019,1106.0,1106.0,154.234244,Statewide
28092,2878_005.4,3.0,28507.7,Arterial,1.0,D1,2.0,3.0,Minor Arterial,829,0.5601,MD,0.2914,8.5,3.4,2019,1106.0,1106.0,180.513733,Statewide
28093,2878_005.4,3.0,28507.7,Arterial,1.0,D1,2.0,3.0,Minor Arterial,829,0.3034,PM,0.2727,10.0,3.6,2019,1106.0,1106.0,91.507321,Statewide
28094,2878_005.4,3.0,28507.7,Arterial,1.0,D1,2.0,3.0,Minor Arterial,829,0.5968,EV,0.2604,11.0,4.3,2019,1106.0,1106.0,171.879832,Statewide
28095,2878_005.4,3.0,28507.7,Arterial,1.0,D2,2.0,3.0,Minor Arterial,829,0.6000,DY,1.0000,9.5,3.8,2019,1106.0,1106.0,663.600000,Statewide
28096,2878_005.4,3.0,28507.7,Arterial,1.0,D2,2.0,3.0,Minor Arterial,829,0.2054,AM,0.1755,17.4,6.8,2019,1106.0,1106.0,39.868756,Statewide
28097,2878_005.4,3.0,28507.7,Arterial,1.0,D2,2.0,3.0,Minor Arterial,829,0.4399,MD,0.2914,11.4,4.7,2019,1106.0,1106.0,141.774667,Statewide
28098,2878_005.4,3.0,28507.7,Arterial,1.0,D2,2.0,3.0,Minor Arterial,829,0.6966,PM,0.2727,6.2,2.3,2019,1106.0,1106.0,210.098879,Statewide
28099,2878_005.4,3.0,28507.7,Arterial,1.0,D2,2.0,3.0,Minor Arterial,829,0.4032,EV,0.2604,10.2,4.4,2019,1106.0,1106.0,116.122568,Statewide


# Adjust for Season, DOW

In [83]:
display(df_VCGroup_Seasons)
display(df_VCGroup_DOWPkHr)
display(df_VCGroup_DOWPkHr[['DOWGROUP','PERIOD']].drop_duplicates())

,SEASONGROUP,SeasonCode,SeasonName,SeasonType
0,S00-Ann,Ann,Annual,Season


,DOWGROUP,DOWPkCode,DOWPkName,HRPCTOF,PERIOD
0,1-Weekday (Tu-Th),WkAM,AM - Weekday,Prd,AM
1,1-Weekday (Tu-Th),WkMD,Midday - Weekday,Prd,MD
2,1-Weekday (Tu-Th),WkPM,PM - Weekday,Prd,PM


,DOWGROUP,PERIOD
0,1-Weekday (Tu-Th),AM
1,1-Weekday (Tu-Th),MD
2,1-Weekday (Tu-Th),PM


In [84]:
#read in data from other notebook pre-processing
df_SeasonFactors = pd.read_csv(csv_SeasonFactors)
df_DOWFactors = pd.read_csv(csv_DOWFactors)
df_TODFactors = pd.read_csv(csv_TODFactors)

df_TODFactors.loc[(df_TODFactors['SEASONGROUP']=='Year'),'SEASONGROUP'] = 'S00-Ann'

df_SeasonFactors = pd.DataFrame.merge(df_SeasonFactors, df_VCGroup_Seasons['SEASONGROUP'].drop_duplicates(), on='SEASONGROUP')
df_DOWFactors = pd.DataFrame.merge(df_DOWFactors, df_VCGroup_DOWPkHr['DOWGROUP'].drop_duplicates(), on='DOWGROUP')
df_TODFactors = pd.DataFrame.merge(df_TODFactors, df_VCGroup_Seasons['SEASONGROUP'].drop_duplicates(), on='SEASONGROUP')
df_TODFactors = pd.DataFrame.merge(df_TODFactors, df_VCGroup_DOWPkHr[['DOWGROUP','PERIOD']].drop_duplicates(), on=('DOWGROUP','PERIOD'))

#hard coded for now
df_TODFactors = df_TODFactors[df_TODFactors['YEARGROUP']=='2015-2019']

df_SeasonFactors = df_SeasonFactors.rename(columns={"FuncGroup": nameFG, "AreaType": nameAT})
df_DOWFactors = df_DOWFactors.rename(columns={"FuncGroup": nameFG, "AreaType": nameAT})
df_TODFactors = df_TODFactors.rename(columns={"FuncGroup": nameFG, "AreaType": nameAT})

display(df_SeasonFactors)
display(df_DOWFactors)
display(df_TODFactors)
display(df_TODFactors[df_TODFactors['SEASONGROUP']=='S00-Ann'])

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,STATIONGROUP,FACTOR,SEASONGROUP
0,CO0,1.0,S00-Ann
1,CO1,1.0,S00-Ann
2,CO2,1.0,S00-Ann
3,CO3,1.0,S00-Ann
4,CO4,1.0,S00-Ann
...,...,...,...
32,COW,1.0,S00-Ann
33,COX,1.0,S00-Ann
34,COY,1.0,S00-Ann
35,COZ,1.0,S00-Ann


,STATIONGROUP,FACTOR,DOWGROUP
0,CO0,0.715249,1-Weekday (Tu-Th)
1,CO1,0.824043,1-Weekday (Tu-Th)
2,CO2,0.790750,1-Weekday (Tu-Th)
3,CO3,0.853478,1-Weekday (Tu-Th)
4,CO4,0.904970,1-Weekday (Tu-Th)
...,...,...,...
32,COW,1.067677,1-Weekday (Tu-Th)
33,COX,1.087913,1-Weekday (Tu-Th)
34,COY,0.967245,1-Weekday (Tu-Th)
35,COZ,1.006499,1-Weekday (Tu-Th)


,STATIONGROUP,YEARGROUP,SEASONGROUP,DOWGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
4,301,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4054,Prd
12,302,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3652,Prd
19,303,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3819,Prd
28,304,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.5003,Prd
37,305,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4619,Prd
...,...,...,...,...,...,...,...
11161,SNC,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0836,DY
11170,XX1,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY
11179,XX2,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0804,DY
11188,XX3,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY


,STATIONGROUP,YEARGROUP,SEASONGROUP,DOWGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
4,301,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4054,Prd
12,302,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3652,Prd
19,303,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3819,Prd
28,304,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.5003,Prd
37,305,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4619,Prd
...,...,...,...,...,...,...,...
11161,SNC,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0836,DY
11170,XX1,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY
11179,XX2,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0804,DY
11188,XX3,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY


In [85]:
df_FacGeoFunGroupAreaTypeVolume_ToStationGroup = pd.read_csv(csv_FacGeoFunGroupAreaTypeVolume_ToStationGroup)
display(df_FacGeoFunGroupAreaTypeVolume_ToStationGroup)
display(df_FacGeoFunGroupAreaTypeVolume_ToStationGroup[(df_FacGeoFunGroupAreaTypeVolume_ToStationGroup['FACGEO']=='Statewide') & (df_FacGeoFunGroupAreaTypeVolume_ToStationGroup['AREATYPE']==1) & (df_FacGeoFunGroupAreaTypeVolume_ToStationGroup['FUNCGROUP']=='Arterial')])

,FACGEO,AREATYPE,VOLUMEFROM,VOLUMETO,FUNCGROUP,STATIONGROUP
0,I80NevSR36,1,0,1000000,Arterial,COQ
1,I80NevSR36,2,0,1000000,Arterial,COS
2,I80NevSR36,3,0,1000000,Arterial,COT
3,I80NevSR36,4,0,1000000,Arterial,COU
4,I80NevSR36,5,0,1000000,Arterial,COU
...,...,...,...,...,...,...
583,Southeast,4,1500,1000000,CD Road,XX3
584,Southeast,5,1500,1000000,CD Road,XX3
585,Statewide,1,0,2500,CD Road,XX3
586,Statewide,1,2500,12500,CD Road,XX3


,FACGEO,AREATYPE,VOLUMEFROM,VOLUMETO,FUNCGROUP,STATIONGROUP
144,Statewide,1,0,2500,Arterial,COP
145,Statewide,1,2500,12500,Arterial,COQ
146,Statewide,1,12500,1000000,Arterial,COR


In [86]:
#add season factor

dfs_PrdForecasts_Season = []

#for (idx, row) in dfs_PrdForecasts.iterrows():

for df_PrdForecasts in dfs_PrdForecasts: 
    
    #joing forecasts data with station group matching
    df_PrdForecasts_Season = pd.DataFrame.merge(df_PrdForecasts,df_FacGeoFunGroupAreaTypeVolume_ToStationGroup,on=('FACGEO',nameFG,nameAT),how='left')
    #display(df_PrdForecasts_Season)
    
    #remove unapplicable volume classes
    df_PrdForecasts_Season = df_PrdForecasts_Season[(df_PrdForecasts_Season.ForecastAADT >= df_PrdForecasts_Season.VOLUMEFROM) & (df_PrdForecasts_Season.ForecastAADT < df_PrdForecasts_Season.VOLUMETO)]

    #join to season factors
    df_PrdForecasts_Season = pd.DataFrame.merge(df_PrdForecasts_Season,df_SeasonFactors,on=('STATIONGROUP'),how='left')
    
    #calculate Average Season
    df_PrdForecasts_Season['AvgSsnPrdVol'] = df_PrdForecasts_Season['AvgAnnualPrdVol'] * df_PrdForecasts_Season['FACTOR'] 
    df_PrdForecasts_Season['AvgSsnPrdVol'] = df_PrdForecasts_Season['AvgSsnPrdVol'].astype(int)
    
    
    df_PrdForecasts_Season = df_PrdForecasts_Season.rename(columns=({'FACTOR':'SsnFactor'}))
    
    display(df_PrdForecasts_Season)
    
    dfs_PrdForecasts_Season.append(df_PrdForecasts_Season)
    

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.2,6.0,2019,1517.0,1517.0,910.200000,Statewide,0,2500,COP,1.0,S00-Ann,910
1,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5332,AM,0.1842,2.9,4.7,2019,1517.0,1517.0,148.992822,Statewide,0,2500,COP,1.0,S00-Ann,148
2,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5058,MD,0.3225,3.5,6.9,2019,1517.0,1517.0,247.453799,Statewide,0,2500,COP,1.0,S00-Ann,247
3,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4941,PM,0.2710,2.4,5.1,2019,1517.0,1517.0,203.127969,Statewide,0,2500,COP,1.0,S00-Ann,203
4,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5466,EV,0.2223,3.8,6.9,2019,1517.0,1517.0,184.329426,Statewide,0,2500,COP,1.0,S00-Ann,184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37415,WFRC_8223,4.0,4343.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,11.0,6.3,2019,0.0,0.0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0
37416,WFRC_8223,4.0,4343.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3618,AM,0.1549,12.6,6.5,2019,0.0,0.0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0
37417,WFRC_8223,4.0,4343.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4524,MD,0.3047,11.1,7.0,2019,0.0,0.0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0
37418,WFRC_8223,4.0,4343.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.5576,PM,0.3914,10.6,5.6,2019,0.0,0.0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_146.9,1.0,1577.9,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.3,6.1,2024,1700.0,1700.0,1020.000000,Statewide,0,2500,COP,1.0,S00-Ann,1020
1,0006_146.9,1.0,1577.9,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5081,AM,0.1762,2.7,5.2,2024,1700.0,1700.0,152.196274,Statewide,0,2500,COP,1.0,S00-Ann,152
2,0006_146.9,1.0,1577.9,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4759,MD,0.3276,3.6,7.0,2024,1700.0,1700.0,265.038228,Statewide,0,2500,COP,1.0,S00-Ann,265
3,0006_146.9,1.0,1577.9,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4928,PM,0.2649,2.9,5.1,2024,1700.0,1700.0,221.922624,Statewide,0,2500,COP,1.0,S00-Ann,221
4,0006_146.9,1.0,1577.9,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5114,EV,0.2313,4.0,6.8,2024,1700.0,1700.0,201.087594,Statewide,0,2500,COP,1.0,S00-Ann,201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38015,WFRC_8223,4.0,4732.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,11.6,6.9,2024,4300.0,4300.0,2580.000000,Statewide,0,1000000,COU,1.0,S00-Ann,2580
38016,WFRC_8223,4.0,4732.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3619,AM,0.1470,12.3,7.8,2024,4300.0,4300.0,228.756990,Statewide,0,1000000,COU,1.0,S00-Ann,228
38017,WFRC_8223,4.0,4732.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4887,MD,0.2875,11.0,7.9,2024,4300.0,4300.0,604.155375,Statewide,0,1000000,COU,1.0,S00-Ann,604
38018,WFRC_8223,4.0,4732.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.5162,PM,0.4138,11.7,6.5,2024,4300.0,4300.0,918.495308,Statewide,0,1000000,COU,1.0,S00-Ann,918


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_146.9,1.0,1639.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.2,5.8,2030,1700.0,1700.0,1020.000000,Statewide,0,2500,COP,1.0,S00-Ann,1020
1,0006_146.9,1.0,1639.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5100,AM,0.1830,2.5,4.8,2030,1700.0,1700.0,158.661000,Statewide,0,2500,COP,1.0,S00-Ann,158
2,0006_146.9,1.0,1639.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4927,MD,0.3148,3.5,6.4,2030,1700.0,1700.0,263.673332,Statewide,0,2500,COP,1.0,S00-Ann,263
3,0006_146.9,1.0,1639.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4656,PM,0.2807,3.0,5.2,2030,1700.0,1700.0,222.179664,Statewide,0,2500,COP,1.0,S00-Ann,222
4,0006_146.9,1.0,1639.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5280,EV,0.2215,3.6,6.5,2030,1700.0,1700.0,198.818400,Statewide,0,2500,COP,1.0,S00-Ann,198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39305,WFRC_8223,4.0,5883.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,11.8,6.3,2030,5400.0,5400.0,3240.000000,Statewide,0,1000000,COU,1.0,S00-Ann,3240
39306,WFRC_8223,4.0,5883.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3068,AM,0.1576,11.5,6.5,2030,5400.0,5400.0,261.099072,Statewide,0,1000000,COU,1.0,S00-Ann,261
39307,WFRC_8223,4.0,5883.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3982,MD,0.2982,10.9,6.7,2030,5400.0,5400.0,641.213496,Statewide,0,1000000,COU,1.0,S00-Ann,641
39308,WFRC_8223,4.0,5883.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4378,PM,0.3991,12.8,5.8,2030,5400.0,5400.0,943.520292,Statewide,0,1000000,COU,1.0,S00-Ann,943


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_146.9,1.0,1715.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.0,5.8,2040,1800.0,1800.0,1080.000000,Statewide,0,2500,COP,1.0,S00-Ann,1080
1,0006_146.9,1.0,1715.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5357,AM,0.1907,2.1,4.3,2040,1800.0,1800.0,183.884382,Statewide,0,2500,COP,1.0,S00-Ann,183
2,0006_146.9,1.0,1715.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5014,MD,0.3207,3.7,6.6,2040,1800.0,1800.0,289.438164,Statewide,0,2500,COP,1.0,S00-Ann,289
3,0006_146.9,1.0,1715.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4875,PM,0.2629,2.7,5.1,2040,1800.0,1800.0,230.694750,Statewide,0,2500,COP,1.0,S00-Ann,230
4,0006_146.9,1.0,1715.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5204,EV,0.2257,3.2,6.7,2040,1800.0,1800.0,211.417704,Statewide,0,2500,COP,1.0,S00-Ann,211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40415,WFRC_8223,4.0,6678.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,11.5,7.5,2040,6100.0,6100.0,3660.000000,Statewide,0,1000000,COU,1.0,S00-Ann,3660
40416,WFRC_8223,4.0,6678.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3032,AM,0.1520,13.2,9.8,2040,6100.0,6100.0,281.127040,Statewide,0,1000000,COU,1.0,S00-Ann,281
40417,WFRC_8223,4.0,6678.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4075,MD,0.3030,10.7,7.0,2040,6100.0,6100.0,753.182250,Statewide,0,1000000,COU,1.0,S00-Ann,753
40418,WFRC_8223,4.0,6678.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4136,PM,0.3887,11.2,6.8,2040,6100.0,6100.0,980.674552,Statewide,0,1000000,COU,1.0,S00-Ann,980


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_146.9,1.0,1849.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.3,6.0,2050,2000.0,2000.0,1200.00000,Statewide,0,2500,COP,1.0,S00-Ann,1200
1,0006_146.9,1.0,1849.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6151,AM,0.1823,2.4,4.0,2050,2000.0,2000.0,224.26546,Statewide,0,2500,COP,1.0,S00-Ann,224
2,0006_146.9,1.0,1849.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5070,MD,0.3186,3.6,7.0,2050,2000.0,2000.0,323.06040,Statewide,0,2500,COP,1.0,S00-Ann,323
3,0006_146.9,1.0,1849.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4522,PM,0.2687,3.2,5.5,2050,2000.0,2000.0,243.01228,Statewide,0,2500,COP,1.0,S00-Ann,243
4,0006_146.9,1.0,1849.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4859,EV,0.2304,4.0,6.9,2050,2000.0,2000.0,223.90272,Statewide,0,2500,COP,1.0,S00-Ann,223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41495,WFRC_8223,4.0,8743.2,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,12.4,6.7,2050,8000.0,8000.0,4800.00000,Statewide,0,1000000,COU,1.0,S00-Ann,4800
41496,WFRC_8223,4.0,8743.2,Arterial,1.0,D2,1.0,4.0,Collector,711,0.2719,AM,0.1551,9.6,5.8,2050,8000.0,8000.0,337.37352,Statewide,0,1000000,COU,1.0,S00-Ann,337
41497,WFRC_8223,4.0,8743.2,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3640,MD,0.3273,12.9,6.5,2050,8000.0,8000.0,953.09760,Statewide,0,1000000,COU,1.0,S00-Ann,953
41498,WFRC_8223,4.0,8743.2,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4335,PM,0.3819,13.1,6.6,2050,8000.0,8000.0,1324.42920,Statewide,0,1000000,COU,1.0,S00-Ann,1324


...

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_146.9,1.0,1640.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.2,5.8,2030,1700.0,1700,1020.000000,Statewide,0,2500,COP,1.0,S00-Ann,1020
1,0006_146.9,1.0,1640.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5113,AM,0.1835,2.5,4.8,2030,1700.0,1700,159.500035,Statewide,0,2500,COP,1.0,S00-Ann,159
2,0006_146.9,1.0,1640.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4970,MD,0.3098,3.5,6.5,2030,1700.0,1700,261.750020,Statewide,0,2500,COP,1.0,S00-Ann,261
3,0006_146.9,1.0,1640.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4701,PM,0.2768,3.0,5.2,2030,1700.0,1700,221.210256,Statewide,0,2500,COP,1.0,S00-Ann,221
4,0006_146.9,1.0,1640.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5130,EV,0.2299,3.6,6.5,2030,1700.0,1700,200.495790,Statewide,0,2500,COP,1.0,S00-Ann,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39375,WFRC_8223,4.0,5411.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,11.1,6.4,2030,5400.0,4900,2940.000000,Statewide,0,1000000,COU,1.0,S00-Ann,2940
39376,WFRC_8223,4.0,5411.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.2972,AM,0.1582,13.9,7.0,2030,5400.0,4900,230.383496,Statewide,0,1000000,COU,1.0,S00-Ann,230
39377,WFRC_8223,4.0,5411.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3711,MD,0.2836,9.4,6.1,2030,5400.0,4900,515.695404,Statewide,0,1000000,COU,1.0,S00-Ann,515
39378,WFRC_8223,4.0,5411.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4284,PM,0.4055,11.3,5.9,2030,5400.0,4900,851.209380,Statewide,0,1000000,COU,1.0,S00-Ann,851


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_146.9,1.0,1673.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.0,5.8,2040,1800.0,1800,1080.000000,Statewide,0,2500,COP,1.0,S00-Ann,1080
1,0006_146.9,1.0,1673.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5531,AM,0.1865,2.2,4.3,2040,1800.0,1800,185.675670,Statewide,0,2500,COP,1.0,S00-Ann,185
2,0006_146.9,1.0,1673.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5000,MD,0.3299,3.7,6.6,2040,1800.0,1800,296.910000,Statewide,0,2500,COP,1.0,S00-Ann,296
3,0006_146.9,1.0,1673.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5098,PM,0.2541,2.7,5.1,2040,1800.0,1800,233.172324,Statewide,0,2500,COP,1.0,S00-Ann,233
4,0006_146.9,1.0,1673.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5235,EV,0.2295,3.2,6.7,2040,1800.0,1800,216.257850,Statewide,0,2500,COP,1.0,S00-Ann,216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40545,WFRC_8223,4.0,8677.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,11.3,7.0,2040,6100.0,8100,4860.000000,Statewide,0,1000000,COU,1.0,S00-Ann,4860
40546,WFRC_8223,4.0,8677.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.2273,AM,0.1658,9.9,6.6,2040,6100.0,8100,305.259354,Statewide,0,1000000,COU,1.0,S00-Ann,305
40547,WFRC_8223,4.0,8677.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.2782,MD,0.3140,11.0,7.6,2040,6100.0,8100,707.573880,Statewide,0,1000000,COU,1.0,S00-Ann,707
40548,WFRC_8223,4.0,8677.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4211,PM,0.3714,12.1,6.1,2040,6100.0,8100,1266.811974,Statewide,0,1000000,COU,1.0,S00-Ann,1266


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_146.9,1.0,1829.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.3,6.0,2050,2000.0,2000,1200.000000,Statewide,0,2500,COP,1.0,S00-Ann,1200
1,0006_146.9,1.0,1829.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5996,AM,0.1903,2.4,4.0,2050,2000.0,2000,228.207760,Statewide,0,2500,COP,1.0,S00-Ann,228
2,0006_146.9,1.0,1829.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5000,MD,0.3280,3.5,7.0,2050,2000.0,2000,328.000000,Statewide,0,2500,COP,1.0,S00-Ann,328
3,0006_146.9,1.0,1829.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4734,PM,0.2553,3.2,5.6,2050,2000.0,2000,241.718040,Statewide,0,2500,COP,1.0,S00-Ann,241
4,0006_146.9,1.0,1829.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4964,EV,0.2264,4.1,7.0,2050,2000.0,2000,224.769920,Statewide,0,2500,COP,1.0,S00-Ann,224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41515,WFRC_8223,4.0,7277.7,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,10.3,4.6,2050,8000.0,6500,3900.000000,Statewide,0,1000000,COU,1.0,S00-Ann,3900
41516,WFRC_8223,4.0,7277.7,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3852,AM,0.2035,7.7,2.7,2050,8000.0,6500,509.523300,Statewide,0,1000000,COU,1.0,S00-Ann,509
41517,WFRC_8223,4.0,7277.7,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3027,MD,0.3379,11.8,5.4,2050,8000.0,6500,664.835145,Statewide,0,1000000,COU,1.0,S00-Ann,664
41518,WFRC_8223,4.0,7277.7,Arterial,1.0,D2,1.0,4.0,Collector,711,0.2706,PM,0.3165,8.3,4.4,2050,8000.0,6500,556.691850,Statewide,0,1000000,COU,1.0,S00-Ann,556


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_146.9,1.0,1856.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.3,6.0,2050,2000.0,2000,1200.000000,Statewide,0,2500,COP,1.0,S00-Ann,1200
1,0006_146.9,1.0,1856.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5887,AM,0.1945,2.4,3.9,2050,2000.0,2000,229.004300,Statewide,0,2500,COP,1.0,S00-Ann,229
2,0006_146.9,1.0,1856.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5117,MD,0.3136,3.6,7.0,2050,2000.0,2000,320.938240,Statewide,0,2500,COP,1.0,S00-Ann,320
3,0006_146.9,1.0,1856.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4451,PM,0.2737,3.2,5.5,2050,2000.0,2000,243.647740,Statewide,0,2500,COP,1.0,S00-Ann,243
4,0006_146.9,1.0,1856.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5047,EV,0.2182,4.1,7.0,2050,2000.0,2000,220.251080,Statewide,0,2500,COP,1.0,S00-Ann,220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41495,WFRC_8223,4.0,8649.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,13.0,7.2,2050,8000.0,7900,4740.000000,Statewide,0,1000000,COU,1.0,S00-Ann,4740
41496,WFRC_8223,4.0,8649.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.2782,AM,0.1535,9.8,7.0,2050,8000.0,7900,337.359230,Statewide,0,1000000,COU,1.0,S00-Ann,337
41497,WFRC_8223,4.0,8649.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3144,MD,0.3380,15.1,7.5,2050,8000.0,7900,839.510880,Statewide,0,1000000,COU,1.0,S00-Ann,839
41498,WFRC_8223,4.0,8649.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4236,PM,0.3734,12.9,6.5,2050,8000.0,7900,1249.560696,Statewide,0,1000000,COU,1.0,S00-Ann,1249


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.3,6.0,2050,2000.0,2000,1200.00000,Statewide,0,2500,COP,1.0,S00-Ann,1200
1,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6020,AM,0.1912,2.4,4.0,2050,2000.0,2000,230.20480,Statewide,0,2500,COP,1.0,S00-Ann,230
2,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5130,MD,0.3159,3.6,7.1,2050,2000.0,2000,324.11340,Statewide,0,2500,COP,1.0,S00-Ann,324
3,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4490,PM,0.2734,3.2,5.5,2050,2000.0,2000,245.51320,Statewide,0,2500,COP,1.0,S00-Ann,245
4,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5065,EV,0.2195,4.1,7.0,2050,2000.0,2000,222.35350,Statewide,0,2500,COP,1.0,S00-Ann,222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41495,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,13.0,7.3,2050,8000.0,8500,5100.00000,Statewide,0,1000000,COU,1.0,S00-Ann,5100
41496,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.2711,AM,0.1730,10.5,5.8,2050,8000.0,8500,398.65255,Statewide,0,1000000,COU,1.0,S00-Ann,398
41497,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3302,MD,0.3310,14.2,7.8,2050,8000.0,8500,929.01770,Statewide,0,1000000,COU,1.0,S00-Ann,929
41498,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4294,PM,0.3706,13.1,6.9,2050,8000.0,8500,1352.65294,Statewide,0,1000000,COU,1.0,S00-Ann,1352


In [87]:
#dfs_PrdForecasts[6]
segid='2878_005.4'
i=4
display(dfs_PrdForecasts[i][dfs_PrdForecasts[i]['SEGID']==segid])
display(df_SeasonFactors[df_SeasonFactors['STATIONGROUP']=='COT'])
display(dfs_PrdForecasts_Season[i][dfs_PrdForecasts_Season[i]['SEGID']==segid])

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO
29160,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.6000,DY,1.0000,11.1,4.2,2050,32500.0,32500.0,19500.000000,Statewide
29161,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.5864,AM,0.1825,6.6,2.5,2050,32500.0,32500.0,3478.085000,Statewide
29162,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.4235,MD,0.3227,11.5,4.4,2050,32500.0,32500.0,4441.562125,Statewide
29163,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.3457,PM,0.2110,14.5,5.6,2050,32500.0,32500.0,2370.637750,Statewide
29164,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.3393,EV,0.2838,13.0,4.5,2050,32500.0,32500.0,3129.533550,Statewide
29165,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6000,DY,1.0000,12.5,4.9,2050,32500.0,32500.0,19500.000000,Statewide
29166,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.4136,AM,0.1825,16.1,6.4,2050,32500.0,32500.0,2453.165000,Statewide
29167,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.5765,MD,0.3227,14.3,5.8,2050,32500.0,32500.0,6046.187875,Statewide
29168,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6543,PM,0.2110,8.7,3.5,2050,32500.0,32500.0,4486.862250,Statewide
29169,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6607,EV,0.2838,12.1,4.5,2050,32500.0,32500.0,6093.966450,Statewide


,STATIONGROUP,FACTOR,SEASONGROUP
29,COT,1.0,S00-Ann


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
29160,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.6000,DY,1.0000,11.1,4.2,2050,32500.0,32500.0,19500.000000,Statewide,0,1000000,COU,1.0,S00-Ann,19500
29161,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.5864,AM,0.1825,6.6,2.5,2050,32500.0,32500.0,3478.085000,Statewide,0,1000000,COU,1.0,S00-Ann,3478
29162,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.4235,MD,0.3227,11.5,4.4,2050,32500.0,32500.0,4441.562125,Statewide,0,1000000,COU,1.0,S00-Ann,4441
29163,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.3457,PM,0.2110,14.5,5.6,2050,32500.0,32500.0,2370.637750,Statewide,0,1000000,COU,1.0,S00-Ann,2370
29164,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.3393,EV,0.2838,13.0,4.5,2050,32500.0,32500.0,3129.533550,Statewide,0,1000000,COU,1.0,S00-Ann,3129
29165,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6000,DY,1.0000,12.5,4.9,2050,32500.0,32500.0,19500.000000,Statewide,0,1000000,COU,1.0,S00-Ann,19500
29166,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.4136,AM,0.1825,16.1,6.4,2050,32500.0,32500.0,2453.165000,Statewide,0,1000000,COU,1.0,S00-Ann,2453
29167,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.5765,MD,0.3227,14.3,5.8,2050,32500.0,32500.0,6046.187875,Statewide,0,1000000,COU,1.0,S00-Ann,6046
29168,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6543,PM,0.2110,8.7,3.5,2050,32500.0,32500.0,4486.862250,Statewide,0,1000000,COU,1.0,S00-Ann,4486
29169,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6607,EV,0.2838,12.1,4.5,2050,32500.0,32500.0,6093.966450,Statewide,0,1000000,COU,1.0,S00-Ann,6093


In [88]:
df_DOWFactors

,STATIONGROUP,FACTOR,DOWGROUP
0,CO0,0.715249,1-Weekday (Tu-Th)
1,CO1,0.824043,1-Weekday (Tu-Th)
2,CO2,0.790750,1-Weekday (Tu-Th)
3,CO3,0.853478,1-Weekday (Tu-Th)
4,CO4,0.904970,1-Weekday (Tu-Th)
...,...,...,...
32,COW,1.067677,1-Weekday (Tu-Th)
33,COX,1.087913,1-Weekday (Tu-Th)
34,COY,0.967245,1-Weekday (Tu-Th)
35,COZ,1.006499,1-Weekday (Tu-Th)


In [89]:
#add dow factor

dfs_PrdForecasts_Season_DOW = []

#for (idx, row) in dfs_PrdForecasts.iterrows():

for df_PrdForecasts_Season in dfs_PrdForecasts_Season: 
    
    #joing forecasts data with season factors
    df_PrdForecasts_Season_DOW = pd.DataFrame.merge(df_PrdForecasts_Season,df_DOWFactors,on=('STATIONGROUP'),how='left')
    
    #calculate Average Season DOW
    df_PrdForecasts_Season_DOW['AvgSsnDOWPrdVol'] = df_PrdForecasts_Season_DOW['AvgSsnPrdVol'] * df_PrdForecasts_Season_DOW['FACTOR'] 
    df_PrdForecasts_Season_DOW['AvgSsnDOWPrdVol'] = df_PrdForecasts_Season_DOW['AvgSsnDOWPrdVol'].astype(int)

    df_PrdForecasts_Season_DOW = df_PrdForecasts_Season_DOW.rename(columns=({'FACTOR':'DOWFactor'}))
    
    display(df_PrdForecasts_Season_DOW)
    
    dfs_PrdForecasts_Season_DOW.append(df_PrdForecasts_Season_DOW)

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.2,6.0,2019,1517.0,1517.0,910.200000,Statewide,0,2500,COP,1.0,S00-Ann,910,0.984028,1-Weekday (Tu-Th),895
1,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5332,AM,0.1842,2.9,4.7,2019,1517.0,1517.0,148.992822,Statewide,0,2500,COP,1.0,S00-Ann,148,0.984028,1-Weekday (Tu-Th),145
2,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5058,MD,0.3225,3.5,6.9,2019,1517.0,1517.0,247.453799,Statewide,0,2500,COP,1.0,S00-Ann,247,0.984028,1-Weekday (Tu-Th),243
3,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4941,PM,0.2710,2.4,5.1,2019,1517.0,1517.0,203.127969,Statewide,0,2500,COP,1.0,S00-Ann,203,0.984028,1-Weekday (Tu-Th),199
4,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5466,EV,0.2223,3.8,6.9,2019,1517.0,1517.0,184.329426,Statewide,0,2500,COP,1.0,S00-Ann,184,0.984028,1-Weekday (Tu-Th),181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37415,WFRC_8223,4.0,4343.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,11.0,6.3,2019,0.0,0.0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0
37416,WFRC_8223,4.0,4343.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3618,AM,0.1549,12.6,6.5,2019,0.0,0.0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0
37417,WFRC_8223,4.0,4343.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4524,MD,0.3047,11.1,7.0,2019,0.0,0.0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0
37418,WFRC_8223,4.0,4343.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.5576,PM,0.3914,10.6,5.6,2019,0.0,0.0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_146.9,1.0,1577.9,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.3,6.1,2024,1700.0,1700.0,1020.000000,Statewide,0,2500,COP,1.0,S00-Ann,1020,0.984028,1-Weekday (Tu-Th),1003
1,0006_146.9,1.0,1577.9,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5081,AM,0.1762,2.7,5.2,2024,1700.0,1700.0,152.196274,Statewide,0,2500,COP,1.0,S00-Ann,152,0.984028,1-Weekday (Tu-Th),149
2,0006_146.9,1.0,1577.9,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4759,MD,0.3276,3.6,7.0,2024,1700.0,1700.0,265.038228,Statewide,0,2500,COP,1.0,S00-Ann,265,0.984028,1-Weekday (Tu-Th),260
3,0006_146.9,1.0,1577.9,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4928,PM,0.2649,2.9,5.1,2024,1700.0,1700.0,221.922624,Statewide,0,2500,COP,1.0,S00-Ann,221,0.984028,1-Weekday (Tu-Th),217
4,0006_146.9,1.0,1577.9,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5114,EV,0.2313,4.0,6.8,2024,1700.0,1700.0,201.087594,Statewide,0,2500,COP,1.0,S00-Ann,201,0.984028,1-Weekday (Tu-Th),197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38015,WFRC_8223,4.0,4732.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,11.6,6.9,2024,4300.0,4300.0,2580.000000,Statewide,0,1000000,COU,1.0,S00-Ann,2580,1.094640,1-Weekday (Tu-Th),2824
38016,WFRC_8223,4.0,4732.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3619,AM,0.1470,12.3,7.8,2024,4300.0,4300.0,228.756990,Statewide,0,1000000,COU,1.0,S00-Ann,228,1.094640,1-Weekday (Tu-Th),249
38017,WFRC_8223,4.0,4732.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4887,MD,0.2875,11.0,7.9,2024,4300.0,4300.0,604.155375,Statewide,0,1000000,COU,1.0,S00-Ann,604,1.094640,1-Weekday (Tu-Th),661
38018,WFRC_8223,4.0,4732.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.5162,PM,0.4138,11.7,6.5,2024,4300.0,4300.0,918.495308,Statewide,0,1000000,COU,1.0,S00-Ann,918,1.094640,1-Weekday (Tu-Th),1004


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_146.9,1.0,1639.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.2,5.8,2030,1700.0,1700.0,1020.000000,Statewide,0,2500,COP,1.0,S00-Ann,1020,0.984028,1-Weekday (Tu-Th),1003
1,0006_146.9,1.0,1639.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5100,AM,0.1830,2.5,4.8,2030,1700.0,1700.0,158.661000,Statewide,0,2500,COP,1.0,S00-Ann,158,0.984028,1-Weekday (Tu-Th),155
2,0006_146.9,1.0,1639.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4927,MD,0.3148,3.5,6.4,2030,1700.0,1700.0,263.673332,Statewide,0,2500,COP,1.0,S00-Ann,263,0.984028,1-Weekday (Tu-Th),258
3,0006_146.9,1.0,1639.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4656,PM,0.2807,3.0,5.2,2030,1700.0,1700.0,222.179664,Statewide,0,2500,COP,1.0,S00-Ann,222,0.984028,1-Weekday (Tu-Th),218
4,0006_146.9,1.0,1639.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5280,EV,0.2215,3.6,6.5,2030,1700.0,1700.0,198.818400,Statewide,0,2500,COP,1.0,S00-Ann,198,0.984028,1-Weekday (Tu-Th),194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39305,WFRC_8223,4.0,5883.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,11.8,6.3,2030,5400.0,5400.0,3240.000000,Statewide,0,1000000,COU,1.0,S00-Ann,3240,1.094640,1-Weekday (Tu-Th),3546
39306,WFRC_8223,4.0,5883.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3068,AM,0.1576,11.5,6.5,2030,5400.0,5400.0,261.099072,Statewide,0,1000000,COU,1.0,S00-Ann,261,1.094640,1-Weekday (Tu-Th),285
39307,WFRC_8223,4.0,5883.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3982,MD,0.2982,10.9,6.7,2030,5400.0,5400.0,641.213496,Statewide,0,1000000,COU,1.0,S00-Ann,641,1.094640,1-Weekday (Tu-Th),701
39308,WFRC_8223,4.0,5883.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4378,PM,0.3991,12.8,5.8,2030,5400.0,5400.0,943.520292,Statewide,0,1000000,COU,1.0,S00-Ann,943,1.094640,1-Weekday (Tu-Th),1032


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_146.9,1.0,1715.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.0,5.8,2040,1800.0,1800.0,1080.000000,Statewide,0,2500,COP,1.0,S00-Ann,1080,0.984028,1-Weekday (Tu-Th),1062
1,0006_146.9,1.0,1715.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5357,AM,0.1907,2.1,4.3,2040,1800.0,1800.0,183.884382,Statewide,0,2500,COP,1.0,S00-Ann,183,0.984028,1-Weekday (Tu-Th),180
2,0006_146.9,1.0,1715.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5014,MD,0.3207,3.7,6.6,2040,1800.0,1800.0,289.438164,Statewide,0,2500,COP,1.0,S00-Ann,289,0.984028,1-Weekday (Tu-Th),284
3,0006_146.9,1.0,1715.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4875,PM,0.2629,2.7,5.1,2040,1800.0,1800.0,230.694750,Statewide,0,2500,COP,1.0,S00-Ann,230,0.984028,1-Weekday (Tu-Th),226
4,0006_146.9,1.0,1715.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5204,EV,0.2257,3.2,6.7,2040,1800.0,1800.0,211.417704,Statewide,0,2500,COP,1.0,S00-Ann,211,0.984028,1-Weekday (Tu-Th),207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40415,WFRC_8223,4.0,6678.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,11.5,7.5,2040,6100.0,6100.0,3660.000000,Statewide,0,1000000,COU,1.0,S00-Ann,3660,1.094640,1-Weekday (Tu-Th),4006
40416,WFRC_8223,4.0,6678.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3032,AM,0.1520,13.2,9.8,2040,6100.0,6100.0,281.127040,Statewide,0,1000000,COU,1.0,S00-Ann,281,1.094640,1-Weekday (Tu-Th),307
40417,WFRC_8223,4.0,6678.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4075,MD,0.3030,10.7,7.0,2040,6100.0,6100.0,753.182250,Statewide,0,1000000,COU,1.0,S00-Ann,753,1.094640,1-Weekday (Tu-Th),824
40418,WFRC_8223,4.0,6678.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4136,PM,0.3887,11.2,6.8,2040,6100.0,6100.0,980.674552,Statewide,0,1000000,COU,1.0,S00-Ann,980,1.094640,1-Weekday (Tu-Th),1072


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_146.9,1.0,1849.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.3,6.0,2050,2000.0,2000.0,1200.00000,Statewide,0,2500,COP,1.0,S00-Ann,1200,0.984028,1-Weekday (Tu-Th),1180
1,0006_146.9,1.0,1849.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6151,AM,0.1823,2.4,4.0,2050,2000.0,2000.0,224.26546,Statewide,0,2500,COP,1.0,S00-Ann,224,0.984028,1-Weekday (Tu-Th),220
2,0006_146.9,1.0,1849.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5070,MD,0.3186,3.6,7.0,2050,2000.0,2000.0,323.06040,Statewide,0,2500,COP,1.0,S00-Ann,323,0.984028,1-Weekday (Tu-Th),317
3,0006_146.9,1.0,1849.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4522,PM,0.2687,3.2,5.5,2050,2000.0,2000.0,243.01228,Statewide,0,2500,COP,1.0,S00-Ann,243,0.984028,1-Weekday (Tu-Th),239
4,0006_146.9,1.0,1849.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4859,EV,0.2304,4.0,6.9,2050,2000.0,2000.0,223.90272,Statewide,0,2500,COP,1.0,S00-Ann,223,0.984028,1-Weekday (Tu-Th),219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41495,WFRC_8223,4.0,8743.2,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,12.4,6.7,2050,8000.0,8000.0,4800.00000,Statewide,0,1000000,COU,1.0,S00-Ann,4800,1.094640,1-Weekday (Tu-Th),5254
41496,WFRC_8223,4.0,8743.2,Arterial,1.0,D2,1.0,4.0,Collector,711,0.2719,AM,0.1551,9.6,5.8,2050,8000.0,8000.0,337.37352,Statewide,0,1000000,COU,1.0,S00-Ann,337,1.094640,1-Weekday (Tu-Th),368
41497,WFRC_8223,4.0,8743.2,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3640,MD,0.3273,12.9,6.5,2050,8000.0,8000.0,953.09760,Statewide,0,1000000,COU,1.0,S00-Ann,953,1.094640,1-Weekday (Tu-Th),1043
41498,WFRC_8223,4.0,8743.2,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4335,PM,0.3819,13.1,6.6,2050,8000.0,8000.0,1324.42920,Statewide,0,1000000,COU,1.0,S00-Ann,1324,1.094640,1-Weekday (Tu-Th),1449


...

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_146.9,1.0,1640.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.2,5.8,2030,1700.0,1700,1020.000000,Statewide,0,2500,COP,1.0,S00-Ann,1020,0.984028,1-Weekday (Tu-Th),1003
1,0006_146.9,1.0,1640.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5113,AM,0.1835,2.5,4.8,2030,1700.0,1700,159.500035,Statewide,0,2500,COP,1.0,S00-Ann,159,0.984028,1-Weekday (Tu-Th),156
2,0006_146.9,1.0,1640.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4970,MD,0.3098,3.5,6.5,2030,1700.0,1700,261.750020,Statewide,0,2500,COP,1.0,S00-Ann,261,0.984028,1-Weekday (Tu-Th),256
3,0006_146.9,1.0,1640.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4701,PM,0.2768,3.0,5.2,2030,1700.0,1700,221.210256,Statewide,0,2500,COP,1.0,S00-Ann,221,0.984028,1-Weekday (Tu-Th),217
4,0006_146.9,1.0,1640.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5130,EV,0.2299,3.6,6.5,2030,1700.0,1700,200.495790,Statewide,0,2500,COP,1.0,S00-Ann,200,0.984028,1-Weekday (Tu-Th),196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39375,WFRC_8223,4.0,5411.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,11.1,6.4,2030,5400.0,4900,2940.000000,Statewide,0,1000000,COU,1.0,S00-Ann,2940,1.094640,1-Weekday (Tu-Th),3218
39376,WFRC_8223,4.0,5411.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.2972,AM,0.1582,13.9,7.0,2030,5400.0,4900,230.383496,Statewide,0,1000000,COU,1.0,S00-Ann,230,1.094640,1-Weekday (Tu-Th),251
39377,WFRC_8223,4.0,5411.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3711,MD,0.2836,9.4,6.1,2030,5400.0,4900,515.695404,Statewide,0,1000000,COU,1.0,S00-Ann,515,1.094640,1-Weekday (Tu-Th),563
39378,WFRC_8223,4.0,5411.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4284,PM,0.4055,11.3,5.9,2030,5400.0,4900,851.209380,Statewide,0,1000000,COU,1.0,S00-Ann,851,1.094640,1-Weekday (Tu-Th),931


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_146.9,1.0,1673.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.0,5.8,2040,1800.0,1800,1080.000000,Statewide,0,2500,COP,1.0,S00-Ann,1080,0.984028,1-Weekday (Tu-Th),1062
1,0006_146.9,1.0,1673.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5531,AM,0.1865,2.2,4.3,2040,1800.0,1800,185.675670,Statewide,0,2500,COP,1.0,S00-Ann,185,0.984028,1-Weekday (Tu-Th),182
2,0006_146.9,1.0,1673.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5000,MD,0.3299,3.7,6.6,2040,1800.0,1800,296.910000,Statewide,0,2500,COP,1.0,S00-Ann,296,0.984028,1-Weekday (Tu-Th),291
3,0006_146.9,1.0,1673.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5098,PM,0.2541,2.7,5.1,2040,1800.0,1800,233.172324,Statewide,0,2500,COP,1.0,S00-Ann,233,0.984028,1-Weekday (Tu-Th),229
4,0006_146.9,1.0,1673.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5235,EV,0.2295,3.2,6.7,2040,1800.0,1800,216.257850,Statewide,0,2500,COP,1.0,S00-Ann,216,0.984028,1-Weekday (Tu-Th),212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40545,WFRC_8223,4.0,8677.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,11.3,7.0,2040,6100.0,8100,4860.000000,Statewide,0,1000000,COU,1.0,S00-Ann,4860,1.094640,1-Weekday (Tu-Th),5319
40546,WFRC_8223,4.0,8677.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.2273,AM,0.1658,9.9,6.6,2040,6100.0,8100,305.259354,Statewide,0,1000000,COU,1.0,S00-Ann,305,1.094640,1-Weekday (Tu-Th),333
40547,WFRC_8223,4.0,8677.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.2782,MD,0.3140,11.0,7.6,2040,6100.0,8100,707.573880,Statewide,0,1000000,COU,1.0,S00-Ann,707,1.094640,1-Weekday (Tu-Th),773
40548,WFRC_8223,4.0,8677.8,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4211,PM,0.3714,12.1,6.1,2040,6100.0,8100,1266.811974,Statewide,0,1000000,COU,1.0,S00-Ann,1266,1.094640,1-Weekday (Tu-Th),1385


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_146.9,1.0,1829.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.3,6.0,2050,2000.0,2000,1200.000000,Statewide,0,2500,COP,1.0,S00-Ann,1200,0.984028,1-Weekday (Tu-Th),1180
1,0006_146.9,1.0,1829.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5996,AM,0.1903,2.4,4.0,2050,2000.0,2000,228.207760,Statewide,0,2500,COP,1.0,S00-Ann,228,0.984028,1-Weekday (Tu-Th),224
2,0006_146.9,1.0,1829.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5000,MD,0.3280,3.5,7.0,2050,2000.0,2000,328.000000,Statewide,0,2500,COP,1.0,S00-Ann,328,0.984028,1-Weekday (Tu-Th),322
3,0006_146.9,1.0,1829.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4734,PM,0.2553,3.2,5.6,2050,2000.0,2000,241.718040,Statewide,0,2500,COP,1.0,S00-Ann,241,0.984028,1-Weekday (Tu-Th),237
4,0006_146.9,1.0,1829.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4964,EV,0.2264,4.1,7.0,2050,2000.0,2000,224.769920,Statewide,0,2500,COP,1.0,S00-Ann,224,0.984028,1-Weekday (Tu-Th),220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41515,WFRC_8223,4.0,7277.7,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,10.3,4.6,2050,8000.0,6500,3900.000000,Statewide,0,1000000,COU,1.0,S00-Ann,3900,1.094640,1-Weekday (Tu-Th),4269
41516,WFRC_8223,4.0,7277.7,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3852,AM,0.2035,7.7,2.7,2050,8000.0,6500,509.523300,Statewide,0,1000000,COU,1.0,S00-Ann,509,1.094640,1-Weekday (Tu-Th),557
41517,WFRC_8223,4.0,7277.7,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3027,MD,0.3379,11.8,5.4,2050,8000.0,6500,664.835145,Statewide,0,1000000,COU,1.0,S00-Ann,664,1.094640,1-Weekday (Tu-Th),726
41518,WFRC_8223,4.0,7277.7,Arterial,1.0,D2,1.0,4.0,Collector,711,0.2706,PM,0.3165,8.3,4.4,2050,8000.0,6500,556.691850,Statewide,0,1000000,COU,1.0,S00-Ann,556,1.094640,1-Weekday (Tu-Th),608


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_146.9,1.0,1856.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.3,6.0,2050,2000.0,2000,1200.000000,Statewide,0,2500,COP,1.0,S00-Ann,1200,0.984028,1-Weekday (Tu-Th),1180
1,0006_146.9,1.0,1856.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5887,AM,0.1945,2.4,3.9,2050,2000.0,2000,229.004300,Statewide,0,2500,COP,1.0,S00-Ann,229,0.984028,1-Weekday (Tu-Th),225
2,0006_146.9,1.0,1856.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5117,MD,0.3136,3.6,7.0,2050,2000.0,2000,320.938240,Statewide,0,2500,COP,1.0,S00-Ann,320,0.984028,1-Weekday (Tu-Th),314
3,0006_146.9,1.0,1856.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4451,PM,0.2737,3.2,5.5,2050,2000.0,2000,243.647740,Statewide,0,2500,COP,1.0,S00-Ann,243,0.984028,1-Weekday (Tu-Th),239
4,0006_146.9,1.0,1856.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5047,EV,0.2182,4.1,7.0,2050,2000.0,2000,220.251080,Statewide,0,2500,COP,1.0,S00-Ann,220,0.984028,1-Weekday (Tu-Th),216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41495,WFRC_8223,4.0,8649.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,13.0,7.2,2050,8000.0,7900,4740.000000,Statewide,0,1000000,COU,1.0,S00-Ann,4740,1.094640,1-Weekday (Tu-Th),5188
41496,WFRC_8223,4.0,8649.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.2782,AM,0.1535,9.8,7.0,2050,8000.0,7900,337.359230,Statewide,0,1000000,COU,1.0,S00-Ann,337,1.094640,1-Weekday (Tu-Th),368
41497,WFRC_8223,4.0,8649.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3144,MD,0.3380,15.1,7.5,2050,8000.0,7900,839.510880,Statewide,0,1000000,COU,1.0,S00-Ann,839,1.094640,1-Weekday (Tu-Th),918
41498,WFRC_8223,4.0,8649.0,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4236,PM,0.3734,12.9,6.5,2050,8000.0,7900,1249.560696,Statewide,0,1000000,COU,1.0,S00-Ann,1249,1.094640,1-Weekday (Tu-Th),1367


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.3,6.0,2050,2000.0,2000,1200.00000,Statewide,0,2500,COP,1.0,S00-Ann,1200,0.984028,1-Weekday (Tu-Th),1180
1,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6020,AM,0.1912,2.4,4.0,2050,2000.0,2000,230.20480,Statewide,0,2500,COP,1.0,S00-Ann,230,0.984028,1-Weekday (Tu-Th),226
2,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5130,MD,0.3159,3.6,7.1,2050,2000.0,2000,324.11340,Statewide,0,2500,COP,1.0,S00-Ann,324,0.984028,1-Weekday (Tu-Th),318
3,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4490,PM,0.2734,3.2,5.5,2050,2000.0,2000,245.51320,Statewide,0,2500,COP,1.0,S00-Ann,245,0.984028,1-Weekday (Tu-Th),241
4,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5065,EV,0.2195,4.1,7.0,2050,2000.0,2000,222.35350,Statewide,0,2500,COP,1.0,S00-Ann,222,0.984028,1-Weekday (Tu-Th),218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41495,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,13.0,7.3,2050,8000.0,8500,5100.00000,Statewide,0,1000000,COU,1.0,S00-Ann,5100,1.094640,1-Weekday (Tu-Th),5582
41496,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.2711,AM,0.1730,10.5,5.8,2050,8000.0,8500,398.65255,Statewide,0,1000000,COU,1.0,S00-Ann,398,1.094640,1-Weekday (Tu-Th),435
41497,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3302,MD,0.3310,14.2,7.8,2050,8000.0,8500,929.01770,Statewide,0,1000000,COU,1.0,S00-Ann,929,1.094640,1-Weekday (Tu-Th),1016
41498,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4294,PM,0.3706,13.1,6.9,2050,8000.0,8500,1352.65294,Statewide,0,1000000,COU,1.0,S00-Ann,1352,1.094640,1-Weekday (Tu-Th),1479


In [90]:
df_TODFactors

,STATIONGROUP,YEARGROUP,SEASONGROUP,DOWGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
4,301,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4054,Prd
12,302,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3652,Prd
19,303,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3819,Prd
28,304,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.5003,Prd
37,305,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4619,Prd
...,...,...,...,...,...,...,...
11161,SNC,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0836,DY
11170,XX1,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY
11179,XX2,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0804,DY
11188,XX3,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY


In [91]:
df_VCGroups

,SEASONGROUP,SeasonType,DOWGROUP,HRPCTOF,PERIOD,VCGroupCode
VCGroupID,,,,,,
0,S00-Ann,Season,1-Weekday (Tu-Th),Prd,AM,AnnWkAM
1,S00-Ann,Season,1-Weekday (Tu-Th),Prd,MD,AnnWkMD
2,S00-Ann,Season,1-Weekday (Tu-Th),Prd,PM,AnnWkPM


In [92]:
df_PrdForecasts_Season_DOW

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,3.3,6.0,2050,2000.0,2000,1200.00000,Statewide,0,2500,COP,1.0,S00-Ann,1200,0.984028,1-Weekday (Tu-Th),1180
1,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6020,AM,0.1912,2.4,4.0,2050,2000.0,2000,230.20480,Statewide,0,2500,COP,1.0,S00-Ann,230,0.984028,1-Weekday (Tu-Th),226
2,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5130,MD,0.3159,3.6,7.1,2050,2000.0,2000,324.11340,Statewide,0,2500,COP,1.0,S00-Ann,324,0.984028,1-Weekday (Tu-Th),318
3,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4490,PM,0.2734,3.2,5.5,2050,2000.0,2000,245.51320,Statewide,0,2500,COP,1.0,S00-Ann,245,0.984028,1-Weekday (Tu-Th),241
4,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5065,EV,0.2195,4.1,7.0,2050,2000.0,2000,222.35350,Statewide,0,2500,COP,1.0,S00-Ann,222,0.984028,1-Weekday (Tu-Th),218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41495,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.6000,DY,1.0000,13.0,7.3,2050,8000.0,8500,5100.00000,Statewide,0,1000000,COU,1.0,S00-Ann,5100,1.094640,1-Weekday (Tu-Th),5582
41496,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.2711,AM,0.1730,10.5,5.8,2050,8000.0,8500,398.65255,Statewide,0,1000000,COU,1.0,S00-Ann,398,1.094640,1-Weekday (Tu-Th),435
41497,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.3302,MD,0.3310,14.2,7.8,2050,8000.0,8500,929.01770,Statewide,0,1000000,COU,1.0,S00-Ann,929,1.094640,1-Weekday (Tu-Th),1016
41498,WFRC_8223,4.0,9261.5,Arterial,1.0,D2,1.0,4.0,Collector,711,0.4294,PM,0.3706,13.1,6.9,2050,8000.0,8500,1352.65294,Statewide,0,1000000,COU,1.0,S00-Ann,1352,1.094640,1-Weekday (Tu-Th),1479


In [93]:
df_PrdForecasts_Season_DOW.columns

Index(['SEGID', 'AREATYPE', 'SegDYVol_TDM', 'FUNCGROUP', 'FGFAC', 'Dir',
       'LANES', 'FT', 'FTCLASS', 'CAP1HL', 'DFAC', 'Prd', 'PRDFAC', 'MDPCT',
       'HVPCT', 'FYEAR', 'ForecastAADT', 'ForecastAADTAdj', 'AvgAnnualPrdVol',
       'FACGEO', 'VOLUMEFROM', 'VOLUMETO', 'STATIONGROUP', 'SsnFactor',
       'SEASONGROUP', 'AvgSsnPrdVol', 'DOWFactor', 'DOWGROUP',
       'AvgSsnDOWPrdVol'],
      dtype='object')

In [94]:
#add tod factor

segid = '2878_005.4'

pd.set_option('display.max_columns', None)

dfs_PrdForecasts_Season_DOW_TOD_wGroups = []

i=0

for df_PrdForecasts_Season_DOW in dfs_PrdForecasts_Season_DOW: 

    display(i)

    df_PrdForecasts_Season_DOW = df_PrdForecasts_Season_DOW.rename(columns={'Prd': namePrd + '_TDM'})

    #display(df_PrdForecasts_Season_DOW[namePrd + '_TDM'])
    
    #joing forecasts data with season factors
    
    df_TODFactors_Prd = df_TODFactors[df_TODFactors['HRPCTOF'] == 'Prd']
    df_TODFactors_DY  = df_TODFactors[df_TODFactors['HRPCTOF'] == 'DY']
    
    #get peak Hr as a Pct of Prd
    df_PrdForecasts_Season_DOW_TOD_Prd = pd.DataFrame.merge(df_PrdForecasts_Season_DOW,df_TODFactors_Prd,left_on=('STATIONGROUP','SEASONGROUP','DOWGROUP',namePrd + '_TDM'),right_on=('STATIONGROUP','SEASONGROUP','DOWGROUP','PERIOD'))
    display(df_PrdForecasts_Season_DOW_TOD_Prd[df_PrdForecasts_Season_DOW_TOD_Prd['SEGID']==segid])
    
    #get peak Hr as a Pct of DY
    df_PrdForecasts_Season_DOW_TOD_DY = pd.DataFrame.merge(df_PrdForecasts_Season_DOW,df_TODFactors_DY,left_on=('STATIONGROUP','SEASONGROUP','DOWGROUP',namePrd + '_TDM'),right_on=('STATIONGROUP','SEASONGROUP','DOWGROUP','HRPCTOF'))
 
    df_PrdForecasts_Season_DOW_TOD = pd.concat([df_PrdForecasts_Season_DOW_TOD_Prd, df_PrdForecasts_Season_DOW_TOD_DY])

    #display(df_PrdForecasts_Season_DOW_TOD)
    
    df_PrdForecasts_Season_DOW_TOD_wGroups = pd.DataFrame.merge(df_PrdForecasts_Season_DOW_TOD, df_VCGroups,on=('SEASONGROUP','DOWGROUP','HRPCTOF','PERIOD'))
      
    dfs_PrdForecasts_Season_DOW_TOD_wGroups.append(df_PrdForecasts_Season_DOW_TOD_wGroups)
    
    display(df_PrdForecasts_Season_DOW_TOD)
    
    i=i+1


0

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
14150,2878_005.4,3.0,28507.7,Arterial,1.0,D1,2.0,3.0,Minor Arterial,829,0.7946,AM,0.1755,5.1,1.8,2019,1106.0,1106.0,154.234244,Statewide,0,1000000,COT,1.0,S00-Ann,154,1.092362,1-Weekday (Tu-Th),168,2015-2019,AM,0.3998,Prd
14151,2878_005.4,3.0,28507.7,Arterial,1.0,D2,2.0,3.0,Minor Arterial,829,0.2054,AM,0.1755,17.4,6.8,2019,1106.0,1106.0,39.868756,Statewide,0,1000000,COT,1.0,S00-Ann,39,1.092362,1-Weekday (Tu-Th),42,2015-2019,AM,0.3998,Prd
17160,2878_005.4,3.0,28507.7,Arterial,1.0,D1,2.0,3.0,Minor Arterial,829,0.5601,MD,0.2914,8.5,3.4,2019,1106.0,1106.0,180.513733,Statewide,0,1000000,COT,1.0,S00-Ann,180,1.092362,1-Weekday (Tu-Th),196,2015-2019,MD,0.1921,Prd
17161,2878_005.4,3.0,28507.7,Arterial,1.0,D2,2.0,3.0,Minor Arterial,829,0.4399,MD,0.2914,11.4,4.7,2019,1106.0,1106.0,141.774667,Statewide,0,1000000,COT,1.0,S00-Ann,141,1.092362,1-Weekday (Tu-Th),154,2015-2019,MD,0.1921,Prd
20170,2878_005.4,3.0,28507.7,Arterial,1.0,D1,2.0,3.0,Minor Arterial,829,0.3034,PM,0.2727,10.0,3.6,2019,1106.0,1106.0,91.507321,Statewide,0,1000000,COT,1.0,S00-Ann,91,1.092362,1-Weekday (Tu-Th),99,2015-2019,PM,0.3708,Prd
20171,2878_005.4,3.0,28507.7,Arterial,1.0,D2,2.0,3.0,Minor Arterial,829,0.6966,PM,0.2727,6.2,2.3,2019,1106.0,1106.0,210.098879,Statewide,0,1000000,COT,1.0,S00-Ann,210,1.092362,1-Weekday (Tu-Th),229,2015-2019,PM,0.3708,Prd


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
0,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5332,AM,0.1842,2.9,4.7,2019,1517.0,1517.0,148.992822,Statewide,0,2500,COP,1.0,S00-Ann,148,0.984028,1-Weekday (Tu-Th),145,2015-2019,AM,0.4093,Prd
1,0006_146.9,1.0,1417.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4668,AM,0.1842,3.3,2.9,2019,1517.0,1517.0,130.438578,Statewide,0,2500,COP,1.0,S00-Ann,130,0.984028,1-Weekday (Tu-Th),127,2015-2019,AM,0.4093,Prd
2,0006_149.9,1.0,2028.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4195,AM,0.1798,11.1,11.1,2019,2441.0,2441.0,184.115110,Statewide,0,2500,COP,1.0,S00-Ann,184,0.984028,1-Weekday (Tu-Th),181,2015-2019,AM,0.4093,Prd
3,0006_149.9,1.0,2028.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.5805,AM,0.1798,9.9,7.6,2019,2441.0,2441.0,254.776690,Statewide,0,2500,COP,1.0,S00-Ann,254,0.984028,1-Weekday (Tu-Th),249,2015-2019,AM,0.4093,Prd
4,0006_150.6,1.0,2269.2,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4439,AM,0.1756,10.6,10.3,2019,2441.0,2441.0,190.273118,Statewide,0,2500,COP,1.0,S00-Ann,190,0.984028,1-Weekday (Tu-Th),186,2015-2019,AM,0.4093,Prd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22447,WFRC_8071,1.0,28.8,Arterial,1.0,D1,1.0,5.0,Collector,577,0.6000,DY,1.0000,5.9,11.9,2019,0.0,0.0,0.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,0,0.955525,1-Weekday (Tu-Th),0,2015-2019,MD,0.0772,DY
22448,WFRC_8071,1.0,28.8,Arterial,1.0,D1,1.0,5.0,Collector,577,0.6000,DY,1.0000,5.9,11.9,2019,0.0,0.0,0.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,0,0.955525,1-Weekday (Tu-Th),0,2015-2019,PM,0.0941,DY
22449,WFRC_8071,1.0,28.8,Arterial,1.0,D2,1.0,5.0,Collector,577,0.6000,DY,1.0000,14.5,7.3,2019,0.0,0.0,0.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,0,0.955525,1-Weekday (Tu-Th),0,2015-2019,AM,0.0483,DY
22450,WFRC_8071,1.0,28.8,Arterial,1.0,D2,1.0,5.0,Collector,577,0.6000,DY,1.0000,14.5,7.3,2019,0.0,0.0,0.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,0,0.955525,1-Weekday (Tu-Th),0,2015-2019,MD,0.0772,DY


1

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
14210,2878_005.4,3.0,51608.0,Arterial,1.0,D1,2.0,3.0,Minor Arterial,829,0.6351,AM,0.1613,4.7,1.9,2024,38000.0,38000.0,3892.78194,Statewide,0,1000000,COT,1.0,S00-Ann,3892,1.092362,1-Weekday (Tu-Th),4251,2015-2019,AM,0.3998,Prd
14211,2878_005.4,3.0,51608.0,Arterial,1.0,D2,2.0,3.0,Minor Arterial,829,0.3649,AM,0.1613,12.5,4.6,2024,38000.0,38000.0,2236.61806,Statewide,0,1000000,COT,1.0,S00-Ann,2236,1.092362,1-Weekday (Tu-Th),2442,2015-2019,AM,0.3998,Prd
17272,2878_005.4,3.0,51608.0,Arterial,1.0,D1,2.0,3.0,Minor Arterial,829,0.5247,MD,0.3204,9.1,3.7,2024,38000.0,38000.0,6388.32744,Statewide,0,1000000,COT,1.0,S00-Ann,6388,1.092362,1-Weekday (Tu-Th),6978,2015-2019,MD,0.1921,Prd
17273,2878_005.4,3.0,51608.0,Arterial,1.0,D2,2.0,3.0,Minor Arterial,829,0.4753,MD,0.3204,10.5,4.3,2024,38000.0,38000.0,5786.87256,Statewide,0,1000000,COT,1.0,S00-Ann,5786,1.092362,1-Weekday (Tu-Th),6320,2015-2019,MD,0.1921,Prd
20334,2878_005.4,3.0,51608.0,Arterial,1.0,D1,2.0,3.0,Minor Arterial,829,0.4338,PM,0.1930,9.8,3.9,2024,38000.0,38000.0,3181.48920,Statewide,0,1000000,COT,1.0,S00-Ann,3181,1.092362,1-Weekday (Tu-Th),3474,2015-2019,PM,0.3708,Prd
20335,2878_005.4,3.0,51608.0,Arterial,1.0,D2,2.0,3.0,Minor Arterial,829,0.5662,PM,0.1930,5.7,2.2,2024,38000.0,38000.0,4152.51080,Statewide,0,1000000,COT,1.0,S00-Ann,4152,1.092362,1-Weekday (Tu-Th),4535,2015-2019,PM,0.3708,Prd


...

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
13688,2878_005.4,4.0,46786.5,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.5628,AM,0.1735,7.7,2.7,2050,32500.0,34500,3368.78010,Statewide,0,1000000,COU,1.0,S00-Ann,3368,1.09464,1-Weekday (Tu-Th),3686,2015-2019,AM,0.4029,Prd
13689,2878_005.4,4.0,46786.5,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.4372,AM,0.1735,17.0,6.5,2050,32500.0,34500,2616.96990,Statewide,0,1000000,COU,1.0,S00-Ann,2616,1.09464,1-Weekday (Tu-Th),2863,2015-2019,AM,0.4029,Prd
17476,2878_005.4,4.0,46786.5,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.4278,MD,0.3207,12.3,4.5,2050,32500.0,34500,4733.24337,Statewide,0,1000000,COU,1.0,S00-Ann,4733,1.09464,1-Weekday (Tu-Th),5180,2015-2019,MD,0.1956,Prd
17477,2878_005.4,4.0,46786.5,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.5722,MD,0.3207,14.4,5.6,2050,32500.0,34500,6330.90663,Statewide,0,1000000,COU,1.0,S00-Ann,6330,1.09464,1-Weekday (Tu-Th),6929,2015-2019,MD,0.1956,Prd
21264,2878_005.4,4.0,46786.5,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.3806,PM,0.2158,15.0,5.0,2050,32500.0,34500,2833.60506,Statewide,0,1000000,COU,1.0,S00-Ann,2833,1.09464,1-Weekday (Tu-Th),3101,2015-2019,PM,0.3615,Prd
21265,2878_005.4,4.0,46786.5,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6194,PM,0.2158,9.2,3.4,2050,32500.0,34500,4611.49494,Statewide,0,1000000,COU,1.0,S00-Ann,4611,1.09464,1-Weekday (Tu-Th),5047,2015-2019,PM,0.3615,Prd


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
0,0006_146.9,1.0,1856.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5887,AM,0.1945,2.4,3.9,2050,2000.0,2000,229.004300,Statewide,0,2500,COP,1.0,S00-Ann,229,0.984028,1-Weekday (Tu-Th),225,2015-2019,AM,0.4093,Prd
1,0006_146.9,1.0,1856.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4113,AM,0.1945,3.4,3.1,2050,2000.0,2000,159.995700,Statewide,0,2500,COP,1.0,S00-Ann,159,0.984028,1-Weekday (Tu-Th),156,2015-2019,AM,0.4093,Prd
2,0073_020.7,1.0,1161.3,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1049,0.5013,AM,0.1492,15.8,7.5,2050,1900.0,1900,142.108524,Statewide,0,2500,COP,1.0,S00-Ann,142,0.984028,1-Weekday (Tu-Th),139,2015-2019,AM,0.4093,Prd
3,0073_020.7,1.0,1161.3,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1049,0.4987,AM,0.1492,19.6,8.0,2050,1900.0,1900,141.371476,Statewide,0,2500,COP,1.0,S00-Ann,141,0.984028,1-Weekday (Tu-Th),138,2015-2019,AM,0.4093,Prd
4,0147_003.1,1.0,591.9,Arterial,1.0,D1,1.0,3.0,Minor Arterial,983,0.4353,AM,0.1436,13.5,2.7,2050,1000.0,950,59.383626,Statewide,0,2500,COP,1.0,S00-Ann,59,0.984028,1-Weekday (Tu-Th),58,2015-2019,AM,0.4093,Prd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24895,WFRC_8071,1.0,64.6,Arterial,1.0,D1,1.0,5.0,Collector,577,0.6000,DY,1.0000,8.8,7.9,2050,40.0,40,24.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,24,0.955525,1-Weekday (Tu-Th),22,2015-2019,MD,0.0772,DY
24896,WFRC_8071,1.0,64.6,Arterial,1.0,D1,1.0,5.0,Collector,577,0.6000,DY,1.0000,8.8,7.9,2050,40.0,40,24.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,24,0.955525,1-Weekday (Tu-Th),22,2015-2019,PM,0.0941,DY
24897,WFRC_8071,1.0,64.6,Arterial,1.0,D2,1.0,5.0,Collector,577,0.6000,DY,1.0000,8.1,12.2,2050,40.0,40,24.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,24,0.955525,1-Weekday (Tu-Th),22,2015-2019,AM,0.0483,DY
24898,WFRC_8071,1.0,64.6,Arterial,1.0,D2,1.0,5.0,Collector,577,0.6000,DY,1.0000,8.1,12.2,2050,40.0,40,24.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,24,0.955525,1-Weekday (Tu-Th),22,2015-2019,MD,0.0772,DY


12

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
13688,2878_005.4,4.0,46570.0,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.5565,AM,0.1734,7.6,2.7,2050,32500.0,34500,3329.14995,Statewide,0,1000000,COU,1.0,S00-Ann,3329,1.09464,1-Weekday (Tu-Th),3644,2015-2019,AM,0.4029,Prd
13689,2878_005.4,4.0,46570.0,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.4435,AM,0.1734,17.2,6.4,2050,32500.0,34500,2653.15005,Statewide,0,1000000,COU,1.0,S00-Ann,2653,1.09464,1-Weekday (Tu-Th),2904,2015-2019,AM,0.4029,Prd
17476,2878_005.4,4.0,46570.0,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.4284,MD,0.3229,12.5,4.4,2050,32500.0,34500,4772.39742,Statewide,0,1000000,COU,1.0,S00-Ann,4772,1.09464,1-Weekday (Tu-Th),5223,2015-2019,MD,0.1956,Prd
17477,2878_005.4,4.0,46570.0,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.5716,MD,0.3229,14.4,5.4,2050,32500.0,34500,6367.65258,Statewide,0,1000000,COU,1.0,S00-Ann,6367,1.09464,1-Weekday (Tu-Th),6969,2015-2019,MD,0.1956,Prd
21264,2878_005.4,4.0,46570.0,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.3766,PM,0.2125,15.0,5.2,2050,32500.0,34500,2760.94875,Statewide,0,1000000,COU,1.0,S00-Ann,2760,1.09464,1-Weekday (Tu-Th),3021,2015-2019,PM,0.3615,Prd
21265,2878_005.4,4.0,46570.0,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6234,PM,0.2125,9.1,3.5,2050,32500.0,34500,4570.30125,Statewide,0,1000000,COU,1.0,S00-Ann,4570,1.09464,1-Weekday (Tu-Th),5002,2015-2019,PM,0.3615,Prd


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
0,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6020,AM,0.1912,2.4,4.0,2050,2000.0,2000,230.204800,Statewide,0,2500,COP,1.0,S00-Ann,230,0.984028,1-Weekday (Tu-Th),226,2015-2019,AM,0.4093,Prd
1,0006_146.9,1.0,1836.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3980,AM,0.1912,3.6,3.3,2050,2000.0,2000,152.195200,Statewide,0,2500,COP,1.0,S00-Ann,152,0.984028,1-Weekday (Tu-Th),149,2015-2019,AM,0.4093,Prd
2,0073_020.7,1.0,1174.3,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1049,0.4889,AM,0.1478,16.2,7.7,2050,1900.0,1900,137.292898,Statewide,0,2500,COP,1.0,S00-Ann,137,0.984028,1-Weekday (Tu-Th),134,2015-2019,AM,0.4093,Prd
3,0073_020.7,1.0,1174.3,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1049,0.5111,AM,0.1478,18.8,7.5,2050,1900.0,1900,143.527102,Statewide,0,2500,COP,1.0,S00-Ann,143,0.984028,1-Weekday (Tu-Th),140,2015-2019,AM,0.4093,Prd
4,0147_003.1,1.0,593.8,Arterial,1.0,D1,1.0,3.0,Minor Arterial,983,0.4767,AM,0.1480,19.1,4.8,2050,1000.0,950,67.024020,Statewide,0,2500,COP,1.0,S00-Ann,67,0.984028,1-Weekday (Tu-Th),65,2015-2019,AM,0.4093,Prd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24895,WFRC_8071,1.0,60.7,Arterial,1.0,D1,1.0,5.0,Collector,577,0.6000,DY,1.0000,8.8,7.9,2050,40.0,40,24.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,24,0.955525,1-Weekday (Tu-Th),22,2015-2019,MD,0.0772,DY
24896,WFRC_8071,1.0,60.7,Arterial,1.0,D1,1.0,5.0,Collector,577,0.6000,DY,1.0000,8.8,7.9,2050,40.0,40,24.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,24,0.955525,1-Weekday (Tu-Th),22,2015-2019,PM,0.0941,DY
24897,WFRC_8071,1.0,60.7,Arterial,1.0,D2,1.0,5.0,Collector,577,0.6000,DY,1.0000,4.5,14.0,2050,40.0,40,24.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,24,0.955525,1-Weekday (Tu-Th),22,2015-2019,AM,0.0483,DY
24898,WFRC_8071,1.0,60.7,Arterial,1.0,D2,1.0,5.0,Collector,577,0.6000,DY,1.0000,4.5,14.0,2050,40.0,40,24.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,24,0.955525,1-Weekday (Tu-Th),22,2015-2019,MD,0.0772,DY


In [100]:
#TROUBLESHOOTING

#df_PrdForecasts_Season_DOW_TOD_DY[(df_PrdForecasts_Season_DOW_TOD_DY['SEGID']=='0006_146.9') & (df_PrdForecasts_Season_DOW_TOD_DY['DOWGroup']=='6-Saturday')].head(20)

segid = '2878_005.4'
i=4

ccsgroup = 'COT'
display(dfs_PrdForecasts_Season[i][dfs_PrdForecasts_Season[i]['SEGID']==segid])
display(dfs_PrdForecasts_Season_DOW[i][dfs_PrdForecasts_Season_DOW[i]['SEGID']==segid])
display(dfs_PrdForecasts_Season_DOW_TOD_wGroups[i][dfs_PrdForecasts_Season_DOW_TOD_wGroups[i]['SEGID']==segid])

display(df_TODFactors_Prd[df_TODFactors_Prd['STATIONGROUP']==ccsgroup])

display(df_PrdForecasts_Season_DOW_TOD_Prd[df_PrdForecasts_Season_DOW_TOD_Prd['SEGID']==segid])

display(df_PrdForecasts_Season_DOW_TOD_DY[(df_PrdForecasts_Season_DOW_TOD_DY['SEGID']==segid)].head(20))


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
29160,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.6000,DY,1.0000,11.1,4.2,2050,32500.0,32500.0,19500.000000,Statewide,0,1000000,COU,1.0,S00-Ann,19500
29161,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.5864,AM,0.1825,6.6,2.5,2050,32500.0,32500.0,3478.085000,Statewide,0,1000000,COU,1.0,S00-Ann,3478
29162,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.4235,MD,0.3227,11.5,4.4,2050,32500.0,32500.0,4441.562125,Statewide,0,1000000,COU,1.0,S00-Ann,4441
29163,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.3457,PM,0.2110,14.5,5.6,2050,32500.0,32500.0,2370.637750,Statewide,0,1000000,COU,1.0,S00-Ann,2370
29164,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.3393,EV,0.2838,13.0,4.5,2050,32500.0,32500.0,3129.533550,Statewide,0,1000000,COU,1.0,S00-Ann,3129
29165,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6000,DY,1.0000,12.5,4.9,2050,32500.0,32500.0,19500.000000,Statewide,0,1000000,COU,1.0,S00-Ann,19500
29166,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.4136,AM,0.1825,16.1,6.4,2050,32500.0,32500.0,2453.165000,Statewide,0,1000000,COU,1.0,S00-Ann,2453
29167,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.5765,MD,0.3227,14.3,5.8,2050,32500.0,32500.0,6046.187875,Statewide,0,1000000,COU,1.0,S00-Ann,6046
29168,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6543,PM,0.2110,8.7,3.5,2050,32500.0,32500.0,4486.862250,Statewide,0,1000000,COU,1.0,S00-Ann,4486
29169,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6607,EV,0.2838,12.1,4.5,2050,32500.0,32500.0,6093.966450,Statewide,0,1000000,COU,1.0,S00-Ann,6093


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
29160,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.6000,DY,1.0000,11.1,4.2,2050,32500.0,32500.0,19500.000000,Statewide,0,1000000,COU,1.0,S00-Ann,19500,1.09464,1-Weekday (Tu-Th),21345
29161,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.5864,AM,0.1825,6.6,2.5,2050,32500.0,32500.0,3478.085000,Statewide,0,1000000,COU,1.0,S00-Ann,3478,1.09464,1-Weekday (Tu-Th),3807
29162,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.4235,MD,0.3227,11.5,4.4,2050,32500.0,32500.0,4441.562125,Statewide,0,1000000,COU,1.0,S00-Ann,4441,1.09464,1-Weekday (Tu-Th),4861
29163,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.3457,PM,0.2110,14.5,5.6,2050,32500.0,32500.0,2370.637750,Statewide,0,1000000,COU,1.0,S00-Ann,2370,1.09464,1-Weekday (Tu-Th),2594
29164,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.3393,EV,0.2838,13.0,4.5,2050,32500.0,32500.0,3129.533550,Statewide,0,1000000,COU,1.0,S00-Ann,3129,1.09464,1-Weekday (Tu-Th),3425
29165,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6000,DY,1.0000,12.5,4.9,2050,32500.0,32500.0,19500.000000,Statewide,0,1000000,COU,1.0,S00-Ann,19500,1.09464,1-Weekday (Tu-Th),21345
29166,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.4136,AM,0.1825,16.1,6.4,2050,32500.0,32500.0,2453.165000,Statewide,0,1000000,COU,1.0,S00-Ann,2453,1.09464,1-Weekday (Tu-Th),2685
29167,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.5765,MD,0.3227,14.3,5.8,2050,32500.0,32500.0,6046.187875,Statewide,0,1000000,COU,1.0,S00-Ann,6046,1.09464,1-Weekday (Tu-Th),6618
29168,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6543,PM,0.2110,8.7,3.5,2050,32500.0,32500.0,4486.862250,Statewide,0,1000000,COU,1.0,S00-Ann,4486,1.09464,1-Weekday (Tu-Th),4910
29169,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6607,EV,0.2838,12.1,4.5,2050,32500.0,32500.0,6093.966450,Statewide,0,1000000,COU,1.0,S00-Ann,6093,1.09464,1-Weekday (Tu-Th),6669


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode
6408,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.5864,AM,0.1825,6.6,2.5,2050,32500.0,32500.0,3478.085000,Statewide,0,1000000,COU,1.0,S00-Ann,3478,1.09464,1-Weekday (Tu-Th),3807,2015-2019,AM,0.4029,Prd,Season,AnnWkAM
6409,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.4136,AM,0.1825,16.1,6.4,2050,32500.0,32500.0,2453.165000,Statewide,0,1000000,COU,1.0,S00-Ann,2453,1.09464,1-Weekday (Tu-Th),2685,2015-2019,AM,0.4029,Prd,Season,AnnWkAM
14708,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.4235,MD,0.3227,11.5,4.4,2050,32500.0,32500.0,4441.562125,Statewide,0,1000000,COU,1.0,S00-Ann,4441,1.09464,1-Weekday (Tu-Th),4861,2015-2019,MD,0.1956,Prd,Season,AnnWkMD
14709,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.5765,MD,0.3227,14.3,5.8,2050,32500.0,32500.0,6046.187875,Statewide,0,1000000,COU,1.0,S00-Ann,6046,1.09464,1-Weekday (Tu-Th),6618,2015-2019,MD,0.1956,Prd,Season,AnnWkMD
23008,2878_005.4,4.0,44706.6,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.3457,PM,0.2110,14.5,5.6,2050,32500.0,32500.0,2370.637750,Statewide,0,1000000,COU,1.0,S00-Ann,2370,1.09464,1-Weekday (Tu-Th),2594,2015-2019,PM,0.3615,Prd,Season,AnnWkPM
23009,2878_005.4,4.0,44706.6,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6543,PM,0.2110,8.7,3.5,2050,32500.0,32500.0,4486.862250,Statewide,0,1000000,COU,1.0,S00-Ann,4486,1.09464,1-Weekday (Tu-Th),4910,2015-2019,PM,0.3615,Prd,Season,AnnWkPM


,STATIONGROUP,YEARGROUP,SEASONGROUP,DOWGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
1184,COT,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3998,Prd
4918,COT,2015-2019,S00-Ann,1-Weekday (Tu-Th),MD,0.1921,Prd
8652,COT,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.3708,Prd


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
13688,2878_005.4,4.0,46570.0,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.5565,AM,0.1734,7.6,2.7,2050,32500.0,34500,3329.14995,Statewide,0,1000000,COU,1.0,S00-Ann,3329,1.09464,1-Weekday (Tu-Th),3644,2015-2019,AM,0.4029,Prd
13689,2878_005.4,4.0,46570.0,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.4435,AM,0.1734,17.2,6.4,2050,32500.0,34500,2653.15005,Statewide,0,1000000,COU,1.0,S00-Ann,2653,1.09464,1-Weekday (Tu-Th),2904,2015-2019,AM,0.4029,Prd
17476,2878_005.4,4.0,46570.0,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.4284,MD,0.3229,12.5,4.4,2050,32500.0,34500,4772.39742,Statewide,0,1000000,COU,1.0,S00-Ann,4772,1.09464,1-Weekday (Tu-Th),5223,2015-2019,MD,0.1956,Prd
17477,2878_005.4,4.0,46570.0,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.5716,MD,0.3229,14.4,5.4,2050,32500.0,34500,6367.65258,Statewide,0,1000000,COU,1.0,S00-Ann,6367,1.09464,1-Weekday (Tu-Th),6969,2015-2019,MD,0.1956,Prd
21264,2878_005.4,4.0,46570.0,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.3766,PM,0.2125,15.0,5.2,2050,32500.0,34500,2760.94875,Statewide,0,1000000,COU,1.0,S00-Ann,2760,1.09464,1-Weekday (Tu-Th),3021,2015-2019,PM,0.3615,Prd
21265,2878_005.4,4.0,46570.0,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6234,PM,0.2125,9.1,3.5,2050,32500.0,34500,4570.30125,Statewide,0,1000000,COU,1.0,S00-Ann,4570,1.09464,1-Weekday (Tu-Th),5002,2015-2019,PM,0.3615,Prd


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
19224,2878_005.4,4.0,46570.0,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.6,DY,1.0,12.2,4.2,2050,32500.0,34500,20700.0,Statewide,0,1000000,COU,1.0,S00-Ann,20700,1.09464,1-Weekday (Tu-Th),22659,2015-2019,AM,0.0670,DY
19225,2878_005.4,4.0,46570.0,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.6,DY,1.0,12.2,4.2,2050,32500.0,34500,20700.0,Statewide,0,1000000,COU,1.0,S00-Ann,20700,1.09464,1-Weekday (Tu-Th),22659,2015-2019,MD,0.0643,DY
19226,2878_005.4,4.0,46570.0,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.6,DY,1.0,12.2,4.2,2050,32500.0,34500,20700.0,Statewide,0,1000000,COU,1.0,S00-Ann,20700,1.09464,1-Weekday (Tu-Th),22659,2015-2019,PM,0.0875,DY
19227,2878_005.4,4.0,46570.0,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6,DY,1.0,13.1,4.8,2050,32500.0,34500,20700.0,Statewide,0,1000000,COU,1.0,S00-Ann,20700,1.09464,1-Weekday (Tu-Th),22659,2015-2019,AM,0.0670,DY
19228,2878_005.4,4.0,46570.0,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6,DY,1.0,13.1,4.8,2050,32500.0,34500,20700.0,Statewide,0,1000000,COU,1.0,S00-Ann,20700,1.09464,1-Weekday (Tu-Th),22659,2015-2019,MD,0.0643,DY
19229,2878_005.4,4.0,46570.0,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6,DY,1.0,13.1,4.8,2050,32500.0,34500,20700.0,Statewide,0,1000000,COU,1.0,S00-Ann,20700,1.09464,1-Weekday (Tu-Th),22659,2015-2019,PM,0.0875,DY


In [101]:
#df_PrdForecasts_Season_DOW_TOD[df_PrdForecasts_Season_DOW_TOD['SEGID']=='0006_146.9'].head(20)

In [102]:
#dftemp = dfs_PrdForecasts_Season_DOW_TOD_wGroups[0].groupby(['VCGroupCode']).agg({'VolMaxHrPct':[np.size]})
df_TODFactors_DY[df_TODFactors_DY['SEASONGROUP']=='S00-Ann']

,STATIONGROUP,YEARGROUP,SEASONGROUP,DOWGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
1871,301,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.0749,DY
1879,302,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.0670,DY
1886,303,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.0614,DY
1895,304,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.0435,DY
1904,305,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.0575,DY
...,...,...,...,...,...,...,...
11161,SNC,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0836,DY
11170,XX1,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY
11179,XX2,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0804,DY
11188,XX3,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY


In [103]:
df_PrdDuration

,PERIOD,PRDHRS
0,AM,3
1,MD,6
2,PM,3
3,EV,12
4,DY,24


In [104]:
dfs_PkHrForecasts = []

for df_PrdForecasts_Season_DOW_TOD_wGroups in dfs_PrdForecasts_Season_DOW_TOD_wGroups: 
    
    df_PkHrForecasts = pd.DataFrame.merge(df_PrdForecasts_Season_DOW_TOD_wGroups,df_PrdDuration,on='PERIOD',how='left')   
    
    df_PkHrForecasts['AvgSsnDOWPkHrVol'] = df_PkHrForecasts['AvgSsnDOWPrdVol'] * df_PkHrForecasts['VOLPKHRPCT']

    #add truck Pctage adjustments
    #PCE for medium and heavy trucks from HCM 2000 Exhibit 23-8
    Emd = 1.2 #same as RVs
    Ehv = 1.5
    df_PkHrForecasts['TrkFac'] = 1 + (df_PkHrForecasts[nameMDPct]/100 * (Emd-1)) + (df_PkHrForecasts[nameHVPct]/100 * (Ehv-1))
    
    #calculate PCE
    df_PkHrForecasts['PrdPCEFlow' ] = (df_PkHrForecasts['AvgSsnDOWPrdVol' ] *  df_PkHrForecasts['TrkFac']) /  df_PkHrForecasts['PRDHRS']
    df_PkHrForecasts['PkHrPCEFlow'] =  df_PkHrForecasts['AvgSsnDOWPkHrVol'] *  df_PkHrForecasts['TrkFac']
    df_PkHrForecasts['PrdVC'      ] =  df_PkHrForecasts['PrdPCEFlow'      ] / (df_PkHrForecasts[nameCap1HL] * df_PkHrForecasts[nameLanes])
    df_PkHrForecasts['PkHrVC'     ] =  df_PkHrForecasts['PkHrPCEFlow'     ] / (df_PkHrForecasts[nameCap1HL] * df_PkHrForecasts[nameLanes])
    
    display(df_PkHrForecasts)
    
    dfs_PkHrForecasts.append(df_PkHrForecasts)

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_146.9,1.0,1417.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5332,AM,0.1842,2.9,4.7,2019,1517.0,1517.0,148.992822,Statewide,0,2500,COP,1.0,S00-Ann,148,0.984028,1-Weekday (Tu-Th),145,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,59.3485,1.0293,49.749500,61.087411,0.046713,0.057359
1,0006_146.9,1.0,1417.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4668,AM,0.1842,3.3,2.9,2019,1517.0,1517.0,130.438578,Statewide,0,2500,COP,1.0,S00-Ann,130,0.984028,1-Weekday (Tu-Th),127,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,51.9811,1.0211,43.226567,53.077901,0.040588,0.049838
2,0006_149.9,1.0,2028.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4195,AM,0.1798,11.1,11.1,2019,2441.0,2441.0,184.115110,Statewide,0,2500,COP,1.0,S00-Ann,184,0.984028,1-Weekday (Tu-Th),181,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,74.0833,1.0777,65.021233,79.839572,0.061053,0.074967
3,0006_149.9,1.0,2028.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.5805,AM,0.1798,9.9,7.6,2019,2441.0,2441.0,254.776690,Statewide,0,2500,COP,1.0,S00-Ann,254,0.984028,1-Weekday (Tu-Th),249,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,101.9157,1.0578,87.797400,107.806427,0.082439,0.101227
4,0006_150.6,1.0,2269.2,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4439,AM,0.1756,10.6,10.3,2019,2441.0,2441.0,190.273118,Statewide,0,2500,COP,1.0,S00-Ann,190,0.984028,1-Weekday (Tu-Th),186,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,76.1298,1.0727,66.507400,81.664436,0.062448,0.076680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22447,2825_000.0,3.0,795.3,Arterial,1.0,D2,1.0,5.0,Collector,580,0.3350,PM,0.2447,27.9,4.6,2019,895.0,895.0,73.367178,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,73,0.955525,1-Weekday (Tu-Th),69,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,26.0406,1.0788,24.812400,28.092599,0.042780,0.048436
22448,2865_001.3,2.0,1996.0,Arterial,1.0,D1,1.0,4.0,Collector,785,0.5553,PM,0.2489,14.5,4.3,2019,2385.0,2385.0,329.640795,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,329,0.955525,1-Weekday (Tu-Th),314,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,118.5036,1.0505,109.952333,124.488032,0.140067,0.158583
22449,2865_001.3,2.0,1996.0,Arterial,1.0,D2,1.0,4.0,Collector,785,0.4447,PM,0.2489,24.9,15.8,2019,2385.0,2385.0,263.985705,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,263,0.955525,1-Weekday (Tu-Th),251,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,94.7274,1.1288,94.442933,106.928289,0.120309,0.136214
22450,WFRC_8071,1.0,28.8,Arterial,1.0,D1,1.0,5.0,Collector,577,0.4016,PM,0.2779,0.0,0.0,2019,0.0,0.0,0.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,0,0.955525,1-Weekday (Tu-Th),0,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,0.0000,1.0000,0.000000,0.000000,0.000000,0.000000


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_146.9,1.0,1577.9,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5081,AM,0.1762,2.7,5.2,2024,1700.0,1700.0,152.196274,Statewide,0,2500,COP,1.0,S00-Ann,152,0.984028,1-Weekday (Tu-Th),149,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,60.9857,1.0314,51.2262,62.900651,0.048100,0.059062
1,0006_146.9,1.0,1577.9,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4919,AM,0.1762,2.3,2.6,2024,1700.0,1700.0,147.343726,Statewide,0,2500,COP,1.0,S00-Ann,147,0.984028,1-Weekday (Tu-Th),144,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,58.9392,1.0176,48.8448,59.976530,0.045864,0.056316
2,0038_003.2,1.0,4104.3,Arterial,1.0,D1,2.0,3.0,Minor Arterial,944,0.4973,AM,0.1801,14.6,12.3,2024,0.0,0.0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.984028,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0907,0.0000,0.000000,0.000000,0.000000
3,0038_003.2,1.0,4104.3,Arterial,1.0,D2,2.0,3.0,Minor Arterial,944,0.5027,AM,0.1801,12.2,10.7,2024,0.0,0.0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.984028,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0779,0.0000,0.000000,0.000000,0.000000
4,0068_000.0,1.0,2119.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1061,0.5721,AM,0.1771,5.4,5.3,2024,1600.0,1600.0,162.110256,Statewide,0,2500,COP,1.0,S00-Ann,162,0.984028,1-Weekday (Tu-Th),159,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,65.0787,1.0373,54.9769,67.506136,0.051816,0.063625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22807,2825_000.0,3.0,884.9,Arterial,1.0,D2,1.0,5.0,Collector,580,0.3264,PM,0.2446,24.6,8.6,2024,1000.0,1000.0,79.837440,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,79,0.955525,1-Weekday (Tu-Th),75,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,28.3050,1.0922,27.3050,30.914721,0.047078,0.053301
22808,2865_001.3,2.0,2111.0,Arterial,1.0,D1,1.0,4.0,Collector,785,0.5623,PM,0.2511,16.4,3.0,2024,2500.0,2500.0,352.983825,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,352,0.955525,1-Weekday (Tu-Th),336,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,126.8064,1.0478,117.3536,132.867746,0.149495,0.169258
22809,2865_001.3,2.0,2111.0,Arterial,1.0,D2,1.0,4.0,Collector,785,0.4377,PM,0.2511,25.0,15.9,2024,2500.0,2500.0,274.766175,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,274,0.955525,1-Weekday (Tu-Th),261,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,98.5014,1.1295,98.2665,111.257331,0.125180,0.141729
22810,WFRC_8071,1.0,28.1,Arterial,1.0,D1,1.0,5.0,Collector,577,0.5208,PM,0.2641,0.0,0.0,2024,0.0,0.0,0.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,0,0.955525,1-Weekday (Tu-Th),0,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,0.0000,1.0000,0.0000,0.000000,0.000000,0.000000


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_146.9,1.0,1639.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5100,AM,0.1830,2.5,4.8,2030,1700.0,1700.0,158.661000,Statewide,0,2500,COP,1.0,S00-Ann,158,0.984028,1-Weekday (Tu-Th),155,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,63.4415,1.0290,53.165000,65.281303,0.049920,0.061297
1,0006_146.9,1.0,1639.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4900,AM,0.1830,2.1,2.5,2030,1700.0,1700.0,152.439000,Statewide,0,2500,COP,1.0,S00-Ann,152,0.984028,1-Weekday (Tu-Th),149,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,60.9857,1.0167,50.496100,62.004161,0.047414,0.058220
2,0038_003.2,1.0,4267.2,Arterial,1.0,D1,2.0,3.0,Minor Arterial,944,0.4864,AM,0.1752,16.9,11.6,2030,0.0,0.0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.984028,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0918,0.000000,0.000000,0.000000,0.000000
3,0038_003.2,1.0,4267.2,Arterial,1.0,D2,2.0,3.0,Minor Arterial,944,0.5136,AM,0.1752,12.6,10.9,2030,0.0,0.0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.984028,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0797,0.000000,0.000000,0.000000,0.000000
4,0068_000.0,1.0,2249.2,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1061,0.6019,AM,0.1852,4.8,4.7,2030,1700.0,1700.0,189.502196,Statewide,0,2500,COP,1.0,S00-Ann,189,0.984028,1-Weekday (Tu-Th),185,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,75.7205,1.0331,63.707833,78.226849,0.060045,0.073729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23581,2825_000.0,3.0,1023.4,Arterial,1.0,D2,1.0,5.0,Collector,580,0.3617,PM,0.2431,19.2,6.7,2030,1100.0,1100.0,96.722197,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,96,0.955525,1-Weekday (Tu-Th),91,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,34.3434,1.0719,32.514300,36.812690,0.056059,0.063470
23582,2865_001.3,2.0,2185.0,Arterial,1.0,D1,1.0,4.0,Collector,785,0.5484,PM,0.2504,13.7,2.7,2030,2600.0,2600.0,357.030336,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,357,0.955525,1-Weekday (Tu-Th),341,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,128.6934,1.0409,118.315633,133.956960,0.150721,0.170646
23583,2865_001.3,2.0,2185.0,Arterial,1.0,D2,1.0,4.0,Collector,785,0.4516,PM,0.2504,25.1,15.8,2030,2600.0,2600.0,294.009664,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,294,0.955525,1-Weekday (Tu-Th),280,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,105.6720,1.1292,105.392000,119.324822,0.134257,0.152006
23584,WFRC_8071,1.0,29.9,Arterial,1.0,D1,1.0,5.0,Collector,577,0.5208,PM,0.2482,0.0,0.0,2030,10.0,10.0,1.292626,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,1,0.955525,1-Weekday (Tu-Th),0,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,0.0000,1.0000,0.000000,0.000000,0.000000,0.000000


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_146.9,1.0,1715.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5357,AM,0.1907,2.1,4.3,2040,1800.0,1800.0,183.884382,Statewide,0,2500,COP,1.0,S00-Ann,183,0.984028,1-Weekday (Tu-Th),180,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,73.6740,1.0257,61.542000,75.567422,0.057786,0.070955
1,0006_146.9,1.0,1715.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4643,AM,0.1907,1.5,3.0,2040,1800.0,1800.0,159.375618,Statewide,0,2500,COP,1.0,S00-Ann,159,0.984028,1-Weekday (Tu-Th),156,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,63.8508,1.0180,52.936000,65.000114,0.049705,0.061033
2,0038_003.2,1.0,4643.0,Arterial,1.0,D1,2.0,3.0,Minor Arterial,944,0.4949,AM,0.1758,14.3,12.4,2040,0.0,0.0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.984028,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0906,0.000000,0.000000,0.000000,0.000000
3,0038_003.2,1.0,4643.0,Arterial,1.0,D2,2.0,3.0,Minor Arterial,944,0.5051,AM,0.1758,12.4,10.9,2040,0.0,0.0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.984028,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0793,0.000000,0.000000,0.000000,0.000000
4,0068_000.0,1.0,2812.1,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1061,0.5725,AM,0.1767,3.6,4.4,2040,2300.0,2300.0,232.669725,Statewide,0,2500,COP,1.0,S00-Ann,232,0.984028,1-Weekday (Tu-Th),228,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,93.3204,1.0292,78.219200,96.045356,0.073722,0.090523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24247,2825_000.0,3.0,1382.6,Arterial,1.0,D2,1.0,5.0,Collector,580,0.3732,PM,0.2497,28.1,10.1,2040,1500.0,1500.0,139.782060,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,139,0.955525,1-Weekday (Tu-Th),132,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,49.8168,1.1067,48.694800,55.132253,0.083957,0.095056
24248,2865_001.3,2.0,2422.0,Arterial,1.0,D1,1.0,4.0,Collector,785,0.5284,PM,0.2548,16.3,1.8,2040,2800.0,2800.0,376.981696,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,376,0.955525,1-Weekday (Tu-Th),359,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,135.4866,1.0416,124.644800,141.122843,0.158783,0.179774
24249,2865_001.3,2.0,2422.0,Arterial,1.0,D2,1.0,4.0,Collector,785,0.4716,PM,0.2548,22.3,14.4,2040,2800.0,2800.0,336.458304,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,336,0.955525,1-Weekday (Tu-Th),321,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,121.1454,1.1166,119.476200,135.270954,0.152199,0.172320
24250,WFRC_8071,1.0,27.8,Arterial,1.0,D1,1.0,5.0,Collector,577,0.5336,PM,0.3479,0.0,0.0,2040,20.0,20.0,3.712789,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,3,0.955525,1-Weekday (Tu-Th),2,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,0.7548,1.0000,0.666667,0.754800,0.001155,0.001308


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_146.9,1.0,1849.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6151,AM,0.1823,2.4,4.0,2050,2000.0,2000.0,224.265460,Statewide,0,2500,COP,1.0,S00-Ann,224,0.984028,1-Weekday (Tu-Th),220,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,90.0460,1.0248,75.152000,92.279141,0.070565,0.086647
1,0006_146.9,1.0,1849.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3849,AM,0.1823,3.9,3.6,2050,2000.0,2000.0,140.334540,Statewide,0,2500,COP,1.0,S00-Ann,140,0.984028,1-Weekday (Tu-Th),137,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,56.0741,1.0258,46.844867,57.520812,0.043986,0.054010
2,0073_020.7,1.0,1145.5,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1049,0.4621,AM,0.1456,15.9,8.8,2050,1900.0,1900.0,127.835344,Statewide,0,2500,COP,1.0,S00-Ann,127,0.984028,1-Weekday (Tu-Th),124,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,50.7532,1.0758,44.466400,54.600293,0.042389,0.052050
3,0073_020.7,1.0,1145.5,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1049,0.5379,AM,0.1456,15.5,9.2,2050,1900.0,1900.0,148.804656,Statewide,0,2500,COP,1.0,S00-Ann,148,0.984028,1-Weekday (Tu-Th),145,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,59.3485,1.0770,52.055000,63.918335,0.049623,0.060933
4,0147_003.1,1.0,627.5,Arterial,1.0,D1,1.0,3.0,Minor Arterial,983,0.4983,AM,0.1394,18.3,4.6,2050,1000.0,1000.0,69.463020,Statewide,0,2500,COP,1.0,S00-Ann,69,0.984028,1-Weekday (Tu-Th),67,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,27.4231,1.0596,23.664400,29.057517,0.024074,0.029560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24895,2825_000.0,3.0,1805.5,Arterial,1.0,D2,1.0,5.0,Collector,580,0.3918,PM,0.2574,21.0,9.4,2050,2000.0,2000.0,201.698640,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,201,0.955525,1-Weekday (Tu-Th),192,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,72.4608,1.0890,69.696000,78.909811,0.120166,0.136051
24896,2865_001.3,2.0,2633.0,Arterial,1.0,D1,1.0,4.0,Collector,785,0.5015,PM,0.2491,16.7,3.3,2050,3100.0,3100.0,387.263315,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,387,0.955525,1-Weekday (Tu-Th),369,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,139.2606,1.0499,129.137700,146.209704,0.164507,0.186254
24897,2865_001.3,2.0,2633.0,Arterial,1.0,D2,1.0,4.0,Collector,785,0.4985,PM,0.2491,21.4,13.8,2050,3100.0,3100.0,384.946685,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,384,0.955525,1-Weekday (Tu-Th),366,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,138.1284,1.1118,135.639600,153.571155,0.172789,0.195632
24898,WFRC_8071,1.0,65.6,Arterial,1.0,D1,1.0,5.0,Collector,577,0.5746,PM,0.2401,7.2,0.0,2050,40.0,40.0,5.518458,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,5,0.955525,1-Weekday (Tu-Th),4,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,1.5096,1.0144,1.352533,1.531338,0.002344,0.002654


...

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_146.9,1.0,1640.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5113,AM,0.1835,2.5,4.8,2030,1700.0,1700,159.500035,Statewide,0,2500,COP,1.0,S00-Ann,159,0.984028,1-Weekday (Tu-Th),156,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,63.8508,1.0290,53.508000,65.702473,0.050242,0.061692
1,0006_146.9,1.0,1640.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4887,AM,0.1835,2.1,2.5,2030,1700.0,1700,152.449965,Statewide,0,2500,COP,1.0,S00-Ann,152,0.984028,1-Weekday (Tu-Th),149,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,60.9857,1.0167,50.496100,62.004161,0.047414,0.058220
2,0038_003.2,1.0,4350.5,Arterial,1.0,D1,2.0,3.0,Minor Arterial,944,0.4957,AM,0.1763,16.2,11.1,2030,0.0,80,6.991353,Statewide,0,2500,COP,1.0,S00-Ann,6,0.984028,1-Weekday (Tu-Th),5,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,2.0465,1.0879,1.813167,2.226387,0.000960,0.001179
3,0038_003.2,1.0,4350.5,Arterial,1.0,D2,2.0,3.0,Minor Arterial,944,0.5043,AM,0.1763,12.5,10.8,2030,0.0,80,7.112647,Statewide,0,2500,COP,1.0,S00-Ann,7,0.984028,1-Weekday (Tu-Th),6,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,2.4558,1.0790,2.158000,2.649808,0.001143,0.001404
4,0068_000.0,1.0,2224.5,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1061,0.6117,AM,0.1786,5.0,4.5,2030,1700.0,1700,185.724354,Statewide,0,2500,COP,1.0,S00-Ann,185,0.984028,1-Weekday (Tu-Th),182,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,74.4926,1.0325,62.638333,76.913609,0.059037,0.072492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23623,2825_000.0,3.0,1007.6,Arterial,1.0,D2,1.0,5.0,Collector,580,0.3481,PM,0.2417,20.4,7.1,2030,1100.0,1100,92.549347,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,92,0.955525,1-Weekday (Tu-Th),87,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,32.8338,1.0763,31.212700,35.339019,0.053815,0.060929
23624,2865_001.3,2.0,2189.0,Arterial,1.0,D1,1.0,4.0,Collector,785,0.5366,PM,0.2435,14.3,2.8,2030,2600.0,2600,339.721460,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,339,0.955525,1-Weekday (Tu-Th),323,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,121.9002,1.0426,112.253267,127.093149,0.142998,0.161902
23625,2865_001.3,2.0,2189.0,Arterial,1.0,D2,1.0,4.0,Collector,785,0.4634,PM,0.2435,25.1,15.8,2030,2600.0,2600,293.378540,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,293,0.955525,1-Weekday (Tu-Th),279,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,105.2946,1.1292,105.015600,118.898662,0.133778,0.151463
23626,WFRC_8071,1.0,29.8,Arterial,1.0,D1,1.0,5.0,Collector,577,0.4465,PM,0.2904,0.0,0.0,2030,10.0,10,1.296636,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,1,0.955525,1-Weekday (Tu-Th),0,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,0.0000,1.0000,0.000000,0.000000,0.000000,0.000000


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_146.9,1.0,1673.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5531,AM,0.1865,2.2,4.3,2040,1800.0,1800,185.675670,Statewide,0,2500,COP,1.0,S00-Ann,185,0.984028,1-Weekday (Tu-Th),182,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,74.4926,1.0259,62.237933,76.421958,0.058439,0.071758
1,0006_146.9,1.0,1673.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4469,AM,0.1865,1.6,3.2,2040,1800.0,1800,150.024330,Statewide,0,2500,COP,1.0,S00-Ann,150,0.984028,1-Weekday (Tu-Th),147,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,60.1671,1.0192,49.940800,61.322308,0.046893,0.057580
2,0038_003.2,1.0,4643.0,Arterial,1.0,D1,2.0,3.0,Minor Arterial,944,0.5028,AM,0.1789,14.1,12.1,2040,0.0,0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.984028,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0887,0.000000,0.000000,0.000000,0.000000
3,0038_003.2,1.0,4643.0,Arterial,1.0,D2,2.0,3.0,Minor Arterial,944,0.4972,AM,0.1789,12.4,10.9,2040,0.0,0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.984028,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0793,0.000000,0.000000,0.000000,0.000000
4,0068_000.0,1.0,2450.4,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1061,0.5834,AM,0.1845,4.4,5.3,2040,2300.0,1900,204.510870,Statewide,0,2500,COP,1.0,S00-Ann,204,0.984028,1-Weekday (Tu-Th),200,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,81.8600,1.0353,69.020000,84.749658,0.065052,0.079877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24325,2825_000.0,3.0,1415.6,Arterial,1.0,D2,1.0,5.0,Collector,580,0.4080,PM,0.2582,24.4,8.7,2040,1500.0,1500,158.018400,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,158,0.955525,1-Weekday (Tu-Th),150,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,56.6100,1.0923,54.615000,61.835103,0.094164,0.106612
24326,2865_001.3,2.0,2406.0,Arterial,1.0,D1,1.0,4.0,Collector,785,0.5093,PM,0.2456,17.6,2.0,2040,2800.0,2800,350.235424,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,350,0.955525,1-Weekday (Tu-Th),334,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,126.0516,1.0452,116.365600,131.749132,0.148236,0.167833
24327,2865_001.3,2.0,2406.0,Arterial,1.0,D2,1.0,4.0,Collector,785,0.4907,PM,0.2456,22.4,14.5,2040,2800.0,2800,337.444576,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,337,0.955525,1-Weekday (Tu-Th),322,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,121.5228,1.1173,119.923533,135.777424,0.152769,0.172965
24328,WFRC_8071,1.0,25.6,Arterial,1.0,D1,1.0,5.0,Collector,577,0.7662,PM,0.2629,0.0,0.0,2040,20.0,20,4.028680,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,4,0.955525,1-Weekday (Tu-Th),3,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,1.1322,1.0000,1.000000,1.132200,0.001733,0.001962


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_146.9,1.0,1829.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5996,AM,0.1903,2.4,4.0,2050,2000.0,2000,228.207760,Statewide,0,2500,COP,1.0,S00-Ann,228,0.984028,1-Weekday (Tu-Th),224,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,91.6832,1.0248,76.518400,93.956943,0.071848,0.088222
1,0006_146.9,1.0,1829.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4004,AM,0.1903,3.6,3.3,2050,2000.0,2000,152.392240,Statewide,0,2500,COP,1.0,S00-Ann,152,0.984028,1-Weekday (Tu-Th),149,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,60.9857,1.0237,50.843767,62.431061,0.047741,0.058621
2,0073_020.7,1.0,1129.3,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1049,0.5019,AM,0.1414,14.7,7.9,2050,1900.0,1900,134.840454,Statewide,0,2500,COP,1.0,S00-Ann,134,0.984028,1-Weekday (Tu-Th),131,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,53.6183,1.0689,46.675300,57.312601,0.044495,0.054635
3,0073_020.7,1.0,1129.3,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1049,0.4981,AM,0.1414,16.9,8.9,2050,1900.0,1900,133.819546,Statewide,0,2500,COP,1.0,S00-Ann,133,0.984028,1-Weekday (Tu-Th),130,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,53.2090,1.0783,46.726333,57.375265,0.044544,0.054695
4,0147_003.1,1.0,575.9,Arterial,1.0,D1,1.0,3.0,Minor Arterial,983,0.4783,AM,0.1597,18.2,6.8,2050,1000.0,950,72.565285,Statewide,0,2500,COP,1.0,S00-Ann,72,0.984028,1-Weekday (Tu-Th),70,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,28.6510,1.0704,24.976000,30.668030,0.025408,0.031198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24907,2825_000.0,3.0,1827.9,Arterial,1.0,D2,1.0,5.0,Collector,580,0.3947,PM,0.2553,20.7,9.2,2050,2000.0,2000,201.533820,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,201,0.955525,1-Weekday (Tu-Th),192,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,72.4608,1.0874,69.593600,78.793874,0.119989,0.135852
24908,2865_001.3,2.0,2673.0,Arterial,1.0,D1,1.0,4.0,Collector,785,0.5000,PM,0.2446,16.8,3.4,2050,3100.0,3100,379.130000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,379,0.955525,1-Weekday (Tu-Th),362,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,136.6188,1.0506,126.772400,143.531711,0.161494,0.182843
24909,2865_001.3,2.0,2673.0,Arterial,1.0,D2,1.0,4.0,Collector,785,0.5000,PM,0.2446,21.4,13.8,2050,3100.0,3100,379.130000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,379,0.955525,1-Weekday (Tu-Th),362,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,136.6188,1.1118,134.157200,151.892782,0.170901,0.193494
24910,WFRC_8071,1.0,58.7,Arterial,1.0,D1,1.0,5.0,Collector,577,0.5530,PM,0.2785,7.2,0.0,2050,40.0,30,4.620315,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,4,0.955525,1-Weekday (Tu-Th),3,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,1.1322,1.0144,1.014400,1.148504,0.001758,0.001990


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_146.9,1.0,1856.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5887,AM,0.1945,2.4,3.9,2050,2000.0,2000,229.004300,Statewide,0,2500,COP,1.0,S00-Ann,229,0.984028,1-Weekday (Tu-Th),225,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,92.0925,1.0243,76.822500,94.330348,0.072134,0.088573
1,0006_146.9,1.0,1856.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4113,AM,0.1945,3.4,3.1,2050,2000.0,2000,159.995700,Statewide,0,2500,COP,1.0,S00-Ann,159,0.984028,1-Weekday (Tu-Th),156,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,63.8508,1.0223,53.159600,65.274673,0.049915,0.061291
2,0073_020.7,1.0,1161.3,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1049,0.5013,AM,0.1492,15.8,7.5,2050,1900.0,1900,142.108524,Statewide,0,2500,COP,1.0,S00-Ann,142,0.984028,1-Weekday (Tu-Th),139,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,56.8927,1.0691,49.534967,60.823986,0.047221,0.057983
3,0073_020.7,1.0,1161.3,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1049,0.4987,AM,0.1492,19.6,8.0,2050,1900.0,1900,141.371476,Statewide,0,2500,COP,1.0,S00-Ann,141,0.984028,1-Weekday (Tu-Th),138,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,56.4834,1.0792,49.643200,60.956885,0.047324,0.058110
4,0147_003.1,1.0,591.9,Arterial,1.0,D1,1.0,3.0,Minor Arterial,983,0.4353,AM,0.1436,13.5,2.7,2050,1000.0,950,59.383626,Statewide,0,2500,COP,1.0,S00-Ann,59,0.984028,1-Weekday (Tu-Th),58,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,23.7394,1.0405,20.116333,24.700846,0.020464,0.025128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24895,2825_000.0,3.0,1812.6,Arterial,1.0,D2,1.0,5.0,Collector,580,0.3861,PM,0.2541,21.6,9.6,2050,2000.0,2000,196.216020,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,196,0.955525,1-Weekday (Tu-Th),187,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,70.5738,1.0912,68.018133,77.010131,0.117273,0.132776
24896,2865_001.3,2.0,2646.0,Arterial,1.0,D1,1.0,4.0,Collector,785,0.5000,PM,0.2471,16.8,3.4,2050,3100.0,3100,383.005000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,383,0.955525,1-Weekday (Tu-Th),365,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,137.7510,1.0506,127.823000,144.721201,0.162832,0.184358
24897,2865_001.3,2.0,2646.0,Arterial,1.0,D2,1.0,4.0,Collector,785,0.5000,PM,0.2471,21.4,13.8,2050,3100.0,3100,383.005000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,383,0.955525,1-Weekday (Tu-Th),365,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,137.7510,1.1118,135.269000,153.151562,0.172317,0.195098
24898,WFRC_8071,1.0,64.6,Arterial,1.0,D1,1.0,5.0,Collector,577,0.5334,PM,0.2538,0.0,7.4,2050,40.0,40,5.415077,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,5,0.955525,1-Weekday (Tu-Th),4,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,1.5096,1.0370,1.382667,1.565455,0.002396,0.002713


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6020,AM,0.1912,2.4,4.0,2050,2000.0,2000,230.204800,Statewide,0,2500,COP,1.0,S00-Ann,230,0.984028,1-Weekday (Tu-Th),226,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,92.5018,1.0248,77.201600,94.795845,0.072490,0.089010
1,0006_146.9,1.0,1836.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3980,AM,0.1912,3.6,3.3,2050,2000.0,2000,152.195200,Statewide,0,2500,COP,1.0,S00-Ann,152,0.984028,1-Weekday (Tu-Th),149,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,60.9857,1.0237,50.843767,62.431061,0.047741,0.058621
2,0073_020.7,1.0,1174.3,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1049,0.4889,AM,0.1478,16.2,7.7,2050,1900.0,1900,137.292898,Statewide,0,2500,COP,1.0,S00-Ann,137,0.984028,1-Weekday (Tu-Th),134,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,54.8462,1.0709,47.833533,58.734796,0.045599,0.055991
3,0073_020.7,1.0,1174.3,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1049,0.5111,AM,0.1478,18.8,7.5,2050,1900.0,1900,143.527102,Statewide,0,2500,COP,1.0,S00-Ann,143,0.984028,1-Weekday (Tu-Th),140,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,57.3020,1.0751,50.171333,61.605380,0.047828,0.058728
4,0147_003.1,1.0,593.8,Arterial,1.0,D1,1.0,3.0,Minor Arterial,983,0.4767,AM,0.1480,19.1,4.8,2050,1000.0,950,67.024020,Statewide,0,2500,COP,1.0,S00-Ann,67,0.984028,1-Weekday (Tu-Th),65,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,26.6045,1.0622,23.014333,28.259300,0.023412,0.028748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24895,2825_000.0,3.0,1819.0,Arterial,1.0,D2,1.0,5.0,Collector,580,0.3757,PM,0.2490,22.6,10.0,2050,2000.0,2000,187.098600,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,187,0.955525,1-Weekday (Tu-Th),178,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,67.1772,1.0952,64.981867,73.572469,0.112038,0.126849
24896,2865_001.3,2.0,2656.0,Arterial,1.0,D1,1.0,4.0,Collector,785,0.5030,PM,0.2478,16.6,3.3,2050,3100.0,3100,386.394540,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,386,0.955525,1-Weekday (Tu-Th),368,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,138.8832,1.0497,128.763200,145.785695,0.164030,0.185714
24897,2865_001.3,2.0,2656.0,Arterial,1.0,D2,1.0,4.0,Collector,785,0.4970,PM,0.2478,21.4,13.8,2050,3100.0,3100,381.785460,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,381,0.955525,1-Weekday (Tu-Th),364,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,137.3736,1.1118,134.898400,152.731968,0.171845,0.194563
24898,WFRC_8071,1.0,60.7,Arterial,1.0,D1,1.0,5.0,Collector,577,0.5840,PM,0.2274,0.0,8.0,2050,40.0,40,5.312064,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,5,0.955525,1-Weekday (Tu-Th),4,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,1.5096,1.0400,1.386667,1.569984,0.002403,0.002721


In [105]:
df_PkHrForecasts.columns

Index(['SEGID', 'AREATYPE', 'SegDYVol_TDM', 'FUNCGROUP', 'FGFAC', 'Dir',
       'LANES', 'FT', 'FTCLASS', 'CAP1HL', 'DFAC', 'Prd_TDM', 'PRDFAC',
       'MDPCT', 'HVPCT', 'FYEAR', 'ForecastAADT', 'ForecastAADTAdj',
       'AvgAnnualPrdVol', 'FACGEO', 'VOLUMEFROM', 'VOLUMETO', 'STATIONGROUP',
       'SsnFactor', 'SEASONGROUP', 'AvgSsnPrdVol', 'DOWFactor', 'DOWGROUP',
       'AvgSsnDOWPrdVol', 'YEARGROUP', 'PERIOD', 'VOLPKHRPCT', 'HRPCTOF',
       'SeasonType', 'VCGroupCode', 'PRDHRS', 'AvgSsnDOWPkHrVol', 'TrkFac',
       'PrdPCEFlow', 'PkHrPCEFlow', 'PrdVC', 'PkHrVC'],
      dtype='object')

In [106]:
df_PkHrForecasts[df_PkHrForecasts['SEASONGROUP']=='S00-Ann']

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_146.9,1.0,1836.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6020,AM,0.1912,2.4,4.0,2050,2000.0,2000,230.204800,Statewide,0,2500,COP,1.0,S00-Ann,230,0.984028,1-Weekday (Tu-Th),226,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,92.5018,1.0248,77.201600,94.795845,0.072490,0.089010
1,0006_146.9,1.0,1836.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3980,AM,0.1912,3.6,3.3,2050,2000.0,2000,152.195200,Statewide,0,2500,COP,1.0,S00-Ann,152,0.984028,1-Weekday (Tu-Th),149,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,60.9857,1.0237,50.843767,62.431061,0.047741,0.058621
2,0073_020.7,1.0,1174.3,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1049,0.4889,AM,0.1478,16.2,7.7,2050,1900.0,1900,137.292898,Statewide,0,2500,COP,1.0,S00-Ann,137,0.984028,1-Weekday (Tu-Th),134,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,54.8462,1.0709,47.833533,58.734796,0.045599,0.055991
3,0073_020.7,1.0,1174.3,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1049,0.5111,AM,0.1478,18.8,7.5,2050,1900.0,1900,143.527102,Statewide,0,2500,COP,1.0,S00-Ann,143,0.984028,1-Weekday (Tu-Th),140,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,57.3020,1.0751,50.171333,61.605380,0.047828,0.058728
4,0147_003.1,1.0,593.8,Arterial,1.0,D1,1.0,3.0,Minor Arterial,983,0.4767,AM,0.1480,19.1,4.8,2050,1000.0,950,67.024020,Statewide,0,2500,COP,1.0,S00-Ann,67,0.984028,1-Weekday (Tu-Th),65,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,26.6045,1.0622,23.014333,28.259300,0.023412,0.028748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24895,2825_000.0,3.0,1819.0,Arterial,1.0,D2,1.0,5.0,Collector,580,0.3757,PM,0.2490,22.6,10.0,2050,2000.0,2000,187.098600,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,187,0.955525,1-Weekday (Tu-Th),178,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,67.1772,1.0952,64.981867,73.572469,0.112038,0.126849
24896,2865_001.3,2.0,2656.0,Arterial,1.0,D1,1.0,4.0,Collector,785,0.5030,PM,0.2478,16.6,3.3,2050,3100.0,3100,386.394540,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,386,0.955525,1-Weekday (Tu-Th),368,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,138.8832,1.0497,128.763200,145.785695,0.164030,0.185714
24897,2865_001.3,2.0,2656.0,Arterial,1.0,D2,1.0,4.0,Collector,785,0.4970,PM,0.2478,21.4,13.8,2050,3100.0,3100,381.785460,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,381,0.955525,1-Weekday (Tu-Th),364,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,137.3736,1.1118,134.898400,152.731968,0.171845,0.194563
24898,WFRC_8071,1.0,60.7,Arterial,1.0,D1,1.0,5.0,Collector,577,0.5840,PM,0.2274,0.0,8.0,2050,40.0,40,5.312064,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,5,0.955525,1-Weekday (Tu-Th),4,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,1.5096,1.0400,1.386667,1.569984,0.002403,0.002721


In [107]:
#15-Min Analysis

dfs_VC = []

for df_PkHrForecasts in dfs_PkHrForecasts:

    df_VC = df_PkHrForecasts
    
    #PHF conditions (from Chad's spreadsheet for Urban (0.92,0.95,0.97,0.98). HCM2000 Exhibit 9.2 says default PHF for Urban Areas is 0.92, Rural is 0.88. Rural estimated at (0.88,0.91,0.93,0.95)
    conditions = [
        (df_VC[nameAT] >= 3) & (df_VC['PkHrVC'] <= 0.85),
        (df_VC[nameAT] >= 3) & (df_VC['PkHrVC'] >  0.85) & (df_VC['PkHrVC'] <= 0.95),
        (df_VC[nameAT] >= 3) & (df_VC['PkHrVC'] >  0.95) & (df_VC['PkHrVC'] <= 1.00),
        (df_VC[nameAT] >= 3) & (df_VC['PkHrVC'] >  1.00),
        (df_VC[nameAT] <= 2) & (df_VC['PkHrVC'] <= 0.85),
        (df_VC[nameAT] <= 2) & (df_VC['PkHrVC'] >  0.85) & (df_VC['PkHrVC'] <= 0.95),
        (df_VC[nameAT] <= 2) & (df_VC['PkHrVC'] >  0.95) & (df_VC['PkHrVC'] <= 1.00),
        (df_VC[nameAT] <= 2) & (df_VC['PkHrVC'] >  1.00)
    ]
    choices = [0.92,0.95,0.97,0.98,0.88,0.91,0.93,0.95]

    df_VC['PHF'] = np.select(conditions, choices, default=0)


    #15-min volume and v/c calcs
    df_VC['Pk15PCEFlow'] = df_VC['PkHrPCEFlow']  / df_VC['PHF']
    df_VC['Pk15VC'     ] = df_VC['Pk15PCEFlow'] / (df_VC[nameCap1HL] * df_VC[nameLanes])

    #effective K-factor
    #df_VC['kFactor_effective']  = df_VC['ForecastPkHr']  / df_VC['ForecastAADT']

    #convert to int and round columns after calcs
    df_VC = df_VC.fillna(0)
    df_VC['AREATYPE'       ] = df_VC['AREATYPE'       ].astype(int)
    df_VC['FT'             ] = df_VC['FT'             ].astype(int)
    df_VC['LANES'          ] = df_VC['LANES'          ].astype(int)
    df_VC['ForecastAADT'   ] = df_VC['ForecastAADT'   ].astype(int)
    df_VC['ForecastAADTAdj'] = df_VC['ForecastAADTAdj'].astype(int)
    df_VC['PrdPCEFlow'     ] = df_VC['PrdPCEFlow'     ].astype(int)
    df_VC['PkHrPCEFlow'    ] = df_VC['PkHrPCEFlow'    ].astype(int)
    df_VC['Pk15PCEFlow'    ] = df_VC['Pk15PCEFlow'    ].astype(int)
    df_VC = df_VC.round({'SeasonFactor':2,'DOWFactor':2,'PrdVC':2,'PkHrVC':2,'Pk15VC':2,nameMDPct:2,nameHVPct:2,'VolMaxHrPct':2,'FactorTrucks':2})
    
    pd.set_option('display.max_columns', None)
    
    display(df_VC)
    
    dfs_VC.append(df_VC)
    

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_146.9,1,1417.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.5332,AM,0.1842,2.9,4.7,2019,1517,1517,148.992822,Statewide,0,2500,COP,1.0,S00-Ann,148,0.98,1-Weekday (Tu-Th),145,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,59.3485,1.0293,49,61,0.05,0.06,0.88,69,0.07
1,0006_146.9,1,1417.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.4668,AM,0.1842,3.3,2.9,2019,1517,1517,130.438578,Statewide,0,2500,COP,1.0,S00-Ann,130,0.98,1-Weekday (Tu-Th),127,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,51.9811,1.0211,43,53,0.04,0.05,0.88,60,0.06
2,0006_149.9,1,2028.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.4195,AM,0.1798,11.1,11.1,2019,2441,2441,184.115110,Statewide,0,2500,COP,1.0,S00-Ann,184,0.98,1-Weekday (Tu-Th),181,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,74.0833,1.0777,65,79,0.06,0.07,0.88,90,0.09
3,0006_149.9,1,2028.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.5805,AM,0.1798,9.9,7.6,2019,2441,2441,254.776690,Statewide,0,2500,COP,1.0,S00-Ann,254,0.98,1-Weekday (Tu-Th),249,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,101.9157,1.0578,87,107,0.08,0.10,0.88,122,0.12
4,0006_150.6,1,2269.2,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.4439,AM,0.1756,10.6,10.3,2019,2441,2441,190.273118,Statewide,0,2500,COP,1.0,S00-Ann,190,0.98,1-Weekday (Tu-Th),186,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,76.1298,1.0727,66,81,0.06,0.08,0.88,92,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22447,2825_000.0,3,795.3,Arterial,1.0,D2,1,5,Collector,580,0.3350,PM,0.2447,27.9,4.6,2019,895,895,73.367178,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,73,0.96,1-Weekday (Tu-Th),69,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,26.0406,1.0788,24,28,0.04,0.05,0.92,30,0.05
22448,2865_001.3,2,1996.0,Arterial,1.0,D1,1,4,Collector,785,0.5553,PM,0.2489,14.5,4.3,2019,2385,2385,329.640795,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,329,0.96,1-Weekday (Tu-Th),314,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,118.5036,1.0505,109,124,0.14,0.16,0.88,141,0.18
22449,2865_001.3,2,1996.0,Arterial,1.0,D2,1,4,Collector,785,0.4447,PM,0.2489,24.9,15.8,2019,2385,2385,263.985705,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,263,0.96,1-Weekday (Tu-Th),251,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,94.7274,1.1288,94,106,0.12,0.14,0.88,121,0.15
22450,WFRC_8071,1,28.8,Arterial,1.0,D1,1,5,Collector,577,0.4016,PM,0.2779,0.0,0.0,2019,0,0,0.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,0,0.96,1-Weekday (Tu-Th),0,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,0.0000,1.0000,0,0,0.00,0.00,0.88,0,0.00


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_146.9,1,1577.9,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.5081,AM,0.1762,2.7,5.2,2024,1700,1700,152.196274,Statewide,0,2500,COP,1.0,S00-Ann,152,0.98,1-Weekday (Tu-Th),149,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,60.9857,1.0314,51,62,0.05,0.06,0.88,71,0.07
1,0006_146.9,1,1577.9,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.4919,AM,0.1762,2.3,2.6,2024,1700,1700,147.343726,Statewide,0,2500,COP,1.0,S00-Ann,147,0.98,1-Weekday (Tu-Th),144,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,58.9392,1.0176,48,59,0.05,0.06,0.88,68,0.06
2,0038_003.2,1,4104.3,Arterial,1.0,D1,2,3,Minor Arterial,944,0.4973,AM,0.1801,14.6,12.3,2024,0,0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.98,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0907,0,0,0.00,0.00,0.88,0,0.00
3,0038_003.2,1,4104.3,Arterial,1.0,D2,2,3,Minor Arterial,944,0.5027,AM,0.1801,12.2,10.7,2024,0,0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.98,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0779,0,0,0.00,0.00,0.88,0,0.00
4,0068_000.0,1,2119.0,Arterial,1.0,D1,1,2,Principal Arterial,1061,0.5721,AM,0.1771,5.4,5.3,2024,1600,1600,162.110256,Statewide,0,2500,COP,1.0,S00-Ann,162,0.98,1-Weekday (Tu-Th),159,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,65.0787,1.0373,54,67,0.05,0.06,0.88,76,0.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22807,2825_000.0,3,884.9,Arterial,1.0,D2,1,5,Collector,580,0.3264,PM,0.2446,24.6,8.6,2024,1000,1000,79.837440,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,79,0.96,1-Weekday (Tu-Th),75,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,28.3050,1.0922,27,30,0.05,0.05,0.92,33,0.06
22808,2865_001.3,2,2111.0,Arterial,1.0,D1,1,4,Collector,785,0.5623,PM,0.2511,16.4,3.0,2024,2500,2500,352.983825,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,352,0.96,1-Weekday (Tu-Th),336,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,126.8064,1.0478,117,132,0.15,0.17,0.88,150,0.19
22809,2865_001.3,2,2111.0,Arterial,1.0,D2,1,4,Collector,785,0.4377,PM,0.2511,25.0,15.9,2024,2500,2500,274.766175,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,274,0.96,1-Weekday (Tu-Th),261,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,98.5014,1.1295,98,111,0.13,0.14,0.88,126,0.16
22810,WFRC_8071,1,28.1,Arterial,1.0,D1,1,5,Collector,577,0.5208,PM,0.2641,0.0,0.0,2024,0,0,0.000000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,0,0.96,1-Weekday (Tu-Th),0,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,0.0000,1.0000,0,0,0.00,0.00,0.88,0,0.00


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_146.9,1,1639.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.5100,AM,0.1830,2.5,4.8,2030,1700,1700,158.661000,Statewide,0,2500,COP,1.0,S00-Ann,158,0.98,1-Weekday (Tu-Th),155,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,63.4415,1.0290,53,65,0.05,0.06,0.88,74,0.07
1,0006_146.9,1,1639.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.4900,AM,0.1830,2.1,2.5,2030,1700,1700,152.439000,Statewide,0,2500,COP,1.0,S00-Ann,152,0.98,1-Weekday (Tu-Th),149,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,60.9857,1.0167,50,62,0.05,0.06,0.88,70,0.07
2,0038_003.2,1,4267.2,Arterial,1.0,D1,2,3,Minor Arterial,944,0.4864,AM,0.1752,16.9,11.6,2030,0,0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.98,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0918,0,0,0.00,0.00,0.88,0,0.00
3,0038_003.2,1,4267.2,Arterial,1.0,D2,2,3,Minor Arterial,944,0.5136,AM,0.1752,12.6,10.9,2030,0,0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.98,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0797,0,0,0.00,0.00,0.88,0,0.00
4,0068_000.0,1,2249.2,Arterial,1.0,D1,1,2,Principal Arterial,1061,0.6019,AM,0.1852,4.8,4.7,2030,1700,1700,189.502196,Statewide,0,2500,COP,1.0,S00-Ann,189,0.98,1-Weekday (Tu-Th),185,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,75.7205,1.0331,63,78,0.06,0.07,0.88,88,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23581,2825_000.0,3,1023.4,Arterial,1.0,D2,1,5,Collector,580,0.3617,PM,0.2431,19.2,6.7,2030,1100,1100,96.722197,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,96,0.96,1-Weekday (Tu-Th),91,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,34.3434,1.0719,32,36,0.06,0.06,0.92,40,0.07
23582,2865_001.3,2,2185.0,Arterial,1.0,D1,1,4,Collector,785,0.5484,PM,0.2504,13.7,2.7,2030,2600,2600,357.030336,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,357,0.96,1-Weekday (Tu-Th),341,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,128.6934,1.0409,118,133,0.15,0.17,0.88,152,0.19
23583,2865_001.3,2,2185.0,Arterial,1.0,D2,1,4,Collector,785,0.4516,PM,0.2504,25.1,15.8,2030,2600,2600,294.009664,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,294,0.96,1-Weekday (Tu-Th),280,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,105.6720,1.1292,105,119,0.13,0.15,0.88,135,0.17
23584,WFRC_8071,1,29.9,Arterial,1.0,D1,1,5,Collector,577,0.5208,PM,0.2482,0.0,0.0,2030,10,10,1.292626,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,1,0.96,1-Weekday (Tu-Th),0,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,0.0000,1.0000,0,0,0.00,0.00,0.88,0,0.00


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_146.9,1,1715.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.5357,AM,0.1907,2.1,4.3,2040,1800,1800,183.884382,Statewide,0,2500,COP,1.0,S00-Ann,183,0.98,1-Weekday (Tu-Th),180,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,73.6740,1.0257,61,75,0.06,0.07,0.88,85,0.08
1,0006_146.9,1,1715.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.4643,AM,0.1907,1.5,3.0,2040,1800,1800,159.375618,Statewide,0,2500,COP,1.0,S00-Ann,159,0.98,1-Weekday (Tu-Th),156,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,63.8508,1.0180,52,65,0.05,0.06,0.88,73,0.07
2,0038_003.2,1,4643.0,Arterial,1.0,D1,2,3,Minor Arterial,944,0.4949,AM,0.1758,14.3,12.4,2040,0,0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.98,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0906,0,0,0.00,0.00,0.88,0,0.00
3,0038_003.2,1,4643.0,Arterial,1.0,D2,2,3,Minor Arterial,944,0.5051,AM,0.1758,12.4,10.9,2040,0,0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.98,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0793,0,0,0.00,0.00,0.88,0,0.00
4,0068_000.0,1,2812.1,Arterial,1.0,D1,1,2,Principal Arterial,1061,0.5725,AM,0.1767,3.6,4.4,2040,2300,2300,232.669725,Statewide,0,2500,COP,1.0,S00-Ann,232,0.98,1-Weekday (Tu-Th),228,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,93.3204,1.0292,78,96,0.07,0.09,0.88,109,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24247,2825_000.0,3,1382.6,Arterial,1.0,D2,1,5,Collector,580,0.3732,PM,0.2497,28.1,10.1,2040,1500,1500,139.782060,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,139,0.96,1-Weekday (Tu-Th),132,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,49.8168,1.1067,48,55,0.08,0.10,0.92,59,0.10
24248,2865_001.3,2,2422.0,Arterial,1.0,D1,1,4,Collector,785,0.5284,PM,0.2548,16.3,1.8,2040,2800,2800,376.981696,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,376,0.96,1-Weekday (Tu-Th),359,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,135.4866,1.0416,124,141,0.16,0.18,0.88,160,0.20
24249,2865_001.3,2,2422.0,Arterial,1.0,D2,1,4,Collector,785,0.4716,PM,0.2548,22.3,14.4,2040,2800,2800,336.458304,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,336,0.96,1-Weekday (Tu-Th),321,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,121.1454,1.1166,119,135,0.15,0.17,0.88,153,0.20
24250,WFRC_8071,1,27.8,Arterial,1.0,D1,1,5,Collector,577,0.5336,PM,0.3479,0.0,0.0,2040,20,20,3.712789,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,3,0.96,1-Weekday (Tu-Th),2,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,0.7548,1.0000,0,0,0.00,0.00,0.88,0,0.00


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_146.9,1,1849.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6151,AM,0.1823,2.4,4.0,2050,2000,2000,224.265460,Statewide,0,2500,COP,1.0,S00-Ann,224,0.98,1-Weekday (Tu-Th),220,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,90.0460,1.0248,75,92,0.07,0.09,0.88,104,0.10
1,0006_146.9,1,1849.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3849,AM,0.1823,3.9,3.6,2050,2000,2000,140.334540,Statewide,0,2500,COP,1.0,S00-Ann,140,0.98,1-Weekday (Tu-Th),137,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,56.0741,1.0258,46,57,0.04,0.05,0.88,65,0.06
2,0073_020.7,1,1145.5,Arterial,1.0,D1,1,2,Principal Arterial,1049,0.4621,AM,0.1456,15.9,8.8,2050,1900,1900,127.835344,Statewide,0,2500,COP,1.0,S00-Ann,127,0.98,1-Weekday (Tu-Th),124,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,50.7532,1.0758,44,54,0.04,0.05,0.88,62,0.06
3,0073_020.7,1,1145.5,Arterial,1.0,D2,1,2,Principal Arterial,1049,0.5379,AM,0.1456,15.5,9.2,2050,1900,1900,148.804656,Statewide,0,2500,COP,1.0,S00-Ann,148,0.98,1-Weekday (Tu-Th),145,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,59.3485,1.0770,52,63,0.05,0.06,0.88,72,0.07
4,0147_003.1,1,627.5,Arterial,1.0,D1,1,3,Minor Arterial,983,0.4983,AM,0.1394,18.3,4.6,2050,1000,1000,69.463020,Statewide,0,2500,COP,1.0,S00-Ann,69,0.98,1-Weekday (Tu-Th),67,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,27.4231,1.0596,23,29,0.02,0.03,0.88,33,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24895,2825_000.0,3,1805.5,Arterial,1.0,D2,1,5,Collector,580,0.3918,PM,0.2574,21.0,9.4,2050,2000,2000,201.698640,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,201,0.96,1-Weekday (Tu-Th),192,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,72.4608,1.0890,69,78,0.12,0.14,0.92,85,0.15
24896,2865_001.3,2,2633.0,Arterial,1.0,D1,1,4,Collector,785,0.5015,PM,0.2491,16.7,3.3,2050,3100,3100,387.263315,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,387,0.96,1-Weekday (Tu-Th),369,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,139.2606,1.0499,129,146,0.16,0.19,0.88,166,0.21
24897,2865_001.3,2,2633.0,Arterial,1.0,D2,1,4,Collector,785,0.4985,PM,0.2491,21.4,13.8,2050,3100,3100,384.946685,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,384,0.96,1-Weekday (Tu-Th),366,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,138.1284,1.1118,135,153,0.17,0.20,0.88,174,0.22
24898,WFRC_8071,1,65.6,Arterial,1.0,D1,1,5,Collector,577,0.5746,PM,0.2401,7.2,0.0,2050,40,40,5.518458,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,5,0.96,1-Weekday (Tu-Th),4,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,1.5096,1.0144,1,1,0.00,0.00,0.88,1,0.00


...

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_146.9,1,1640.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.5113,AM,0.1835,2.5,4.8,2030,1700,1700,159.500035,Statewide,0,2500,COP,1.0,S00-Ann,159,0.98,1-Weekday (Tu-Th),156,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,63.8508,1.0290,53,65,0.05,0.06,0.88,74,0.07
1,0006_146.9,1,1640.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.4887,AM,0.1835,2.1,2.5,2030,1700,1700,152.449965,Statewide,0,2500,COP,1.0,S00-Ann,152,0.98,1-Weekday (Tu-Th),149,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,60.9857,1.0167,50,62,0.05,0.06,0.88,70,0.07
2,0038_003.2,1,4350.5,Arterial,1.0,D1,2,3,Minor Arterial,944,0.4957,AM,0.1763,16.2,11.1,2030,0,80,6.991353,Statewide,0,2500,COP,1.0,S00-Ann,6,0.98,1-Weekday (Tu-Th),5,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,2.0465,1.0879,1,2,0.00,0.00,0.88,2,0.00
3,0038_003.2,1,4350.5,Arterial,1.0,D2,2,3,Minor Arterial,944,0.5043,AM,0.1763,12.5,10.8,2030,0,80,7.112647,Statewide,0,2500,COP,1.0,S00-Ann,7,0.98,1-Weekday (Tu-Th),6,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,2.4558,1.0790,2,2,0.00,0.00,0.88,3,0.00
4,0068_000.0,1,2224.5,Arterial,1.0,D1,1,2,Principal Arterial,1061,0.6117,AM,0.1786,5.0,4.5,2030,1700,1700,185.724354,Statewide,0,2500,COP,1.0,S00-Ann,185,0.98,1-Weekday (Tu-Th),182,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,74.4926,1.0325,62,76,0.06,0.07,0.88,87,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23623,2825_000.0,3,1007.6,Arterial,1.0,D2,1,5,Collector,580,0.3481,PM,0.2417,20.4,7.1,2030,1100,1100,92.549347,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,92,0.96,1-Weekday (Tu-Th),87,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,32.8338,1.0763,31,35,0.05,0.06,0.92,38,0.07
23624,2865_001.3,2,2189.0,Arterial,1.0,D1,1,4,Collector,785,0.5366,PM,0.2435,14.3,2.8,2030,2600,2600,339.721460,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,339,0.96,1-Weekday (Tu-Th),323,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,121.9002,1.0426,112,127,0.14,0.16,0.88,144,0.18
23625,2865_001.3,2,2189.0,Arterial,1.0,D2,1,4,Collector,785,0.4634,PM,0.2435,25.1,15.8,2030,2600,2600,293.378540,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,293,0.96,1-Weekday (Tu-Th),279,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,105.2946,1.1292,105,118,0.13,0.15,0.88,135,0.17
23626,WFRC_8071,1,29.8,Arterial,1.0,D1,1,5,Collector,577,0.4465,PM,0.2904,0.0,0.0,2030,10,10,1.296636,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,1,0.96,1-Weekday (Tu-Th),0,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,0.0000,1.0000,0,0,0.00,0.00,0.88,0,0.00


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_146.9,1,1673.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.5531,AM,0.1865,2.2,4.3,2040,1800,1800,185.675670,Statewide,0,2500,COP,1.0,S00-Ann,185,0.98,1-Weekday (Tu-Th),182,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,74.4926,1.0259,62,76,0.06,0.07,0.88,86,0.08
1,0006_146.9,1,1673.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.4469,AM,0.1865,1.6,3.2,2040,1800,1800,150.024330,Statewide,0,2500,COP,1.0,S00-Ann,150,0.98,1-Weekday (Tu-Th),147,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,60.1671,1.0192,49,61,0.05,0.06,0.88,69,0.07
2,0038_003.2,1,4643.0,Arterial,1.0,D1,2,3,Minor Arterial,944,0.5028,AM,0.1789,14.1,12.1,2040,0,0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.98,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0887,0,0,0.00,0.00,0.88,0,0.00
3,0038_003.2,1,4643.0,Arterial,1.0,D2,2,3,Minor Arterial,944,0.4972,AM,0.1789,12.4,10.9,2040,0,0,0.000000,Statewide,0,2500,COP,1.0,S00-Ann,0,0.98,1-Weekday (Tu-Th),0,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,0.0000,1.0793,0,0,0.00,0.00,0.88,0,0.00
4,0068_000.0,1,2450.4,Arterial,1.0,D1,1,2,Principal Arterial,1061,0.5834,AM,0.1845,4.4,5.3,2040,2300,1900,204.510870,Statewide,0,2500,COP,1.0,S00-Ann,204,0.98,1-Weekday (Tu-Th),200,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,81.8600,1.0353,69,84,0.07,0.08,0.88,96,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24325,2825_000.0,3,1415.6,Arterial,1.0,D2,1,5,Collector,580,0.4080,PM,0.2582,24.4,8.7,2040,1500,1500,158.018400,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,158,0.96,1-Weekday (Tu-Th),150,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,56.6100,1.0923,54,61,0.09,0.11,0.92,67,0.12
24326,2865_001.3,2,2406.0,Arterial,1.0,D1,1,4,Collector,785,0.5093,PM,0.2456,17.6,2.0,2040,2800,2800,350.235424,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,350,0.96,1-Weekday (Tu-Th),334,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,126.0516,1.0452,116,131,0.15,0.17,0.88,149,0.19
24327,2865_001.3,2,2406.0,Arterial,1.0,D2,1,4,Collector,785,0.4907,PM,0.2456,22.4,14.5,2040,2800,2800,337.444576,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,337,0.96,1-Weekday (Tu-Th),322,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,121.5228,1.1173,119,135,0.15,0.17,0.88,154,0.20
24328,WFRC_8071,1,25.6,Arterial,1.0,D1,1,5,Collector,577,0.7662,PM,0.2629,0.0,0.0,2040,20,20,4.028680,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,4,0.96,1-Weekday (Tu-Th),3,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,1.1322,1.0000,1,1,0.00,0.00,0.88,1,0.00


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_146.9,1,1829.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.5996,AM,0.1903,2.4,4.0,2050,2000,2000,228.207760,Statewide,0,2500,COP,1.0,S00-Ann,228,0.98,1-Weekday (Tu-Th),224,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,91.6832,1.0248,76,93,0.07,0.09,0.88,106,0.10
1,0006_146.9,1,1829.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.4004,AM,0.1903,3.6,3.3,2050,2000,2000,152.392240,Statewide,0,2500,COP,1.0,S00-Ann,152,0.98,1-Weekday (Tu-Th),149,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,60.9857,1.0237,50,62,0.05,0.06,0.88,70,0.07
2,0073_020.7,1,1129.3,Arterial,1.0,D1,1,2,Principal Arterial,1049,0.5019,AM,0.1414,14.7,7.9,2050,1900,1900,134.840454,Statewide,0,2500,COP,1.0,S00-Ann,134,0.98,1-Weekday (Tu-Th),131,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,53.6183,1.0689,46,57,0.04,0.05,0.88,65,0.06
3,0073_020.7,1,1129.3,Arterial,1.0,D2,1,2,Principal Arterial,1049,0.4981,AM,0.1414,16.9,8.9,2050,1900,1900,133.819546,Statewide,0,2500,COP,1.0,S00-Ann,133,0.98,1-Weekday (Tu-Th),130,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,53.2090,1.0783,46,57,0.04,0.05,0.88,65,0.06
4,0147_003.1,1,575.9,Arterial,1.0,D1,1,3,Minor Arterial,983,0.4783,AM,0.1597,18.2,6.8,2050,1000,950,72.565285,Statewide,0,2500,COP,1.0,S00-Ann,72,0.98,1-Weekday (Tu-Th),70,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,28.6510,1.0704,24,30,0.03,0.03,0.88,34,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24907,2825_000.0,3,1827.9,Arterial,1.0,D2,1,5,Collector,580,0.3947,PM,0.2553,20.7,9.2,2050,2000,2000,201.533820,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,201,0.96,1-Weekday (Tu-Th),192,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,72.4608,1.0874,69,78,0.12,0.14,0.92,85,0.15
24908,2865_001.3,2,2673.0,Arterial,1.0,D1,1,4,Collector,785,0.5000,PM,0.2446,16.8,3.4,2050,3100,3100,379.130000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,379,0.96,1-Weekday (Tu-Th),362,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,136.6188,1.0506,126,143,0.16,0.18,0.88,163,0.21
24909,2865_001.3,2,2673.0,Arterial,1.0,D2,1,4,Collector,785,0.5000,PM,0.2446,21.4,13.8,2050,3100,3100,379.130000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,379,0.96,1-Weekday (Tu-Th),362,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,136.6188,1.1118,134,151,0.17,0.19,0.88,172,0.22
24910,WFRC_8071,1,58.7,Arterial,1.0,D1,1,5,Collector,577,0.5530,PM,0.2785,7.2,0.0,2050,40,30,4.620315,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,4,0.96,1-Weekday (Tu-Th),3,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,1.1322,1.0144,1,1,0.00,0.00,0.88,1,0.00


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_146.9,1,1856.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.5887,AM,0.1945,2.4,3.9,2050,2000,2000,229.004300,Statewide,0,2500,COP,1.0,S00-Ann,229,0.98,1-Weekday (Tu-Th),225,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,92.0925,1.0243,76,94,0.07,0.09,0.88,107,0.10
1,0006_146.9,1,1856.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.4113,AM,0.1945,3.4,3.1,2050,2000,2000,159.995700,Statewide,0,2500,COP,1.0,S00-Ann,159,0.98,1-Weekday (Tu-Th),156,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,63.8508,1.0223,53,65,0.05,0.06,0.88,74,0.07
2,0073_020.7,1,1161.3,Arterial,1.0,D1,1,2,Principal Arterial,1049,0.5013,AM,0.1492,15.8,7.5,2050,1900,1900,142.108524,Statewide,0,2500,COP,1.0,S00-Ann,142,0.98,1-Weekday (Tu-Th),139,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,56.8927,1.0691,49,60,0.05,0.06,0.88,69,0.07
3,0073_020.7,1,1161.3,Arterial,1.0,D2,1,2,Principal Arterial,1049,0.4987,AM,0.1492,19.6,8.0,2050,1900,1900,141.371476,Statewide,0,2500,COP,1.0,S00-Ann,141,0.98,1-Weekday (Tu-Th),138,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,56.4834,1.0792,49,60,0.05,0.06,0.88,69,0.07
4,0147_003.1,1,591.9,Arterial,1.0,D1,1,3,Minor Arterial,983,0.4353,AM,0.1436,13.5,2.7,2050,1000,950,59.383626,Statewide,0,2500,COP,1.0,S00-Ann,59,0.98,1-Weekday (Tu-Th),58,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,23.7394,1.0405,20,24,0.02,0.03,0.88,28,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24895,2825_000.0,3,1812.6,Arterial,1.0,D2,1,5,Collector,580,0.3861,PM,0.2541,21.6,9.6,2050,2000,2000,196.216020,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,196,0.96,1-Weekday (Tu-Th),187,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,70.5738,1.0912,68,77,0.12,0.13,0.92,83,0.14
24896,2865_001.3,2,2646.0,Arterial,1.0,D1,1,4,Collector,785,0.5000,PM,0.2471,16.8,3.4,2050,3100,3100,383.005000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,383,0.96,1-Weekday (Tu-Th),365,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,137.7510,1.0506,127,144,0.16,0.18,0.88,164,0.21
24897,2865_001.3,2,2646.0,Arterial,1.0,D2,1,4,Collector,785,0.5000,PM,0.2471,21.4,13.8,2050,3100,3100,383.005000,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,383,0.96,1-Weekday (Tu-Th),365,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,137.7510,1.1118,135,153,0.17,0.20,0.88,174,0.22
24898,WFRC_8071,1,64.6,Arterial,1.0,D1,1,5,Collector,577,0.5334,PM,0.2538,0.0,7.4,2050,40,40,5.415077,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,5,0.96,1-Weekday (Tu-Th),4,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,1.5096,1.0370,1,1,0.00,0.00,0.88,1,0.00


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_146.9,1,1836.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6020,AM,0.1912,2.4,4.0,2050,2000,2000,230.204800,Statewide,0,2500,COP,1.0,S00-Ann,230,0.98,1-Weekday (Tu-Th),226,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,92.5018,1.0248,77,94,0.07,0.09,0.88,107,0.10
1,0006_146.9,1,1836.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3980,AM,0.1912,3.6,3.3,2050,2000,2000,152.195200,Statewide,0,2500,COP,1.0,S00-Ann,152,0.98,1-Weekday (Tu-Th),149,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,60.9857,1.0237,50,62,0.05,0.06,0.88,70,0.07
2,0073_020.7,1,1174.3,Arterial,1.0,D1,1,2,Principal Arterial,1049,0.4889,AM,0.1478,16.2,7.7,2050,1900,1900,137.292898,Statewide,0,2500,COP,1.0,S00-Ann,137,0.98,1-Weekday (Tu-Th),134,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,54.8462,1.0709,47,58,0.05,0.06,0.88,66,0.06
3,0073_020.7,1,1174.3,Arterial,1.0,D2,1,2,Principal Arterial,1049,0.5111,AM,0.1478,18.8,7.5,2050,1900,1900,143.527102,Statewide,0,2500,COP,1.0,S00-Ann,143,0.98,1-Weekday (Tu-Th),140,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,57.3020,1.0751,50,61,0.05,0.06,0.88,70,0.07
4,0147_003.1,1,593.8,Arterial,1.0,D1,1,3,Minor Arterial,983,0.4767,AM,0.1480,19.1,4.8,2050,1000,950,67.024020,Statewide,0,2500,COP,1.0,S00-Ann,67,0.98,1-Weekday (Tu-Th),65,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,26.6045,1.0622,23,28,0.02,0.03,0.88,32,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24895,2825_000.0,3,1819.0,Arterial,1.0,D2,1,5,Collector,580,0.3757,PM,0.2490,22.6,10.0,2050,2000,2000,187.098600,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,187,0.96,1-Weekday (Tu-Th),178,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,67.1772,1.0952,64,73,0.11,0.13,0.92,79,0.14
24896,2865_001.3,2,2656.0,Arterial,1.0,D1,1,4,Collector,785,0.5030,PM,0.2478,16.6,3.3,2050,3100,3100,386.394540,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,386,0.96,1-Weekday (Tu-Th),368,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,138.8832,1.0497,128,145,0.16,0.19,0.88,165,0.21
24897,2865_001.3,2,2656.0,Arterial,1.0,D2,1,4,Collector,785,0.4970,PM,0.2478,21.4,13.8,2050,3100,3100,381.785460,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,381,0.96,1-Weekday (Tu-Th),364,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,137.3736,1.1118,134,152,0.17,0.19,0.88,173,0.22
24898,WFRC_8071,1,60.7,Arterial,1.0,D1,1,5,Collector,577,0.5840,PM,0.2274,0.0,8.0,2050,40,40,5.312064,RuralRecLowVol,0,1000000,COL,1.0,S00-Ann,5,0.96,1-Weekday (Tu-Th),4,2015-2019,PM,0.3774,Prd,Season,AnnWkPM,3,1.5096,1.0400,1,1,0.00,0.00,0.88,1,0.00


In [108]:
df = dfs_VC[0]

df = df[(df[nameSegID]=='0015_282.2') * (df['VCGroupCode']=='AnnWkPM')]

df

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\computation\expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
18960,0015_282.2,3,172989.8,Arterial,0.0255,D1,1,4,Collector,783,0.3354,PM,0.4505,12.6,7.8,2019,198543,198543,764.984462,I15,0,1000000,COT,1.0,S00-Ann,764,1.09,1-Weekday (Tu-Th),834,2015-2019,PM,0.3708,Prd,Season,AnnWkPM,3,309.2472,1.0642,295,329,0.38,0.42,0.92,357,0.46
18961,0015_282.2,3,172989.8,Arterial,0.0255,D2,1,4,Collector,783,0.6646,PM,0.4505,8.9,5.6,2019,198543,198543,1515.827887,I15,0,1000000,COT,1.0,S00-Ann,1515,1.09,1-Weekday (Tu-Th),1654,2015-2019,PM,0.3708,Prd,Season,AnnWkPM,3,613.3032,1.0458,576,641,0.74,0.82,0.92,697,0.89
22078,0015_282.2,3,172989.8,Freeway,0.8616,D1,3,33,Freeway,1668,0.4722,PM,0.2136,5.7,4.3,2019,198543,198543,17253.908922,I15,0,1000000,CO6,1.0,S00-Ann,17253,1.07,1-Weekday (Tu-Th),18380,2015-2019,PM,0.3510,Prd,Season,AnnWkPM,3,6451.3800,1.0329,6328,6663,1.26,1.33,0.98,6799,1.36
22079,0015_282.2,3,172989.8,Freeway,0.8616,D2,3,33,Freeway,1668,0.5278,PM,0.2136,4.6,3.4,2019,198543,198543,19285.500062,I15,0,1000000,CO6,1.0,S00-Ann,19285,1.07,1-Weekday (Tu-Th),20545,2015-2019,PM,0.3510,Prd,Season,AnnWkPM,3,7211.2950,1.0262,7027,7400,1.40,1.48,0.98,7551,1.51
22080,0015_282.2,3,172989.8,Managed,0.1129,D1,1,38,Freeway,1704,0.4171,PM,0.3214,0.0,0.0,2019,198543,198543,3004.931553,I15,0,1000000,CO6,1.0,S00-Ann,3004,1.07,1-Weekday (Tu-Th),3200,2015-2019,PM,0.3510,Prd,Season,AnnWkPM,3,1123.2000,1.0000,1066,1123,0.63,0.66,0.92,1220,0.72
22081,0015_282.2,3,172989.8,Managed,0.1129,D2,1,38,Freeway,1703,0.5829,PM,0.3214,0.0,0.0,2019,198543,198543,4199.411657,I15,0,1000000,CO6,1.0,S00-Ann,4199,1.07,1-Weekday (Tu-Th),4473,2015-2019,PM,0.3510,Prd,Season,AnnWkPM,3,1570.0230,1.0000,1491,1570,0.88,0.92,0.95,1652,0.97


In [109]:
df_VC.columns

Index(['SEGID', 'AREATYPE', 'SegDYVol_TDM', 'FUNCGROUP', 'FGFAC', 'Dir',
       'LANES', 'FT', 'FTCLASS', 'CAP1HL', 'DFAC', 'Prd_TDM', 'PRDFAC',
       'MDPCT', 'HVPCT', 'FYEAR', 'ForecastAADT', 'ForecastAADTAdj',
       'AvgAnnualPrdVol', 'FACGEO', 'VOLUMEFROM', 'VOLUMETO', 'STATIONGROUP',
       'SsnFactor', 'SEASONGROUP', 'AvgSsnPrdVol', 'DOWFactor', 'DOWGROUP',
       'AvgSsnDOWPrdVol', 'YEARGROUP', 'PERIOD', 'VOLPKHRPCT', 'HRPCTOF',
       'SeasonType', 'VCGroupCode', 'PRDHRS', 'AvgSsnDOWPkHrVol', 'TrkFac',
       'PrdPCEFlow', 'PkHrPCEFlow', 'PrdVC', 'PkHrVC', 'PHF', 'Pk15PCEFlow',
       'Pk15VC'],
      dtype='object')

In [190]:
df1 = df_VC.groupby(['SEGID'                  ],as_index=False).agg(num=(nameDir,'size'))
df2 = df_VC.groupby(['SEGID','ForecastAADTAdj'],as_index=False).agg(num=(nameDir,'size'))
df3 = pd.DataFrame.merge(df1,df2,on='SEGID',how='outer')

In [203]:
df2g = df2.groupby(['SEGID'],as_index=False).agg(numFor=('ForecastAADTAdj','size'))
df2g[df2g['numFor']==2]


,SEGID,numFor
159,0038_003.2,2


In [204]:
df2[df2['SEGID']=='0038_003.2']

,SEGID,ForecastAADTAdj,num
159,0038_003.2,4600,6
160,0038_003.2,4900,6


In [126]:
dfs_VC_MaxDirFG = []

for df_VC in dfs_VC:

    df_VC_MaxDirFG = df_VC.groupby([nameSegID,'PERIOD','SEASONGROUP','DOWGROUP','HRPCTOF','VCGroupCode'], as_index=False).agg({nameDir:[np.size],nameCap1HL:[np.sum],'PrdPCEFlow':[np.sum],'PkHrPCEFlow':[np.sum],'Pk15PCEFlow':[np.sum],'PrdVC':[np.max],'PkHrVC':[np.max],'Pk15VC':[np.max]})

    df_VC_MaxDirFG.columns = ([nameSegID,'PERIOD','SEASONGROUP','DOWGROUP','HRPCTOF','VCGroupCode','numRecords',nameCap1HL+'Sum','PrdPCEFlowSum','PkHrPCEFlowSum','Pk15PCEFlowSum','PrdVCMax','PkHrVCMax','Pk15VCMax'])
    
    dfs_VC_MaxDirFG.append(df_VC_MaxDirFG)
    
    display(df_VC_MaxDirFG)
                                            
    #df_Peak_VC_wGroups = pd.DataFrame.merge(df_VCGroups,df_Peak_VC,on=('SeasonGroup','DOWGroup','Prd','HrPctOf'))

,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,92,114,129,0.05,0.06,0.07
1,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,83,97,110,0.04,0.05,0.05
2,0006_146.9,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,137,151,173,0.06,0.07,0.08
3,0006_149.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,152,186,212,0.08,0.10,0.12
4,0006_149.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,134,157,178,0.07,0.08,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10915,WFRC_8222,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,0,0,0,0.00,0.00,0.00
10916,WFRC_8222,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,1422,0,0,0,0.00,0.00,0.00
10917,WFRC_8223,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1422,0,0,0,0.00,0.00,0.00
10918,WFRC_8223,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,0,0,0,0.00,0.00,0.00


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,99,121,139,0.05,0.06,0.07
1,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,94,110,125,0.05,0.05,0.06
2,0006_146.9,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,150,166,189,0.07,0.08,0.09
3,0006_149.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,155,188,214,0.09,0.10,0.12
4,0006_149.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,143,163,186,0.07,0.08,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11053,WFRC_8222,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,114,134,146,0.08,0.10,0.11
11054,WFRC_8222,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,1422,463,503,547,0.33,0.36,0.39
11055,WFRC_8223,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1422,242,292,318,0.22,0.26,0.29
11056,WFRC_8223,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,237,279,303,0.17,0.20,0.22


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,103,127,144,0.05,0.06,0.07
1,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,89,105,119,0.04,0.05,0.06
2,0006_146.9,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,159,178,202,0.08,0.09,0.10
3,0006_149.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,177,216,246,0.10,0.12,0.14
4,0006_149.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,149,169,192,0.07,0.08,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11347,WFRC_8222,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,151,177,192,0.11,0.13,0.14
11348,WFRC_8222,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,1422,596,647,704,0.43,0.47,0.51
11349,WFRC_8223,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1422,323,391,424,0.31,0.38,0.41
11350,WFRC_8223,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,309,363,395,0.26,0.31,0.34


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,113,140,158,0.06,0.07,0.08
1,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,97,113,129,0.05,0.05,0.06
2,0006_146.9,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,157,175,198,0.08,0.08,0.10
3,0006_149.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,190,231,263,0.10,0.12,0.14
4,0006_149.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,170,193,220,0.08,0.09,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11545,WFRC_8222,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,176,207,225,0.13,0.15,0.16
11546,WFRC_8222,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,1422,633,686,746,0.47,0.51,0.55
11547,WFRC_8223,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1422,354,428,464,0.34,0.42,0.45
11548,WFRC_8223,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,354,416,452,0.29,0.35,0.38


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,121,149,169,0.07,0.09,0.10
1,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,109,127,144,0.05,0.06,0.07
2,0006_146.9,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,180,200,226,0.09,0.10,0.12
3,0006_149.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,274,334,379,0.16,0.19,0.22
4,0006_149.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,223,253,288,0.11,0.12,0.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11800,WFRC_8222,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,545,639,695,0.39,0.46,0.50
11801,WFRC_8222,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,1422,992,1076,1170,0.71,0.77,0.83
11802,WFRC_8223,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1422,472,571,620,0.48,0.59,0.64
11803,WFRC_8223,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,505,593,645,0.45,0.53,0.58


...

,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,103,127,144,0.05,0.06,0.07
1,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,88,103,117,0.04,0.05,0.06
2,0006_146.9,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,158,175,200,0.08,0.09,0.10
3,0006_149.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,168,205,233,0.10,0.12,0.13
4,0006_149.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,148,169,192,0.07,0.08,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11353,WFRC_8222,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,112,132,144,0.08,0.10,0.11
11354,WFRC_8222,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,1422,535,579,630,0.39,0.43,0.46
11355,WFRC_8223,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1422,293,355,386,0.29,0.35,0.38
11356,WFRC_8223,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,264,310,337,0.23,0.28,0.30


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,111,137,155,0.06,0.07,0.08
1,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,100,117,133,0.05,0.06,0.06
2,0006_146.9,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,152,169,193,0.07,0.08,0.09
3,0006_149.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,186,226,256,0.10,0.12,0.14
4,0006_149.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,165,187,213,0.08,0.09,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11554,WFRC_8222,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,342,402,437,0.25,0.29,0.31
11555,WFRC_8222,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,1422,854,927,1007,0.61,0.66,0.72
11556,WFRC_8223,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1422,514,622,676,0.56,0.68,0.74
11557,WFRC_8223,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,492,578,629,0.50,0.59,0.64


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,126,155,176,0.07,0.09,0.10
1,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,111,130,148,0.05,0.06,0.07
2,0006_146.9,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,170,189,215,0.08,0.09,0.11
3,0006_149.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,257,313,356,0.15,0.18,0.20
4,0006_149.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,228,259,295,0.11,0.12,0.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11809,WFRC_8222,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,434,510,554,0.31,0.36,0.39
11810,WFRC_8222,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,1422,757,821,893,0.56,0.60,0.66
11811,WFRC_8223,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1422,500,604,656,0.44,0.53,0.57
11812,WFRC_8223,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,421,494,537,0.41,0.49,0.53


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,129,159,181,0.07,0.09,0.10
1,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,106,124,142,0.05,0.06,0.07
2,0006_146.9,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,183,203,230,0.10,0.11,0.12
3,0006_149.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,269,327,372,0.15,0.18,0.20
4,0006_149.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,220,250,284,0.10,0.12,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11800,WFRC_8222,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,525,616,670,0.37,0.44,0.47
11801,WFRC_8222,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,1422,994,1078,1172,0.71,0.77,0.84
11802,WFRC_8223,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1422,467,565,614,0.48,0.58,0.63
11803,WFRC_8223,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,518,608,661,0.50,0.59,0.64


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,127,156,177,0.07,0.09,0.10
1,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,108,126,143,0.05,0.06,0.07
2,0006_146.9,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,183,203,230,0.09,0.10,0.12
3,0006_149.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,271,330,375,0.15,0.18,0.21
4,0006_149.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,229,259,295,0.11,0.12,0.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11800,WFRC_8222,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,541,634,689,0.39,0.45,0.49
11801,WFRC_8222,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,1422,1005,1091,1186,0.72,0.78,0.85
11802,WFRC_8223,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1422,561,678,737,0.58,0.70,0.76
11803,WFRC_8223,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1422,546,642,697,0.52,0.61,0.66


In [127]:
display(df_Scenarios)

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryCSV,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Yes,Base19,Base,2019 Base,2019,v832_SE19_Net19_Summary_SEGID_Detailed.csv,NaN,NaN
1,Yes,TIP24,TIP,2024 TIP Projects Scenario,2024,v832_SE24_Net24_Summary_SEGID_Detailed.csv,Base19,NaN
2,Yes,RTP30,RTP,2030 Fiscally-Constrained Scenario,2030,v832_SE30_Net30_Summary_SEGID_Detailed.csv,TIP24,NaN
3,Yes,RTP40,RTP,2040 Fiscally-Constrained Scenario,2040,v832_SE40_Net40_Summary_SEGID_Detailed.csv,RTP30,NaN
4,Yes,RTP50,RTP,2050 Fiscally-Constrained Scenario,2050,v832_SE50_Net50_Summary_SEGID_Detailed.csv,RTP40,NaN
...,...,...,...,...,...,...,...,...
8,Yes,UF30,Needs,2030 Needs-Based Scenario,2030,v832_Needs_SE30_Net30_Summary_SEGID_Detailed.csv,TIP24,RTP30
9,Yes,UF40,Needs,2040 Needs-Based Scenario,2040,v832_Needs_SE40_Net40_Summary_SEGID_Detailed.csv,UF30,RTP40
10,Yes,UF50,Needs,2050 Needs-Based Scenario,2050,v832_Needs_SE50_Net50_Summary_SEGID_Detailed.csv,UF40,RTP50


In [128]:
df = dfs_VC[0]

df = df[(df[nameSegID]=='0015_295.6') * (df['VCGroupCode']=='AnnWkPM')]

display(df)

df = dfs_VC_MaxDirFG[0]

df = df[(df[nameSegID]=='0015_295.6')]

pd.set_option('display.max_columns', None)

display(df)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\computation\expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
22118,0015_295.6,4,293880.7,Freeway,0.7359,D1,4,34,Freeway,1816,0.4477,PM,0.1918,8.5,5.1,2019,263955,263955,16679.552445,I15,0,1000000,CO6,1.0,S00-Ann,16679,1.07,1-Weekday (Tu-Th),17769,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,6236.919,1.0425,6174,6501,0.85,0.90,0.95,6844,0.94
22119,0015_295.6,4,293880.7,Freeway,0.7359,D2,4,34,Freeway,1816,0.5523,PM,0.1918,6.4,4.3,2019,263955,263955,20576.539682,I15,0,1000000,CO6,1.0,S00-Ann,20576,1.07,1-Weekday (Tu-Th),21920,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,7693.920,1.0343,7557,7957,1.04,1.10,0.98,8120,1.12
22120,0015_295.6,4,293880.7,Managed,0.1081,D1,1,38,Freeway,1704,0.4342,PM,0.3152,0.0,0.0,2019,263955,263955,3905.095103,I15,0,1000000,CO6,1.0,S00-Ann,3905,1.07,1-Weekday (Tu-Th),4160,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,1460.160,1.0000,1386,1460,0.81,0.86,0.95,1537,0.90
22121,0015_295.6,4,293880.7,Managed,0.1081,D2,1,38,Freeway,1704,0.5658,PM,0.3152,0.0,0.0,2019,263955,263955,5088.675286,I15,0,1000000,CO6,1.0,S00-Ann,5088,1.07,1-Weekday (Tu-Th),5420,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,1902.420,1.0000,1806,1902,1.06,1.12,0.98,1941,1.14
22122,0015_295.6,4,293880.7,CD Road,0.1561,D1,2,31,Freeway,1473,0.0000,PM,0.2012,0.0,0.0,2019,263955,263955,0.000000,I15,0,1000000,CO6,1.0,S00-Ann,0,1.07,1-Weekday (Tu-Th),0,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,0.000,1.0000,0,0,0.00,0.00,0.92,0,0.00
22123,0015_295.6,4,293880.7,CD Road,0.1561,D2,3,31,Freeway,1473,1.0000,PM,0.2012,6.9,4.4,2019,263955,263955,8290.119151,I15,0,1000000,CO6,1.0,S00-Ann,8290,1.07,1-Weekday (Tu-Th),8831,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,3099.681,1.0358,3049,3210,0.69,0.73,0.92,3489,0.79


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
225,0015_295.6,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,6,9986,17032,18911,19710,0.88,0.98,1.01
226,0015_295.6,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,6,9986,15892,17841,18986,0.89,1.00,1.03
227,0015_295.6,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,6,9986,19972,21030,21931,1.06,1.12,1.14


# VC Groups as Columns

In [129]:
df_VCGroups

,SEASONGROUP,SeasonType,DOWGROUP,HRPCTOF,PERIOD,VCGroupCode
VCGroupID,,,,,,
0,S00-Ann,Season,1-Weekday (Tu-Th),Prd,AM,AnnWkAM
1,S00-Ann,Season,1-Weekday (Tu-Th),Prd,MD,AnnWkMD
2,S00-Ann,Season,1-Weekday (Tu-Th),Prd,PM,AnnWkPM


In [130]:
df_VC_MaxDirFG_wVCGroups_15

,VCGroupCode,SEGID,Pk15VCMax
0,AnnWkAM15,0006_146.9,0.07
1,AnnWkMD15,0006_146.9,0.06
2,AnnWkPM15,0006_146.9,0.09
3,AnnWkAM15,0006_149.9,0.12
4,AnnWkMD15,0006_149.9,0.09
...,...,...,...
11056,AnnWkMD15,WFRC_8222,0.11
11057,AnnWkPM15,WFRC_8222,0.39
11058,AnnWkAM15,WFRC_8223,0.29
11059,AnnWkMD15,WFRC_8223,0.22


In [131]:
dfs_VC_MaxDirFG_wVCGroups_15 = []

for df_VC_MaxDirFG in dfs_VC_MaxDirFG:

    df_VC_MaxDirFG_wVCGroups_15 = df_VC_MaxDirFG[['VCGroupCode','SEGID','Pk15VCMax']].copy()
    
    df_VC_MaxDirFG_wVCGroups_15['VCGroupCode'] = df_VC_MaxDirFG_wVCGroups_15['VCGroupCode'] + "15"
    
    df_VC_MaxDirFG_wVCGroups_15 = df_VC_MaxDirFG_wVCGroups_15.round({'Pk15VC': 2})
    
    df_VC_MaxDirFG_wVCGroups_15 = df_VC_MaxDirFG_wVCGroups_15.pivot(index='SEGID', columns="VCGroupCode",values='Pk15VCMax')
    
    df_VC_MaxDirFG_wVCGroups_15 = df_VC_MaxDirFG_wVCGroups_15.reset_index()
    
    dfs_VC_MaxDirFG_wVCGroups_15.append(df_VC_MaxDirFG_wVCGroups_15)

    display(df_VC_MaxDirFG_wVCGroups_15)
    

VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_146.9,0.07,0.05,0.08
1,0006_149.9,0.12,0.09,0.15
2,0006_150.6,0.11,0.09,0.15
3,0006_152.6,0.12,0.10,0.15
4,0006_152.9,0.18,0.13,0.21
...,...,...,...,...
3635,WFRC_8218,0.00,0.00,0.00
3636,WFRC_8220,0.00,0.00,0.00
3637,WFRC_8221,0.00,0.00,0.00
3638,WFRC_8222,0.00,0.00,0.00


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_146.9,0.07,0.06,0.09
1,0006_149.9,0.12,0.09,0.15
2,0006_150.6,0.11,0.09,0.15
3,0006_152.6,0.13,0.10,0.17
4,0006_152.9,0.23,0.17,0.31
...,...,...,...,...
3681,WFRC_8218,0.00,0.00,0.00
3682,WFRC_8220,0.00,0.00,0.00
3683,WFRC_8221,0.07,0.06,0.14
3684,WFRC_8222,0.10,0.11,0.39


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_146.9,0.07,0.06,0.10
1,0006_149.9,0.14,0.09,0.16
2,0006_150.6,0.14,0.09,0.16
3,0006_152.6,0.13,0.10,0.18
4,0006_152.9,0.23,0.17,0.31
...,...,...,...,...
3779,WFRC_8219,0.11,0.08,0.14
3780,WFRC_8220,0.00,0.00,0.00
3781,WFRC_8221,0.10,0.09,0.15
3782,WFRC_8222,0.15,0.14,0.51


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_146.9,0.08,0.06,0.10
1,0006_149.9,0.14,0.10,0.27
2,0006_150.6,0.14,0.11,0.27
3,0006_152.6,0.15,0.12,0.24
4,0006_152.9,0.27,0.20,0.38
...,...,...,...,...
3845,WFRC_8219,0.10,0.08,0.14
3846,WFRC_8220,0.00,0.00,0.00
3847,WFRC_8221,0.09,0.10,0.17
3848,WFRC_8222,0.18,0.16,0.55


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_146.9,0.10,0.07,0.12
1,0006_149.9,0.22,0.14,0.28
2,0006_150.6,0.21,0.14,0.27
3,0006_152.6,0.20,0.16,0.28
4,0006_152.9,0.34,0.27,0.42
...,...,...,...,...
3930,WFRC_8219,0.10,0.09,0.14
3931,WFRC_8220,0.00,0.00,0.00
3932,WFRC_8221,0.27,0.27,0.66
3933,WFRC_8222,0.57,0.50,0.83


...

VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_146.9,0.07,0.06,0.10
1,0006_149.9,0.13,0.09,0.17
2,0006_150.6,0.13,0.09,0.17
3,0006_152.6,0.13,0.10,0.17
4,0006_152.9,0.23,0.17,0.30
...,...,...,...,...
3781,WFRC_8219,0.11,0.08,0.14
3782,WFRC_8220,0.00,0.00,0.00
3783,WFRC_8221,0.08,0.09,0.14
3784,WFRC_8222,0.13,0.11,0.46


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_146.9,0.08,0.06,0.09
1,0006_149.9,0.14,0.10,0.19
2,0006_150.6,0.14,0.11,0.19
3,0006_152.6,0.14,0.11,0.17
4,0006_152.9,0.27,0.20,0.33
...,...,...,...,...
3848,WFRC_8219,0.11,0.09,0.15
3849,WFRC_8220,0.01,0.01,0.02
3850,WFRC_8221,0.27,0.30,0.72
3851,WFRC_8222,0.35,0.31,0.72


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_146.9,0.10,0.07,0.11
1,0006_149.9,0.20,0.14,0.29
2,0006_150.6,0.20,0.14,0.28
3,0006_152.6,0.20,0.16,0.28
4,0006_152.9,0.34,0.27,0.42
...,...,...,...,...
3933,WFRC_8219,0.13,0.11,0.18
3934,WFRC_8220,0.00,0.00,0.00
3935,WFRC_8221,0.38,0.32,0.56
3936,WFRC_8222,0.48,0.39,0.66


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_146.9,0.10,0.07,0.12
1,0006_149.9,0.20,0.13,0.27
2,0006_150.6,0.20,0.14,0.26
3,0006_152.6,0.21,0.16,0.27
4,0006_152.9,0.36,0.27,0.41
...,...,...,...,...
3930,WFRC_8219,0.11,0.10,0.16
3931,WFRC_8220,0.00,0.00,0.00
3932,WFRC_8221,0.28,0.29,0.70
3933,WFRC_8222,0.53,0.47,0.84


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_146.9,0.10,0.07,0.12
1,0006_149.9,0.21,0.14,0.26
2,0006_150.6,0.20,0.14,0.26
3,0006_152.6,0.21,0.17,0.26
4,0006_152.9,0.36,0.27,0.41
...,...,...,...,...
3930,WFRC_8219,0.11,0.09,0.16
3931,WFRC_8220,0.00,0.00,0.00
3932,WFRC_8221,0.28,0.28,0.65
3933,WFRC_8222,0.59,0.49,0.85


In [117]:
df_VC_MaxDirFG.columns

Index(['SEGID', 'ForecastAADT', 'PERIOD', 'SEASONGROUP', 'DOWGROUP', 'HRPCTOF',
       'VCGroupCode', 'numRecords', 'CAP1HLSum', 'PrdPCEFlowSum',
       'PkHrPCEFlowSum', 'Pk15PCEFlowSum', 'PrdVCMax', 'PkHrVCMax',
       'Pk15VCMax'],
      dtype='object')

In [124]:
dfs_VC_MaxDirFG_wVCGroups_Hr = []

for df_VC_MaxDirFG in dfs_VC_MaxDirFG:

    df_VC_MaxDirFG_wVCGroups_Hr = df_VC_MaxDirFG[['VCGroupCode','SEGID','PkHrVCMax']].copy()
    
    df_VC_MaxDirFG_wVCGroups_Hr['VCGroupCode'] = df_VC_MaxDirFG_wVCGroups_Hr['VCGroupCode'] + "Hr"
    
    df_VC_MaxDirFG_wVCGroups_Hr = df_VC_MaxDirFG_wVCGroups_Hr.round({'PkHrVC': 2})
    
    df_VC_MaxDirFG_wVCGroups_Hr = df_VC_MaxDirFG_wVCGroups_Hr.pivot(index=('SEGID'), columns="VCGroupCode",values='PkHrVCMax')
    
    df_VC_MaxDirFG_wVCGroups_Hr = df_VC_MaxDirFG_wVCGroups_Hr.reset_index()
    
    dfs_VC_MaxDirFG_wVCGroups_Hr.append(df_VC_MaxDirFG_wVCGroups_Hr)

    display(df_VC_MaxDirFG_wVCGroups_Hr)
    

VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_146.9,0.06,0.05,0.07
1,0006_149.9,0.10,0.08,0.14
2,0006_150.6,0.10,0.08,0.13
3,0006_152.6,0.10,0.08,0.14
4,0006_152.9,0.16,0.11,0.19
...,...,...,...,...
3635,WFRC_8218,0.00,0.00,0.00
3636,WFRC_8220,0.00,0.00,0.00
3637,WFRC_8221,0.00,0.00,0.00
3638,WFRC_8222,0.00,0.00,0.00


ValueError: Index contains duplicate entries, cannot reshape

In [122]:
dfs_VC_MaxDirFG_wVCGroups_Prd = []

for df_VC_MaxDirFG in dfs_VC_MaxDirFG:

    df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG[df_VC_MaxDirFG['HRPCTOF'] == 'Prd'].copy()
    
    df_VC_MaxDirFG_wVCGroups_Prd['VCGroupCode'] = df_VC_MaxDirFG_wVCGroups_Prd['VCGroupCode'] + "Prd"
    
    df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG_wVCGroups_Prd[['VCGroupCode','SEGID','PrdVCMax']]
    
    df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG_wVCGroups_Prd.round({'PrdVC': 2})
    
    df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG_wVCGroups_Prd.pivot(index=('SEGID','ForecastAADT'), columns="VCGroupCode",values='PrdVCMax')
    
    df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG_wVCGroups_Prd.reset_index()
    
    dfs_VC_MaxDirFG_wVCGroups_Prd.append(df_VC_MaxDirFG_wVCGroups_Prd)

    display(df_VC_MaxDirFG_wVCGroups_Prd)

KeyError: 'ForecastAADT'

In [120]:
df_Scenarios

,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryCSV,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Yes,Base19,Base,2019 Base,2019,v832_SE19_Net19_Summary_SEGID_Detailed.csv,NaN,NaN
1,Yes,TIP24,TIP,2024 TIP Projects Scenario,2024,v832_SE24_Net24_Summary_SEGID_Detailed.csv,Base19,NaN
2,Yes,RTP30,RTP,2030 Fiscally-Constrained Scenario,2030,v832_SE30_Net30_Summary_SEGID_Detailed.csv,TIP24,NaN
3,Yes,RTP40,RTP,2040 Fiscally-Constrained Scenario,2040,v832_SE40_Net40_Summary_SEGID_Detailed.csv,RTP30,NaN
4,Yes,RTP50,RTP,2050 Fiscally-Constrained Scenario,2050,v832_SE50_Net50_Summary_SEGID_Detailed.csv,RTP40,NaN
...,...,...,...,...,...,...,...,...
8,Yes,UF30,Needs,2030 Needs-Based Scenario,2030,v832_Needs_SE30_Net30_Summary_SEGID_Detailed.csv,TIP24,RTP30
9,Yes,UF40,Needs,2040 Needs-Based Scenario,2040,v832_Needs_SE40_Net40_Summary_SEGID_Detailed.csv,UF30,RTP40
10,Yes,UF50,Needs,2050 Needs-Based Scenario,2050,v832_Needs_SE50_Net50_Summary_SEGID_Detailed.csv,UF40,RTP50


In [121]:
dfs_VC_MaxDirFG_wVCGroups = []

i=-1
for (idx, row) in df_Scenarios.iterrows():

    if row.loc['Process'] == 'Yes':

        i=i+1
        
        df_VC_MaxDirFG_wVCGroups = pd.DataFrame.merge(dfs_VC_MaxDirFG_wVCGroups_15[i],dfs_VC_MaxDirFG_wVCGroups_Hr[i],on='SEGID')
        df_VC_MaxDirFG_wVCGroups = pd.DataFrame.merge(df_VC_MaxDirFG_wVCGroups,dfs_VC_MaxDirFG_wVCGroups_Prd[i],on='SEGID')
        dfs_VC_MaxDirFG_wVCGroups.append(df_VC_MaxDirFG_wVCGroups)
        display(df_VC_MaxDirFG_wVCGroups)

VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_146.9,0.07,0.05,0.08,0.06,0.05,0.07,0.05,0.04,0.06
1,0006_149.9,0.12,0.09,0.15,0.10,0.08,0.14,0.08,0.07,0.12
2,0006_150.6,0.11,0.09,0.15,0.10,0.08,0.13,0.08,0.07,0.12
3,0006_152.6,0.12,0.10,0.15,0.10,0.08,0.14,0.08,0.07,0.12
4,0006_152.9,0.18,0.13,0.21,0.16,0.11,0.19,0.13,0.10,0.18
...,...,...,...,...,...,...,...,...,...,...
3635,WFRC_8218,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3636,WFRC_8220,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3637,WFRC_8221,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3638,WFRC_8222,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


IndexError: list index out of range

# Export final tables to CSV

In [ ]:
df_Scenarios.to_csv(os.path.join(dirResults, r'scenarios.csv'),index=False)

In [ ]:
if export_results: 
    df_Scenarios.to_csv(os.path.join(dirResults, r'scenarios.csv'),index=False)
    df_VCGroup_DOWPkHr.to_csv(os.path.join(dirResults, r'vcgroups_dowpeak.csv'),index=False)
    df_VCGroup_Seasons.to_csv(os.path.join(dirResults, r'vcgroups_seasons.csv'),index=False)
    df_VCGroups.to_csv(os.path.join(dirResults, r'vcgroups.csv'),index=False)

In [ ]:
if export_results: 

    df_Scenarios_toExport = df_Scenarios[df_Scenarios['Process']=='Yes']
    
    i=0
    for (idx, row) in df_Scenarios_toExport.iterrows():    
    
        filename = os.path.join(dirResults,"3_VC_MaxDirFG_wVCGroups_" + row.loc['ScenarioCode'] + '.csv')
        display('Exporting ' + filename)
        dfs_VC_MaxDirFG_wVCGroups[i].to_csv(filename, index=False)

        filename = os.path.join(dirResults,"2_VC_MaxDirFG_" + row.loc['ScenarioCode'] + '.csv')
        display('Exporting ' + filename)
        dfs_VC_MaxDirFG[i].to_csv(filename, index=False)

        filename = os.path.join(dirResults,"1_VC_" + row.loc['ScenarioCode'] + '.csv')
        display('Exporting ' + filename)
        dfs_VC[i].to_csv(filename, index=False)
        i=i+1

In [ ]:
for (idx, row) in df_Scenarios_toExport.iterrows():
    display(row.loc['ScenarioCode'])

In [ ]:
df = dfs_VC_MaxDirFG_wVCGroups[0]
df[df['SEGID']=='0079_003.0']
df

In [ ]:
df_Scenarios_toProcess

# Exports for WebApp

In [ ]:
if export_results:
    
    df_Scenarios_json = df_Scenarios.copy()
    df_Scenarios_json = df_Scenarios_json[['ScenarioCode','ScenarioName']]
    df_Scenarios_json.columns = ('value','label')
    df_Scenarios_json['filename'] = "VC_" + df_Scenarios_json['value'] + '.json'
    
    df_VCGroup_Seasons_json = df_VCGroup_Seasons.copy()
    df_VCGroup_Seasons_json = df_VCGroup_Seasons_json[['SeasonCode','SeasonName','SeasonType']]
    df_VCGroup_Seasons_json.columns = ('value','label','SeasonType')
        
    df_VCGroup_DOWPkHr_json = df_VCGroup_DOWPkHr.copy()
    df_VCGroup_DOWPkHr_json = df_VCGroup_DOWPkHr_json[['DOWPkCode','DOWPkName']]
    df_VCGroup_DOWPkHr_json.columns = ('value','label')

    df_Scenarios_json.to_json(os.path.join(dirResults, r'WebAppData\scenarios.json'),orient='records')
    #df_Scenarios_json = df_Scenarios_json.reset_index()
    #df_Scenarios_json.to_json(os.path.join(dirResults, r'WebAppData\scenarios_test.json'))
    df_VCGroup_Seasons_json.to_json(os.path.join(dirResults, r'WebAppData\vcgroups_seasons.json'),orient='records')
    df_VCGroup_DOWPkHr_json.to_json(os.path.join(dirResults, r'WebAppData\vcgroups_dowpeaks.json'),orient='records')
    

In [ ]:
df_Scenarios_json

In [ ]:
if export_results:
    
    df_Scenarios_toExport = df_Scenarios_toProcess.reset_index()
    
    for index, df_VC_MaxDirFG_wVCGroups in enumerate(dfs_VC_MaxDirFG_wVCGroups):    

        filename = "WebAppData\VC_" + df_Scenarios_toExport['ScenarioCode'].iloc[index] + '.json'
        #display(filename)

        #df_Peak_VC_MaxDirFG_wVCGroups = df_Peak_VC_MaxDirFG_wVCGroups.rename(columns={"SEGID": 'S'})

        #export to csv in data-processed
        df_VC_MaxDirFG_wVCGroups.to_json(os.path.join(dirResults, filename),orient='records')

        display('Done exporting ' + filename)

# Temp

In [ ]:
dfs_VC_MaxDirFG_wVCGroups =[]
df_TDMsf = []

display ('Scenario Codes:')

for (idx, row) in df_Scenarios.iterrows():    

    print(row.loc['ScenarioCode'])
    
    df_TDM = dfs_TDM[idx]
    df_TDMsf = df_TDM[['SEGID','FT']]
    
    df_TDMsfmax = df_TDMsf.groupby('SEGID', as_index=False).agg({'FT':[np.max]})
    df_TDMsfmax.columns = ['SEGID','FT']
    #df_TDMsf.append(df_TDMsf)
    
    df_VC_MaxDirFG_wVCGroups = pd.read_csv(os.path.join(dirResults, '3_VC_MaxDirFG_wVCGroups_' + row.loc['ScenarioCode'] + '.csv'))
    display(df_VC_MaxDirFG_wVCGroups)
    
    df_new = pd.DataFrame.merge(df_TDMsfmax,df_VC_MaxDirFG_wVCGroups,on=('SEGID'))
    display(df_new)
    
    filename = os.path.join(dirResults,r"temp\3_VC_MaxDirFG_wVCGroups_" + row.loc['ScenarioCode'] + '.csv')
    display('Exporting ' + filename)
    df_new.to_csv(filename,index=False)
    #dfs_VC_MaxDirFG_wVCGroups.append(dfs_Peak_VC_MaxDirFG_wVCGroups)


In [ ]:
df_TDMsfmax = df_TDMsf.groupby('SEGID', as_index=False).agg({'FT':[np.max]})
df_TDMsfmax.columns = ['SEGID','FT']
df_TDMsfmax

In [ ]:
df_VCGroup_Seasons

In [ ]:
df_TDMsfmax